In [102]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib_inline.backend_inline import set_matplotlib_formats
import numpy as np
import seaborn as sns
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import calinski_harabasz_score
from typing import Dict, Optional, Tuple, Any, List
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
from sklearn.metrics import precision_score
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
from scipy import stats
from collections import defaultdict
from math import log2
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import spearmanr
import re, gc, hashlib
from typing import Optional, Iterable
from time import perf_counter
from datetime import datetime, timedelta
from sklearn.metrics import ndcg_score
import warnings, math, json, os
from sklearn.linear_model import LogisticRegression
from typing import *
from xgboost import XGBRanker, XGBRegressor
from catboost import CatBoostRanker, CatBoostRegressor, Pool

# def _rmse(y_true, y_pred):
#     return float(np.sqrt(mean_squared_error(y_true, y_pred)))

# def _spearman(y_true, y_pred):
#     c = spearmanr(y_true, y_pred).correlation
#     return float(c) if np.isfinite(c) else np.nan

# import warnings
# warnings.filterwarnings('ignore')



In [103]:
# 한글 폰트
plt.rc("font", family = 'AppleGothic')
plt.rc("axes", unicode_minus = False)

# 글씨 선명하게
set_matplotlib_formats("retina")

In [104]:
ads_sch = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/스케줄.csv')
print(f"데이터 수 : {ads_sch.shape}")
ads_sch.head()

데이터 수 : (527696, 11)


,sch_idx,ads_idx,sch_type,mda_idx_arr,sch_sdatetime,sch_edatetime,sch_adv_pay,sch_ads_pay,sch_mda_pay,sch_clk_num,sch_turn_num
0,1257478,463989,M,"562,563",2025-09-11 09:42:03,9999-12-31 23:59:59,18,18,14,99999999,14
1,1257477,463989,A,NaN,2025-09-11 09:42:03,9999-12-31 23:59:59,18,18,12,99999999,14
2,1257476,463988,M,"562,563",2025-09-11 09:42:02,9999-12-31 23:59:59,18,18,14,99999999,18
3,1257475,463988,A,NaN,2025-09-11 09:42:02,9999-12-31 23:59:59,18,18,12,99999999,18
4,1257474,463987,M,"562,563",2025-09-11 09:41:55,9999-12-31 23:59:59,18,18,14,99999999,7


In [105]:
ads_time = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정_시간별적립보고서(최종).csv')
print(f"데이터 수 : {ads_time.shape}")
ads_time.head()

데이터 수 : (279916, 10)


,rpt_time_date,rpt_time_time,ads_idx,mda_idx,rpt_time_clk,rpt_time_turn,rpt_time_acost,rpt_time_earn,weekday,week_type
0,2025-07-27,0,11059,398,5,0,0,0,6,주말
1,2025-07-27,0,13209,583,3,0,0,0,6,주말
2,2025-07-27,0,13928,303,3,3,510,360,6,주말
3,2025-07-27,0,13928,324,3,3,510,360,6,주말
4,2025-07-27,0,13928,340,6,6,1020,720,6,주말


In [106]:
ads_domain = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/광고도메인리스트_1년치.csv')
print(f"데이터 수 : {ads_domain.shape}")
ads_domain.head()

데이터 수 : (233408, 24)


,Unnamed: 0,domain,row_id,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,ads_icon_img,ads_summary,ads_save_way,ads_day_cap,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate
0,0,금융,153088,154946,KoM69LPFUB,8,50,3,7,어센틱금융그룹 내보험조회,https://static.adbrix.igaworks.com/adpopcorn/2025/1/22/68edc7f0-e995-412d-b034-a1d368a42ad8.png,[참여 방법]\n1. 광고 참여 버튼 터치하여 이벤트 페이지로 이동\n2. 이벤트 페이지 상세 설명 확인하여 참여\n\n[유의사항]\n- 최초 참여자 대상입니다.\n- 7일 이내 참여해야 포인트가 지급됩니다.\n- 포인트 지급까지 최대 10분까지도 소요될 수 있습니다.\n- WIFI가 아닌 환경에서는 데이터 이용료가 발생할 수 있습니다.,보험조회,N,2025-02-12 10:00:00,2029-11-29 00:00:00,0,100,7,800,640,2444400,NONE,2025-02-12 10:18:46
1,21,게임,135518,137375,RBjgpLVOmR,71,910,3,8,DK모바일 리본 사전예약,https://cdnet.nasmob.com/nstation/image/20250109155855_677f73af3185d.jpg,사전예약 완료하고 리워드 받으세요.\n*최초 참여 시에만 리워드가 지급됩니다.\n\n[참여 방법]\n1. 광고 참여 버튼 터치하여 이벤트페이지 이동\n2. 사전예약 완료\n\n[유의사항]\n- 최초 참여자 대상입니다.\n- 7일 이내 참여해야 포인트가 지급됩니다.\n- 포인트 지급까지 최대 10분까지도 소요될 수 있습니다.\n- WIFI가 아닌 환경에서는 데이터 이용료가 발생할 수 있습니다.\n- 광고 추적허용 동의하지 않은 경우 적립되지 않습니다. (iOS 한정),사전예약,N,2025-01-10 18:00:00,2030-01-01 00:00:00,0,100,7,200,150,2431500,NONE,2025-01-10 18:48:52
2,23,게임,427141,429013,TUSVjcBYtK,76,1355,3,5,인생존망겜(레벨 67 달성),https://cf.smaad.net/911915789/554142578.png,"톡 하면 진화! BANG 터지는 즐거움!\n\n\n족장님, \""인생존망겜\""에 오신 것을 환영합니다!\n\n예지력을 지닌 당신은 뜻밖의 계기로 시간을 거슬러 빙하기에 도달하게 됩니다. 부족민들과 생존의 위기를 극복하고, 새로운 문명을 건설하세요!\n\n\n손끝으로 즐기는 다양한 경영\n\n채집, 사냥, 농사, 벌목, 채굴, 제련... 이 모든 걸 손끝 하나로 간단하게 처리할 수 있습니다. 부족민들과 협력하여 찬란한 문명을 완성하세요!\n\n\n시공의 문을 열자\n\n빙하기, 석기시대를 거쳐 번영의 시대에 이르기까지... 문명이 진화할수록 의식주와 산업, 부족의 모든 면이 눈에 띄게 달라집니다.\n\n각 시대에서 새로운 콘텐츠를 체험하며 흥미진진한 여정을 이어가세요.\n\n\n개조의 시대! 더 크고 더 강하게!\n\n이름 없는 작은 부족에서 초강대 제국으로의 성장까지. 이 모든 것은 당신의 손에 달려 있습니다.\n\n극한의 날씨, 예기치 못한 재난... 다양한 시련 속에서 결단력을 발휘해 위기를 극복하세요!\n\n\n전설적인 명사들 총집결\n\n역사 속 명사들을 모으고, 천하의 인재들을 영입하여 최강의 경영&전투 팀을 구성하세요!\n\n그들은 시대도, 전문 분야도 다르지만, 모두 뛰어난 능력을 갖췄죠. 이들을 전략적으로 배치하여 각자의 능력을 최대한 끌어내 보세요.\n\n\n발명과 창조, 문명의 도약\n\n시대가 발전하면 새로운 기술이 해금됩니다. 기술 트리를 활성화하여 지혜의 빛을 밝혀 보세요.\n\n석기시대에서 철기시대를 거쳐 산업혁명에 이르기까지... 각 시대의 기술을 통해 눈부신 변화를 경험하고, 문명의 도약을 이뤄내세요.\n\n▼승인 조건\n신규 앱 설치 후, 레벨 67 달성\n※ 성과도달 소요시간: 신규 앱 설치 후 약 1일 이내(18시간 정도)\n※ 광고 클릭 후(당일을 1일로 계산)14일이내에 달성\n\n▼성과도달방법\n1.신규 앱 설치 후 실행\n2.14일 이내에 레벨 67 달성\n\n▼미승인 조건\n·과거에 해당 앱을 설치한 적이 있으면 \""신규 앱 설치\""에 해당하지 않으므로 성과 대상에서 제외됩니다.\n※ 광고 클릭 후 (당일을 1일로 계산) 15일 이상 경과하여 도달한 경우, 성과 대상에서 제외됩니다.\n·공공 Wi-Fi를 사용하여 설치한 경우\n·동일한 IP 주소에서 다른 기기로 설치하거나 성과 지점에 도달한 경우\n·설치 시 네트워크가 불안정한 경우\n·설치 중 다른 작업을 한 경우\n·광고 차단 프로그램을 사용하는 경우\n·오프라인 상태에서 앱을 사용한 경우\n·에뮬레이터를 사용하여 참여한 경우\n·과거에 동일한 앱을 설치하거나 성과 지점에 도달한 적이 있는 경우\n·새로운 기기로 데이터를 이전하여 성과 조건에 도달한 경우 또는 다른 기기로 데이터를 이전하여 성과 조건에 도달한 경우\n·성과 확인 자료가 부족하거나, 부정, 허위, 중복, 장난으로 신청한 경우\n·본 광고 이외의 경로로 앱을 설치한 경우\n·문의 시 해당 앱을 삭제했거나 성과 조건 달성 전에 앱을 삭제한 경우\n·광고주가 정상적인 성과가 아니라고 판단한 경우\n·국외에서 접속하거나 국외 IP 주소로 클릭한 경우\n·기기의 \""광고 맞춤설정 사용 안 함\"" 설정이 켜져 있는 경우, 성과 대상에서 제외됩니다.\n·기기 제조사의 개인정보 보호 설정에서 앱 추적이 제한된 경우, 성과 대상에서 제외됩니다.\n·기기의 \""광고 추적 제한\"" 설정이 켜져 있거나 \""Do Not Track\"" 설정이 켜져 있는 경우, 성과 대상에서 제외됩니다.\n\n▼주의사항\n·광고 클릭 후 1시간 이내에 앱을 처음 실행하지 않으면, 성과가 반영되지 않을 수 있습니다.\n※ 최초 실행이 1시간 이상 소요될 경우, 다시 광고를 클릭해 주세요.\n·동일한 앱에서의 광고 참여는 1인당 1회만 가능합니다. (OS가 달라도 1회만 참여 가능)\n※ 다른 성과 지점에서 광고에 참여한 적이 있더라도 성과가 인정되지 않습니다.\n·과거에 동일 앱으로 광고에 참여한 이력이 있는 경우, 다른 기기로 최초 설치를 하더라도 성과가 반영되지 않습니다.\n·성과 달성 후 포인트가 적립되기까지 최대 24시간이 소요될 수 있습니다.\n·앱 내 캠페인(이벤트)은 예고 없이 변경되거나 종료될 수 있습니다.\n·기기의 \""앱 트래킹 요청 허용\"" 설정이 꺼져 있는 경우, 포인트를 획득하지 못할 수 있습니다.\n·URL을 다른 사용자와 공유하거나 복사한 URL을 사용하는 것을 삼가해 주세요. 복사한 URL을 통한 설치는 성과 대상에서 제외됩니다.\n·광고 ID를 의도적으로 삭제하거나 초기화하는 경우, 부정 이용으로 간주될 수 있으므로 삭제나 초기화를 삼가해 주세요.\n·앱 내 버그 관련 문의 시 광고 ID 제공을 요청드릴 수 있습니다.\n※ 제공이 불가한 경우, 조사가 어려울 수 있습니다.\n\n【라이브 종료후 성과접수기간】종료일을 포함하여 14일간\n※상기 기간 내에 성과조건을 달성하지 않았을 경우 미승인 처리됩니다.\n\n▼성과 조사에 대해서\n·본 조사는 앱 설치를 전제로 성과를 확인합니다. \n 광고를 통해 앱 설치가 이루어지지 않으면, 성과 조건을 달성하더라도 포인트 지급 대상에서 제외됩니다.\n·성과 요건을 달성했음에도 포인트가 지급되지 않을 경우, 본 사이트의 문의 폼을 통해 연락해 주세요.\n·스폰서 사이트에 직접 문의하는 것은 삼가해 주세요.\n·클릭 발생일로부터 60일이 지나면 조사가 불가합니다.\n·성과 조사가 완료되기 전에 앱을 삭제하면 성과 조사 대상에서 제외됩니다.\n

In [107]:
ads_list = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정2_광고목록.csv')
print(f"데이터 수 : {ads_list.shape}")
ads_list.head()

데이터 수 : (445039, 21)


,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,ads_icon_img,ads_summary,ads_save_way,ads_day_cap,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate
0,160,tr71opm6C1,6,10,1,1,리니지레드나이츠,https://lh3.googleusercontent.com/oDGXxngO9oOPImzdzn5lewYXGUy_qh45O4l-FdQ4K0zibGnQEXcIK4ekA2RQ1hhNpNs=w300,"[앱설명]<br />[설치형] 매력만점 영웅들과 소환수들이 함께하는 취향저격 RPG! ‘리니지 레드나이츠’<br /><br />[적립방법]\n구글 마켓 이동 > 리니지 레드나이츠 설치 후 설치확인\n\n[게임소개]\n2016년을 마무리할 강한 녀석이 온다! \n매력만점 영웅들과 소환수들이 함께하는 취향저격 RPG! ‘리니지 레드나이츠’\n꿀잼보장 탐험모드, 더욱 끈끈해진 혈맹 그리고 치열한 혈맹 전투 점령전까지! \n혼자 하는 게임은 이제 그만! \n함께하는 즐거움이 가득한 레드나이츠로 우리 함께 탐험을 떠나볼까요?\n#레드나이츠는처음이지? #지금당장다운로드 #일단은혈맹가입\n■■■■■ 게임 특징 ■■■■■\n\n▶ 볼거리 가득한 시네마틱 스토리라인\n- 데스나이트가 이제 내 편?! 레전드 몬스터들의 유쾌한 반전!\n- 오랫동안 숨겨져 온 이야기가 영화처럼 펼쳐진다!\n\n▶ 함께하는 즐거움이 있는 혈맹과 커뮤니티\n- Since 1998, MMORPG의 정수만을 담았다!\n- 다채로운 혈맹 콘텐츠와 한층 강화된 커뮤니티\n\n▶ 단계별로 진화하는 대규모 PvP 전장\n- 혈맹원들과 함께하는 필드 쟁탈, 점령전\n- 손 끝 떨리는 한 판 승부, 요새전\n- 혈맹 연합 vs 혈맹 연합 Coming Soon 공성전\n\n▶ 끝없이 도전할 수 있는 다양한 게임 모드\n- 시원하게 한 판 붙자! ‘5인 콜로세움’\n- ALL-IN! 갈 때까지 가보자! ‘오만의 탑’\n- 환상의 드래곤 레이드! ‘안타라스’를 공략하라\n\n▶ 매일 새로움을 주는 이벤트 시스템\n- 쿠작! 놓칠 수 없는 보너스 타임!\n- 스토리 탐험의 오아시스, 신전 발견\n- 매일 새로운 요일던전과 화끈한 보상의 스페셜던전까지!\n\n\n■■■■■ 공식 페이스북 ■■■■■ https://www.facebook.com/LineageRedKnights/\n\n* 게임 이용을 위해서는 아래의 접근 권한을 필요로 합니다.\n(게임의 각종 설정과 캐시 저장을 위해 필요)\n- READ_EXTERNAL_STORAGE\n- WRITE_EXTERNAL_STORAGE\n----\n개발자 연락처 :\n(주)엔씨소프트: 경기도 성남시 분당구 판교로 644번길 12\n전화: 1600-0020\n사업자등록번호: 144-85-04244\n통신판매업 신고번호: 제 2013-경기성남-1268 호\n통신판매업 신고기관: 성남시",받기,N,2016-12-01 00:00:00,9999-12-31 23:59:59,0,100,2,170,150,982800,NONE,2017-04-06 13:19:53
1,284,xlNpfkgtLX,6,10,1,1,강철의함대:Ocean Overlord,http://nextapps-nas.aws.appang.kr/icon/20161229124815888767968_1482983323.png,"[앱설명]<br />[설치형] 한, 중, 일 아시아 유저들과 함께 대규모 해상 전투를 즐겨라!<br /><br />[적립 방법]\n구글플레이 이동 > 강철의 함대 다운로드 후 설치확인\n\n[게임 소개]\nVICTORY FOR KOREA! 강철의함대:Ocean Overlord!!\n한, 중, 일 아시아 유저들과 함께 대규모 해상 전투를 즐겨라! \n■전 서버 HOT 이벤트 진행 중■\n1. 로그인 2일 차에 S급 비스마르크급, 7일 차에는 특급함장영입카드 100% 증정\n2. 첫 충전하면 S급 비스마르크급, 그라프체펠린급 함선과 S급 되니츠 함장, 고급설계도*4, 연료(중)*5 증정!\n3. 5, 10, 15....레벨업 할 때마다 다양한 아이템 증정!\n\n■나만의 함대를 이끌고 승리하라!\n1, 2차 세계대전에서 활약한 역사 속 \n유명 함대를 이끌고 펼치는 대규모 리얼 해상 전투.\n\n■36 VS 36 역대급 해상 전투!\n최대 72대 함선들의 뜨거운 해상 전투를 즐기세요.\n\n■나만의 함대 운영! \n나만의 전략적 함대 운영으로 \n아시아 지역의 No.1 제독이 되어보세요.\n\n■뺏고 뺏기는 치열한 전투! \n역사적 사실을 바탕으로 펼쳐지는 \nPVP, PVE, GVG 등 다양한 전투를 즐기세요.\n\n■해상 전투 게임의 끝판왕!\n100% 실사에 가깝게 구현 된 함선, 함장 컬렉션! \n차원이 다른 글로벌 해상 전투 시뮬레이션 게임을 즐기세요.\n\n■강철의함대:Ocean Overlord 공식카페에서 \n대한민국 제독님들의 승리 전략과 최신 게임 정보를 가장 먼저 만나보세요! \n공식카페: http://cafe.naver.com/oceanoverlord\n\n=========================================\n■ 상품 정보 및 이용 조건 안내\n* 유료아이템 구매 시 별도의 요금이 부과됩니다.\n* 구매 후 사용된 상품 및 즉시 반영되는 월정액 상품은 환불이 불가능합니다.\n* 결제 후 상품이 미지급 되었을 경우, 고객센터로 문의 바랍니다.\n* 결제 취소는 '게임펍 고객센터'를 통해서만 처리되며, 회수하지 않은 아이템에 대한 결제 취소 적발 시 영구 제재가 적용됩니다.\n* 게임펍 고객센터: 1666-7198/ 평일 10:00 ~ 19:00 시간 동안 상담 가능\n\n------------------------------------------\n* 주소: 서울특별시 용산구 원효로 204(원효로 2가) \n* 사업자 번호: 279-81-00148\n* 통신 판매업 번호: 제 2015-서울용산-00649 호\n----\n개발자 연락처 :\n서울특별시 용산구 원효로 204(원효로 2가)",받기,N,2016-12-29 00:00:00,9999-12-31 23:59:59,0,100,2,180,160,987200,NONE,2017-01-10 10:37:22
2,292,7Hj8V5QKiy,6,10,1,1,스노우 SNOW,http://nextapps-nas.aws.appang.kr/icon/20160923110742923603784_1474596474.png,"[앱설명]<br />[설치형] 셀카, 얼굴인식 스티커, 꿀잼 카메라<br /><br />[적립 방법]\n구글 마켓 이동 > SNOW 다운로드 후 설치확인\n\n[앱 소개]\nSNOW에서 친구들과 재밌는 영상 메시지를 주고 받으세요!\n\n얼굴인식 스티커와 다양한 효과로 꿀잼 영상을 찍어 친구에게 보내세요.\n친구와 채팅도 하고 다양한 표정으로 움짤을 찍어 보낼 수 있어요.\n내 스토리에 영상을 올리고 더 많은 친구들과 일상을 공유해보세요!\n\n\n■ SNOW를 즐기는 다양한 방법\n\n1. 얼굴 인식 스티커와 끝판왕 필터\n인스타를 도배한 귀요미 강아지, 볼빵빵 다람쥐 스티커 등 SNOW에서만 만날 수 있는 꿀잼스티커로 숨겨 왔던 비글미 대방출! 자동 보정 필터로 사진, 동영상은 더욱 근사하게!\n\n2. 그냥 채팅 NO, 움짤채팅!\nㅋㅋㅋ, ㅠㅠ보다 100배 실감나는 움짤메시지! 지금 그 기분, 움짤로 찍어 채팅방에 보내 보세요.\n\n3. 내 일상을 공유하는 새로운 방식, 스토리\n스토리에 영상을 올리면, 나의 하루가 일어난 순서대로 영화처럼 이어져서 보여요. 48시간 후에는 자동으로 사라져서 부담 없이 얼마든지 올릴 수 있어요!\n\n4. 하루 만에 사라지는 펑!메시지\nSNOW에서 영상 메시지를 보내면 친구가 단 하루 동안만 다시 보기 할 수 있어요! 이제 엽사도, 초근접셀카도 두려워하지 말고 보내세요! (친구가 캡

In [108]:
# 문자열 출력 제한 해제
pd.set_option("display.max_colwidth", None)

In [109]:
pd.reset_option("display.max_colwidth")

In [110]:
ads_list[ads_list['ads_idx'] == 34045]

,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,ads_icon_img,ads_summary,ads_save_way,ads_day_cap,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate
32209,34045,IhvxEGKBOh,8,50,10,3,이마트 유튜브,https://static.adbrix.igaworks.com/adpopcorn/2...,이마트 유튜브 채널 \'구독 및 좋아요\'하면 리워드 지급\n(20~59세만 참여 ...,구독 및 좋아요,N,2024-04-19 15:00:00,2029-11-01 00:00:00,20,59,7,230,160,2729400,NONE,2024-04-19 15:31:37


In [111]:
ads_rwd = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정_광고적립.csv')
print(f"데이터 수 : {ads_rwd.shape}")
ads_rwd.head()

데이터 수 : (1475031, 14)


,rwd_idx,ads_idx,ads_code,mda_idx,pub_sub_rel_id,dvc_idx,advid,click_key,adv_cost,earn_cost,click_date,regdate,ctit,cost_diff
0,238304818,413490,CFZ6lzEmeD,22,27496512,32947806,09b6a808-4f76-4584-87c5-626f21f472a0,b0e22dba943810794d25c993dff425c4e16d0744,190,120,2025-07-26 00:00:31,2025-07-26 00:05:02,271,70
1,238304829,413490,CFZ6lzEmeD,22,27496512,47387244,ba4fd0c8-4f06-4dfa-9c3f-c6b39ee7066e,0679ff4c6a14beba296e44d7ffdd88129f42ffbf,190,120,2025-07-26 00:00:32,2025-07-26 00:08:17,465,70
2,238304816,413490,CFZ6lzEmeD,22,27496512,32425124,64c675d0-8edd-4478-9906-a806651fdf92,68eb28d636724061da21e490a1afff6a39cd78f7,190,120,2025-07-26 00:00:42,2025-07-26 00:04:43,241,70
3,238304817,413490,CFZ6lzEmeD,22,27496513,34006545,a08ebb04-e70a-4492-89d4-17fa96bf8c75,37da42f12b6f732d8c8bf1a609c01e569b7f8a09,190,120,2025-07-26 00:00:54,2025-07-26 00:04:45,231,70
4,238304802,133653,rl8ud4uNQC,801,1,56867331,dbe54f40-41b5-4423-ae2f-048fd6ecf5af,624b1bbc57c9ea81005c74e1956d48e76d974816,260,195,2025-07-26 00:01:07,2025-07-26 00:02:04,57,65


In [112]:
ads_info = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정2_광고참여정보.csv')
print(f"데이터 수 : {ads_info.shape}")
ads_info.head()

데이터 수 : (16854865, 14)


,click_key,ads_idx,dvc_idx,mda_idx,pub_sub_rel_id,contract_price,media_price,click_day,click_time,click_date,exp_day,network,user_ip,converted
0,000000d54b9faad47ee99d6cd3cf53894dd4baa5,313780,61906528,539,144350110,6000,4500,2025-08-17,21,2025-08-17 21:07:37,2025-09-16,0,16.184.28.219,0
1,000002b4d92f7648b455877c2676452efcd22a09,412426,34422806,58,46032732,180,170,2025-07-26,2,2025-07-26 02:18:24,2025-08-25,NaN,35.78.117.76,1
2,0000057e97361ff3d0263aaecee34cfaa3ba30fb,443660,38366075,808,1,170,120,2025-08-12,18,2025-08-12 18:17:59,2025-09-11,WIFI,39.7.55.192,1
3,00000607f60139015da3ee1dd5499db3faa100dc,360192,61894110,539,144350110,6000,4500,2025-08-17,3,2025-08-17 03:35:02,2025-09-16,0,211.252.100.157,0
4,0000066bc25d4a6d147c27326cf972a4de88024e,372307,61956954,539,144350110,15600,11700,2025-08-18,8,2025-08-18 08:01:26,2025-09-17,0,3.38.148.211,0


In [113]:
ads_join = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/참여적립조인.csv')
print(f"데이터 수 : {ads_join.shape}")
ads_join.head()

데이터 수 : (16854865, 22)


,Unnamed: 0,click_key,ads_idx,dvc_idx,mda_idx,pub_sub_rel_id,contract_price,media_price,click_day,click_time,click_date,exp_day,network,user_ip,rwd_idx,ads_code,advid,adv_cost,earn_cost,regdate,ctit,conversion
0,0,000000d54b9faad47ee99d6cd3cf53894dd4baa5,313780,61906528,539,144350110,6000,4500,2025-08-17,21,2025-08-17 21:07:37,2025-09-16,0,16.184.28.219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,000002b4d92f7648b455877c2676452efcd22a09,412426,34422806,58,46032732,180,170,2025-07-26,2,2025-07-26 02:18:24,2025-08-25,NaN,35.78.117.76,238313226.0,yuRe4YoPuZ,40ceb01c-9b60-4452-bacd-0e061fb18ab4,180.0,170.0,2025-07-26 02:19:21,57.0,1
2,2,0000057e97361ff3d0263aaecee34cfaa3ba30fb,443660,38366075,808,1,170,120,2025-08-12,18,2025-08-12 18:17:59,2025-09-11,WIFI,39.7.55.192,239017177.0,WuIp8HKmt9,73793f44-a471-4351-ae1c-d09accaf223a,170.0,120.0,2025-08-12 18:19:21,82.0,1
3,3,00000607f60139015da3ee1dd5499db3faa100dc,360192,61894110,539,144350110,6000,4500,2025-08-17,3,2025-08-17 03:35:02,2025-09-16,0,211.252.100.157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,4,0000066bc25d4a6d147c27326cf972a4de88024e,372307,61956954,539,144350110,15600,11700,2025-08-18,8,2025-08-18 08:01:26,2025-09-17,0,3.38.148.211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [114]:
ads_segment = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/ads_segment.csv')
print(f"데이터 수 : {ads_segment.shape}")
ads_segment.head()

데이터 수 : (9517, 40)


,Unnamed: 0,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,ctit_median,ads_rejoin_type,contract_price,media_price,first_click,last_click,ads_name,ads_sdate,expire,days_active,daily_avg_conversions,cvr,margin,roi,total_net_return,ads_level_detailed,daily_clicks,daily_users,scale_level,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster
0,0,6508,1,1,2,0,10,금융,7,2381500,NaN,NaN,NONE,1400,1100,2025-07-29 08:54:30,2025-07-29 08:55:55,스마트피싱보호,2020-04-23 17:00:00,0,1,0.0,0.0,300,0.3,0,MEDIUM,2.000000,1.000000,SMALL,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0
1,1,6985,2,30,46,0,7,금융,7,1201938,NaN,NaN,NONE,2700,2300,2025-07-26 11:02:18,2025-08-24 05:39:13,DB손해보험 다이렉트,2020-06-30 15:00:00,0,29,0.0,0.0,400,0.2,0,MEDIUM,1.586207,1.034483,MEDIUM,LOW,MEDIUM_LOW,2,0,0,3,0,5,MEDIUM,0
2,2,8327,1,6,11,0,10,미디어/컨텐츠,7,1673458,NaN,NaN,NONE,3200,2420,2025-08-02 08:30:52,2025-08-18 22:32:25,파일썬,2020-11-10 11:21:00,0,17,0.0,0.0,780,0.3,0,MEDIUM,0.647059,0.352941,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0
3,3,9264,1,1,1,0,10,금융,7,2372800,NaN,NaN,NONE,11000,8250,2025-08-14 13:54:12,2025-08-14 13:54:12,하나 가득담은 운전자보험가입,2020-11-30 00:00:00,0,1,0.0,0.0,2750,0.3,0,LARGE,1.000000,1.000000,SMALL,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0
4,4,9716,2,4,6,1,8,생활,2,2699900,74.0,74.0,NONE,600,336,2025-07-28 01:22:36,2025-07-29 07:14:36,베이비러브 참여 신청,2021-01-11 16:00:00,0,2,0.5,0.2,264,0.8,264,MEDIUM,3.000000,2.000000,MEDIUM,LOW,MEDIUM_LOW,2,0,1,1,0,4,SMALL,0


In [115]:
ads_segment[ads_segment['ads_idx'] == 55718]

,Unnamed: 0,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,ctit_median,ads_rejoin_type,contract_price,media_price,first_click,last_click,ads_name,ads_sdate,expire,days_active,daily_avg_conversions,cvr,margin,roi,total_net_return,ads_level_detailed,daily_clicks,daily_users,scale_level,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster
265,265,55718,6,755,1735,358,1,뷰티,1,2749800,141.041899,66.0,NONE,230,180,2025-07-26 00:44:14,2025-08-25 09:16:39,여신티켓,2024-06-24 13:00:00,0,31,11.548387,0.2,50,0.3,17900,LARGE,55.967742,24.354839,MEGA,LOW,MEGA_LOW,3,2,2,4,0,11,LARGE,0


In [116]:
media_portfolio = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/media_portfolio.csv')
print(f"데이터 수 : {media_portfolio.shape}")
media_portfolio.head()

데이터 수 : (189, 50)


,Unnamed: 0,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,LARGE,MEDIUM,MEGA,SMALL,total_ads,MEGA_ratio,LARGE_ratio,MEDIUM_ratio,SMALL_ratio,ads_category_0_pct,ads_category_1_pct,ads_category_2_pct,ads_category_3_pct,ads_category_4_pct,ads_category_5_pct,ads_category_6_pct,ads_category_7_pct,ads_category_8_pct,ads_category_10_pct,ads_category_11_pct,ads_category_13_pct,ads_os_type_1_pct,ads_os_type_2_pct,ads_os_type_3_pct,ads_os_type_7_pct,domain_게임_pct,domain_교육_pct,domain_금융_pct,domain_기타_pct,domain_미디어/컨텐츠_pct,domain_뷰티_pct,domain_비영리/공공_pct,domain_생활_pct,domain_식당/카페_pct,domain_식음료_pct,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit
0,0,12,75700,232438,82938,2025-07-26 00:00:02,2025-08-25 11:22:16,31,2675.419355,7,0,17,0,24,70.8,29.2,0.0,0.0,8.333333,12.500000,25.000000,29.166667,0.0,0.000000,0.0,0.000000,25.000000,0.000000,0.0,0.0,0.000000,70.833333,0.000000,29.166667,29.166667,0.00000,4.166667,0.000000,4.166667,0.000000,16.666667,12.500000,0.00000,16.666667,0.000000,0.000000,4.166667,0.000000,12.500000,3003668
1,1,14,31834,84763,22983,2025-07-26 00:01:22,2025-08-25 11:22:40,31,741.387097,31,5,22,1,59,37.3,52.5,8.5,1.7,0.000000,20.338983,18.644068,8.474576,0.0,25.423729,0.0,3.389831,22.033898,1.694915,0.0,0.0,42.372881,38.983051,0.000000,18.644068,44.067797,0.00000,11.864407,3.389831,8.474576,5.084746,0.000000,13.559322,0.00000,3.389831,0.000000,0.000000,3.389831,1.694915,5.084746,2733547
2,2,18,2825,5961,250,2025-07-26 00:00:49,2025-08-25 11:13:19,31,8.064516,19,0,4,0,23,17.4,82.6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,60.869565,0.0,0.000000,30.434783,8.695652,0.0,0.0,0.000000,82.608696,0.000000,17.391304,60.869565,0.00000,13.043478,0.000000,13.043478,0.000000,0.000000,8.695652,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,4.347826,182366
3,3,22,37668,83355,35232,2025-07-26 00:00:31,2025-08-25 11:14:50,31,1136.516129,67,58,33,8,166,19.9,40.4,34.9,4.8,8.433735,6.626506,10.843373,5.421687,0.0,10.240964,0.0,6.626506,26.506024,25.301205,0.0,0.0,0.000000,40.361446,3.012048,56.626506,19.277108,0.60241,26.506024,0.000000,10.240964,1.807229,3.012048,11.445783,0.60241,4.216867,1.204819,7.831325,4.819277,1.204819,7.228916,2541770
4,4,26,402,678,70,2025-07-26 01:45:31,2025-08-25 10:22:36,31,2.258065,10,5,3,1,19,15.8,52.6,26.3,5.3,0.000000,0.000000,15.789474,0.000000,0.0,47.368421,0.0,0.000000,26.315789,10.526316,0.0,0.0,78.947368,0.000000,0.000000,21.052632,52.631579,0.00000,15.789474,0.000000,10.526316,0.000000,0.000000,10.526316,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,10.526316,2690


In [117]:
ads_new = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/신규가상광고.csv')
print(f"데이터 수 : {ads_new.shape}")
ads_new.head()

데이터 수 : (8, 10)


,ads_idx,ads_type,ads_category,ads_name,ads_os_type,ads_contract_price,ads_media_price,ads_reward_price,ads_rejoin_type,domain
0,500000,12,11,청정원 선물세트,7,6181,4000,4000,REJOINABLE,식음료
1,500001,3,4,옹짬뽕 포항양덕본점,7,18,14,12,ADS_CODE_DAILY_UPDATE,식당/카페
2,500002,3,4,추석선물세트 (리뷰 전체보기 클릭 후 URL 제출),7,17,8,8,NONE,식당/카페
3,500003,11,3,네이버 뉴스 제주뉴스 구독,7,230,160,180,NONE,미디어/컨텐츠
4,500004,7,3,폴라베어 인스타그램 구독,1,230,160,180,NONE,커머스


In [118]:
media_performance_classification = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/media_performance_classification.csv')
print(f"데이터 수 : {media_performance_classification.shape}")
media_performance_classification.head(20)

데이터 수 : (188, 55)


,Unnamed: 0.1,Unnamed: 0,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,LARGE,MEDIUM,MEGA,SMALL,total_ads,MEGA_ratio,LARGE_ratio,MEDIUM_ratio,SMALL_ratio,ads_category_0_pct,ads_category_1_pct,ads_category_2_pct,ads_category_3_pct,ads_category_4_pct,ads_category_5_pct,ads_category_6_pct,ads_category_7_pct,ads_category_8_pct,ads_category_10_pct,ads_category_11_pct,ads_category_13_pct,ads_os_type_1_pct,ads_os_type_2_pct,ads_os_type_3_pct,ads_os_type_7_pct,domain_게임_pct,domain_교육_pct,domain_금융_pct,domain_기타_pct,domain_미디어/컨텐츠_pct,domain_뷰티_pct,domain_비영리/공공_pct,domain_생활_pct,domain_식당/카페_pct,domain_식음료_pct,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit,conversion_rate,last_click_dt,classification,basic_classification
0,0,0,12,75700,232438,82938,2025-07-26 00:00:02,2025-08-25 11:22:16,31,2675.419355,7,0,17,0,24,70.8,29.2,0.0,0.0,8.333333,12.500000,25.000000,29.166667,0.000000,0.000000,0.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,70.833333,0.000000,29.166667,29.166667,0.000000,4.166667,0.000000,4.166667,0.000000,16.666667,12.500000,0.000000,16.666667,0.000000,0.000000,4.166667,0.000000,12.500000,3003668,0.356818,2025-08-25 11:22:16,특화전문형_MEGA특화,특화전문형
1,1,1,14,31834,84763,22983,2025-07-26 00:01:22,2025-08-25 11:22:40,31,741.387097,31,5,22,1,59,37.3,52.5,8.5,1.7,0.000000,20.338983,18.644068,8.474576,0.000000,25.423729,0.000000,3.389831,22.033898,1.694915,0.000000,0.000000,42.372881,38.983051,0.000000,18.644068,44.067797,0.000000,11.864407,3.389831,8.474576,5.084746,0.000000,13.559322,0.000000,3.389831,0.000000,0.000000,3.389831,1.694915,5.084746,2733547,0.271144,2025-08-25 11:22:40,안정공급형,안정공급형
2,2,2,18,2825,5961,250,2025-07-26 00:00:49,2025-08-25 11:13:19,31,8.064516,19,0,4,0,23,17.4,82.6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,60.869565,0.000000,0.000000,30.434783,8.695652,0.000000,0.000000,0.000000,82.608696,0.000000,17.391304,60.869565,0.000000,13.043478,0.000000,13.043478,0.000000,0.000000,8.695652,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.347826,182366,0.041939,2025-08-25 11:13:19,특화전문형_게임특화,특화전문형
3,3,3,22,37668,83355,35232,2025-07-26 00:00:31,2025-08-25 11:14:50,31,1136.516129,67,58,33,8,166,19.9,40.4,34.9,4.8,8.433735,6.626506,10.843373,5.421687,0.000000,10.240964,0.000000,6.626506,26.506024,25.301205,0.000000,0.000000,0.000000,40.361446,3.012048,56.626506,19.277108,0.602410,26.506024,0.000000,10.240964,1.807229,3.012048,11.445783,0.602410,4.216867,1.204819,7.831325,4.819277,1.204819,7.228916,2541770,0.422674,2025-08-25 11:14:50,대량처리형,대량처리형
4,4,4,26,402,678,70,2025-07-26 01:45:31,2025-08-25 10:22:36,31,2.258065,10,5,3,1,19,15.8,52.6,26.3,5.3,0.000000,0.000000,15.789474,0.000000,0.000000,47.368421,0.000000,0.000000,26.315789,10.526316,0.000000,0.000000,78.947368,0.000000,0.000000,21.052632,52.631579,0.000000,15.789474,0.000000,10.526316,0.000000,0.000000,10.526316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.526316,2690,0.103245,2025-08-25 10:22:36,특화전문형_게임특화,특화전문형
5,5,5,30,3606,5160,391,2025-07-26 00:42:21,2025-08-25 11:06:44,31,12.612903,14,37,9,0,60,15.0,23.3,61.7,0.0,0.000000,5.000000,1.666667,8.333333,0.000000,38.333333,0.000000,3.333333,20.000000,23.333333,0.000000,0.000000,0.000000,58.333333,0.000000,41.666667,41.666667,1.666667,18.333333,0.000000,11.666667,1.666667,3.333333,8.333333,0.000000,1.666667,0.000000,3.333333,1.666667,3.333333,3.333333,22520,0.075775,2025-08-25 11:06:44,안정공급형(후보),안정공급형(후보)
6,6,6,32,870,1574,163,2025-07-26 10:48:49,2025-08-25 10:38:39,30,5.433333,5,0,2,0,7,28.6,71.4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.285714,71.428571,14.285714,0.000000,0.000000,0.000000,57.142857,0.000000,42.857143,0.000000,0.000000,14.285714,0.000000,28.571429,0.000000,0.000000,28.571429,0.000000,0.000000,0.000000,0.000000,14.285714,0.000000,14.285714,191080,0.103558,2025-08-25 10:38:39,관리 필요,관리 필요
7,7,7,54,6395,12838,4299

In [119]:
media_performance_classification.head(20)

,Unnamed: 0.1,Unnamed: 0,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,LARGE,MEDIUM,MEGA,SMALL,total_ads,MEGA_ratio,LARGE_ratio,MEDIUM_ratio,SMALL_ratio,ads_category_0_pct,ads_category_1_pct,ads_category_2_pct,ads_category_3_pct,ads_category_4_pct,ads_category_5_pct,ads_category_6_pct,ads_category_7_pct,ads_category_8_pct,ads_category_10_pct,ads_category_11_pct,ads_category_13_pct,ads_os_type_1_pct,ads_os_type_2_pct,ads_os_type_3_pct,ads_os_type_7_pct,domain_게임_pct,domain_교육_pct,domain_금융_pct,domain_기타_pct,domain_미디어/컨텐츠_pct,domain_뷰티_pct,domain_비영리/공공_pct,domain_생활_pct,domain_식당/카페_pct,domain_식음료_pct,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit,conversion_rate,last_click_dt,classification,basic_classification
0,0,0,12,75700,232438,82938,2025-07-26 00:00:02,2025-08-25 11:22:16,31,2675.419355,7,0,17,0,24,70.8,29.2,0.0,0.0,8.333333,12.500000,25.000000,29.166667,0.000000,0.000000,0.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,70.833333,0.000000,29.166667,29.166667,0.000000,4.166667,0.000000,4.166667,0.000000,16.666667,12.500000,0.000000,16.666667,0.000000,0.000000,4.166667,0.000000,12.500000,3003668,0.356818,2025-08-25 11:22:16,특화전문형_MEGA특화,특화전문형
1,1,1,14,31834,84763,22983,2025-07-26 00:01:22,2025-08-25 11:22:40,31,741.387097,31,5,22,1,59,37.3,52.5,8.5,1.7,0.000000,20.338983,18.644068,8.474576,0.000000,25.423729,0.000000,3.389831,22.033898,1.694915,0.000000,0.000000,42.372881,38.983051,0.000000,18.644068,44.067797,0.000000,11.864407,3.389831,8.474576,5.084746,0.000000,13.559322,0.000000,3.389831,0.000000,0.000000,3.389831,1.694915,5.084746,2733547,0.271144,2025-08-25 11:22:40,안정공급형,안정공급형
2,2,2,18,2825,5961,250,2025-07-26 00:00:49,2025-08-25 11:13:19,31,8.064516,19,0,4,0,23,17.4,82.6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,60.869565,0.000000,0.000000,30.434783,8.695652,0.000000,0.000000,0.000000,82.608696,0.000000,17.391304,60.869565,0.000000,13.043478,0.000000,13.043478,0.000000,0.000000,8.695652,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.347826,182366,0.041939,2025-08-25 11:13:19,특화전문형_게임특화,특화전문형
3,3,3,22,37668,83355,35232,2025-07-26 00:00:31,2025-08-25 11:14:50,31,1136.516129,67,58,33,8,166,19.9,40.4,34.9,4.8,8.433735,6.626506,10.843373,5.421687,0.000000,10.240964,0.000000,6.626506,26.506024,25.301205,0.000000,0.000000,0.000000,40.361446,3.012048,56.626506,19.277108,0.602410,26.506024,0.000000,10.240964,1.807229,3.012048,11.445783,0.602410,4.216867,1.204819,7.831325,4.819277,1.204819,7.228916,2541770,0.422674,2025-08-25 11:14:50,대량처리형,대량처리형
4,4,4,26,402,678,70,2025-07-26 01:45:31,2025-08-25 10:22:36,31,2.258065,10,5,3,1,19,15.8,52.6,26.3,5.3,0.000000,0.000000,15.789474,0.000000,0.000000,47.368421,0.000000,0.000000,26.315789,10.526316,0.000000,0.000000,78.947368,0.000000,0.000000,21.052632,52.631579,0.000000,15.789474,0.000000,10.526316,0.000000,0.000000,10.526316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.526316,2690,0.103245,2025-08-25 10:22:36,특화전문형_게임특화,특화전문형
5,5,5,30,3606,5160,391,2025-07-26 00:42:21,2025-08-25 11:06:44,31,12.612903,14,37,9,0,60,15.0,23.3,61.7,0.0,0.000000,5.000000,1.666667,8.333333,0.000000,38.333333,0.000000,3.333333,20.000000,23.333333,0.000000,0.000000,0.000000,58.333333,0.000000,41.666667,41.666667,1.666667,18.333333,0.000000,11.666667,1.666667,3.333333,8.333333,0.000000,1.666667,0.000000,3.333333,1.666667,3.333333,3.333333,22520,0.075775,2025-08-25 11:06:44,안정공급형(후보),안정공급형(후보)
6,6,6,32,870,1574,163,2025-07-26 10:48:49,2025-08-25 10:38:39,30,5.433333,5,0,2,0,7,28.6,71.4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.285714,71.428571,14.285714,0.000000,0.000000,0.000000,57.142857,0.000000,42.857143,0.000000,0.000000,14.285714,0.000000,28.571429,0.000000,0.000000,28.571429,0.000000,0.000000,0.000000,0.000000,14.285714,0.000000,14.285714,191080,0.103558,2025-08-25 10:38:39,관리 필요,관리 필요
7,7,7,54,6395,12838,4299

In [120]:
ads_new.columns

Index(['ads_idx', 'ads_type', 'ads_category', 'ads_name', 'ads_os_type',
       'ads_contract_price', 'ads_media_price', 'ads_reward_price',
       'ads_rejoin_type', 'domain'],
      dtype='object')

In [121]:
pd.set_option("display.max_columns", None) # 열 전체 출력


In [122]:
ㅋㅋ

NameError: name 'ᄏᄏ' is not defined

In [ ]:
# 계약종료형 매체 목록
banned_mda = set(
    media_performance_classification
      .loc[media_performance_classification["basic_classification"].astype(str).str.strip() == "계약종료형", "mda_idx"]
      .dropna().astype(int).unique()
)
print(f"[Exclude] 계약종료형 mda 개수 = {len(banned_mda)}")


[Exclude] 계약종료형 mda 개수 = 51


In [ ]:
# =========================================================
#  Ad × Media 추천 + 전환수 예측 (Decomposition: Click × CVR, OS 제외)
#  - 누수 방지: 모든 피처는 OBS에서만 생성, 라벨은 TGT
#  - 클릭: Tweedie 회귀(권장), CVR: logit 회귀
#  - 예측: yhat_turn = yhat_click × yhat_cvr
#  - 계약종료형(banned_mda) 매체 전 과정 제외
# =========================================================
# 전제: import pandas as pd, numpy as np, lightgbm as lgb, warnings
#       from typing import *
#       from sklearn.metrics import roc_auc_score
warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm")

# -------------------------------
# 유틸
# -------------------------------
def _to_num(s): return pd.to_numeric(s, errors="coerce").fillna(0)
def _to_dt(s):  return pd.to_datetime(s, errors="coerce")

def _norm_cat_col(df: pd.DataFrame, col: str) -> pd.DataFrame:
    if col in df.columns:
        df = df.copy()
        df[col] = df[col].astype(str).str.strip().str.lower()
        df[col] = df[col].replace({"<na>":"__na__", "nan":"__na__", "none":"__na__", "nat":"__na__"})
    return df

def _norm_cat(s): return pd.Series(s, copy=False).astype("string").str.strip().str.lower()

def _pick_col(cols, cand):
    for c in cand:
        if c in cols: return c
    return None

def _ensure_col(df: pd.DataFrame, out_col: str, cands: List[str]) -> pd.DataFrame:
    col = _pick_col(df.columns, cands)
    df = df.copy()
    if col is None: df[out_col] = "__na__"
    else:           df[out_col] = _norm_cat(df[col])
    return df

# -------------------------------
# 1) 윈도우
# -------------------------------
def split_obs_tgt_by_dates(
    ads_time: pd.DataFrame,
    obs_start: str = "2025-07-25",
    tgt_start: str = "2025-08-20",
    obs_end: Optional[str] = None,
    tgt_end: Optional[str] = None,
    tgt_max_days: int = 6
):
    t = ads_time.copy()
    t["date"] = _to_dt(t["rpt_time_date"])
    if t["date"].isna().all():
        raise ValueError("ads_time['rpt_time_date']에 유효한 날짜가 없습니다.")
    obs_start = pd.to_datetime(obs_start).normalize()
    tgt_start = pd.to_datetime(tgt_start).normalize()
    obs_end = (pd.to_datetime(obs_end).normalize() if obs_end is not None
               else (tgt_start - pd.Timedelta(days=1)).normalize())
    last_date = t["date"].max().normalize()
    tgt_end = (pd.to_datetime(tgt_end).normalize() if tgt_end is not None
               else min(last_date, (tgt_start + pd.Timedelta(days=tgt_max_days)).normalize()))
    obs = t[(t["date"] >= obs_start) & (t["date"] <= obs_end)].copy()
    tgt = t[(t["date"] >= tgt_start) & (t["date"] <= tgt_end)].copy()
    return obs, tgt, obs_start, tgt_end, (tgt_start - pd.Timedelta(days=1)).normalize()

# -------------------------------
# 2) 시간/매체/카테고리/도메인 컨텍스트 (OBS만)
# -------------------------------
def _build_time_signals(obs: pd.DataFrame) -> pd.DataFrame:
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["weekday"] = _to_dt(t["rpt_time_date"]).dt.weekday  # 0~6
    wd = t.groupby(["mda_idx","weekday"], as_index=False).agg(clk=("clk","sum"))
    wd["is_weekend"] = wd["weekday"].isin([5,6]).astype(int)
    g = wd.groupby("mda_idx", as_index=False).agg(clk_total=("clk","sum"))
    w = wd.merge(g, on="mda_idx", how="left")
    day_ratio = (w.assign(ratio=w["clk"]/w["clk_total"].replace(0,np.nan))
                   .sort_values(["mda_idx","ratio"], ascending=[True,False])
                   .groupby("mda_idx").head(1)[["mda_idx","ratio"]]
                   .rename(columns={"ratio":"mda_day_ratio"}))
    weekend_ratio = (w.groupby(["mda_idx","is_weekend"], as_index=False)
                       .agg(clk=("clk","sum")).merge(g,on="mda_idx",how="left"))
    weekend_ratio["mda_weekend_ratio"] = (weekend_ratio["clk"] /
                                          weekend_ratio["clk_total"].replace(0,np.nan)).fillna(0)
    weekend_ratio = weekend_ratio[weekend_ratio["is_weekend"]==1][["mda_idx","mda_weekend_ratio"]]
    out = day_ratio.merge(weekend_ratio, on="mda_idx", how="outer").fillna(0.0)
    out["mda_idx"] = out["mda_idx"].astype(int)
    return out

def build_attr_context(obs: pd.DataFrame, ads_list_with_domain: pd.DataFrame):
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["conv"] = _to_num(t.get("rpt_time_turn", 0))

    # mda 전역
    mda_all = (t.groupby("mda_idx", as_index=False)
                 .agg(mda_clk_global=("clk","sum"), mda_conv_global=("conv","sum")))
    mda_all["mda_cvr_global"] = (mda_all["mda_conv_global"] /
                                 mda_all["mda_clk_global"].replace(0,np.nan)).fillna(0.0)
    mu, sd = mda_all["mda_conv_global"].mean(), mda_all["mda_conv_global"].std(ddof=0)
    mda_all["mda_pop_norm"] = (mda_all["mda_conv_global"] - mu) / (sd + 1e-9)

    # 카테고리×매체
    al = ads_list_with_domain[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    al["ads_category"] = _norm_cat(al["ads_category"])
    t2 = t.merge(al, on="ads_idx", how="left")
    cat_mda = (t2.groupby(["mda_idx","ads_category"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    cat_mda["cat_cvr"] = (cat_mda["conv"]/cat_mda["clk"].replace(0,np.nan)).fillna(0.0)
    mda_tot = cat_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    cat_mda = cat_mda.merge(mda_tot, on="mda_idx", how="left")
    cat_mda["cat_share"] = (cat_mda["conv"]/cat_mda["mda_conv_total"].replace(0,np.nan)).fillna(0.0)

    # 도메인×매체 (ads_list 도메인 기반)
    t_dom = t.merge(
        ads_list_with_domain[["ads_idx","ads_domain"]].drop_duplicates("ads_idx"),
        on="ads_idx", how="left"
    )
    t_dom["ads_domain"] = _norm_cat(t_dom["ads_domain"])
    dom_mda = (t_dom.groupby(["mda_idx","ads_domain"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    dom_mda["dom_cvr"] = (dom_mda["conv"]/dom_mda["clk"].replace(0,np.nan)).fillna(0.0)
    mda_tot2 = dom_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    dom_mda = dom_mda.merge(mda_tot2, on="mda_idx", how="left")
    dom_mda["dom_share"] = (dom_mda["conv"]/dom_mda["mda_conv_total"].replace(0,np.nan)).fillna(0.0)

    # 시간 신호
    time_sig = _build_time_signals(obs)

    # (선택) 광고주 신호
    adv_ctx = None
    if "adv_idx" in ads_list_with_domain.columns:
        t_adv = t.merge(ads_list_with_domain[["ads_idx","adv_idx"]].drop_duplicates("ads_idx"),
                        on="ads_idx", how="left")
        adv_all = (t_adv.groupby("adv_idx", as_index=False)
                        .agg(adv_clk_global=("clk","sum"),
                             adv_conv_global=("conv","sum")))
        adv_all["adv_cvr_global"] = (adv_all["adv_conv_global"] /
                                     adv_all["adv_clk_global"].replace(0,np.nan)).fillna(0.0)
        mu2, sd2 = adv_all["adv_conv_global"].mean(), adv_all["adv_conv_global"].std(ddof=0)
        adv_all["adv_pop_norm"] = (adv_all["adv_conv_global"] - mu2) / (sd2 + 1e-9)
        adv_ctx = adv_all

    return {"mda_all": mda_all, "cat_mda": cat_mda, "dom_mda": dom_mda,
            "time_sig": time_sig, "adv_ctx": adv_ctx}

# -------------------------------
# 3) 학습 대상 광고
# -------------------------------
def pick_train_ads_from_tgt(tgt: pd.DataFrame, min_pos_in_tgt: int = 1,
                            max_train_ads: Optional[int] = 20000, rng_seed: int = 42) -> np.ndarray:
    t = tgt.copy()
    t["turn"] = _to_num(t.get("rpt_time_turn", 0))
    pos = (t.groupby("ads_idx")["turn"].apply(lambda s: (s > 0).sum())
             .reset_index(name="pos_cnt"))
    ok = pos[pos["pos_cnt"] >= max(1,int(min_pos_in_tgt))]["ads_idx"].astype(int).values
    if max_train_ads is not None and len(ok) > max_train_ads:
        rng = np.random.RandomState(rng_seed)
        ok = rng.choice(ok, size=max_train_ads, replace=False)
    return np.sort(ok)

# -------------------------------
# 4) 후보 생성
# -------------------------------
def create_candidates(ads_list: pd.DataFrame, mda_all: pd.DataFrame, cat_mda: pd.DataFrame,
                      topC=10, add_random=2, cat_topM=5,
                      min_active_clk=200, min_active_conv=20,
                      rng_seed=42, ad_ids_filter=None, max_per_ad=20):
    mda_active = mda_all[(mda_all["mda_clk_global"]>=min_active_clk) |
                         (mda_all["mda_conv_global"]>=min_active_conv)].copy()
    if mda_active.empty: mda_active = mda_all.copy()
    mda_ids_global = (mda_active.sort_values("mda_conv_global", ascending=False)["mda_idx"]
                      .astype(int).values)
    cat_top = (cat_mda.sort_values(["ads_category","cat_cvr"], ascending=[True,False])
                      .groupby("ads_category").head(cat_topM)[["ads_category","mda_idx"]])
    cat2top = {c: g["mda_idx"].astype(int).tolist() for c,g in cat_top.groupby("ads_category")}
    rng = np.random.RandomState(rng_seed)
    ad_ids = (np.array(ad_ids_filter, dtype=int) if (ad_ids_filter is not None and len(ad_ids_filter)>0)
              else ads_list["ads_idx"].dropna().astype(int).unique())
    ad_meta = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    ad_meta["ads_idx"] = ad_meta["ads_idx"].astype(int)
    ad_meta["ads_category"] = ad_meta["ads_category"].astype(str).str.strip().str.lower()
    ad2cat = ad_meta.set_index("ads_idx")["ads_category"].to_dict()

    pairs=[]
    for ad in ad_ids:
        base=[]
        cat = ad2cat.get(int(ad), None)
        if cat in cat2top: base.extend(cat2top[cat])
        base.extend(mda_ids_global[:min(topC,len(mda_ids_global))])
        base=list(dict.fromkeys(base))
        remain=np.setdiff1d(mda_ids_global, np.array(base, dtype=int))
        if len(remain)>0 and add_random>0:
            base.extend(rng.choice(remain, size=min(add_random,len(remain)), replace=False).tolist())
        if len(base)>max_per_ad: base=base[:max_per_ad]
        pairs += [(int(ad), int(m)) for m in base]
    out = pd.DataFrame(pairs, columns=["ads_idx","mda_idx"]).drop_duplicates()
    print(f"[Candidates] pairs={len(out):,} | ads={len(ad_ids):,} | active_mda={len(mda_active):,} | topC={topC}, rnd={add_random}, cat_topM={cat_topM}, per_ad≤{max_per_ad}")
    return out

# -------------------------------
# 5) 피처 & 라벨 (OBS 피처 / TGT 라벨)
# -------------------------------
FEAT_ALLOW = [
    # mda 전역
    "mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm",
    # 카테고리×매체
    "cat_cvr","cat_share",
    # 광고 가격
    "price_reward_log","price_contract_log","price_ratio",
    # 도메인×매체
    "dom_cvr","dom_share",
    # 시간 신호
    "mda_day_ratio","mda_weekend_ratio",
    # 광고주 신호
    "adv_cvr_global","adv_pop_norm",
    # 외부 포트폴리오가 있으면
    "mda_size","active_ratio","day_ratio","weekend_ratio"
]

def prepare_training_attr_only(
    obs: pd.DataFrame, tgt: pd.DataFrame, ads_list_with_domain: pd.DataFrame,
    topC=10, add_random=2, cat_topM=5, min_active_clk=200, min_active_conv=20,
    max_train_ads: Optional[int] = 20000, min_pos_in_tgt: int = 1,
    media_portfolio: Optional[pd.DataFrame] = None,
    excluded_mda_ids: Optional[set] = None  # ★ 추가: 학습 데이터에서 제외
):
    # ★ mda 제외(OBS/TGT에서 이미 제거했더라도 안전망)
    if excluded_mda_ids:
        obs = obs[~obs["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()
        tgt = tgt[~tgt["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    train_ads = pick_train_ads_from_tgt(tgt, min_pos_in_tgt=min_pos_in_tgt, max_train_ads=max_train_ads)
    actx = build_attr_context(obs, ads_list_with_domain)
    mda_all, cat_mda, dom_mda = actx["mda_all"], actx["cat_mda"], actx["dom_mda"]

    cand = create_candidates(
        ads_list=ads_list_with_domain, mda_all=mda_all, cat_mda=cat_mda,
        topC=topC, add_random=add_random, cat_topM=cat_topM,
        min_active_clk=min_active_clk, min_active_conv=min_active_conv,
        ad_ids_filter=train_ads, max_per_ad=20
    )

    # ★ 후보 그리드에서도 제외
    if excluded_mda_ids:
        cand = cand[~cand["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    df = cand.merge(
        mda_all[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]],
        on="mda_idx", how="left"
    )

    ad_meta = ads_list_with_domain[["ads_idx","ads_category","ads_reward_price","ads_contract_price","ads_domain","adv_idx"]].drop_duplicates("ads_idx").copy()
    ad_meta = _norm_cat_col(ad_meta, "ads_category")
    ad_meta = _ensure_col(ad_meta, "ads_domain", ["ads_domain","domain"])
    for c in ["ads_reward_price","ads_contract_price"]:
        ad_meta[c] = _to_num(ad_meta[c])
    ad_meta["price_reward_log"]   = np.log1p(ad_meta["ads_reward_price"])
    ad_meta["price_contract_log"] = np.log1p(ad_meta["ads_contract_price"])
    ad_meta["price_ratio"]        = (ad_meta["ads_reward_price"] /
                                     ad_meta["ads_contract_price"].replace(0,np.nan)).fillna(0.0)

    df = df.merge(ad_meta[["ads_idx","ads_category","ads_domain","adv_idx",
                           "price_reward_log","price_contract_log","price_ratio"]],
                  on="ads_idx", how="left")

    df      = _norm_cat_col(df, "ads_category")
    cat_mda = _norm_cat_col(cat_mda, "ads_category")
    df = df.merge(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]],
                  on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0,"cat_share":0.0})

    dom_mda = dom_mda.copy()
    dom_mda["ads_domain"] = _norm_cat(dom_mda["ads_domain"])
    df["ads_domain"] = _norm_cat(df["ads_domain"])
    df = df.merge(dom_mda[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                  on=["mda_idx","ads_domain"], how="left").fillna({"dom_cvr":0.0,"dom_share":0.0})

    time_sig = actx["time_sig"]
    df = df.merge(time_sig, on="mda_idx", how="left").fillna({"mda_day_ratio":0.0,"mda_weekend_ratio":0.0})

    if actx["adv_ctx"] is not None:
        adv_ctx = actx["adv_ctx"].copy()
        df = df.merge(adv_ctx[["adv_idx","adv_cvr_global","adv_pop_norm"]], on="adv_idx", how="left")
    for c in ["adv_cvr_global","adv_pop_norm"]:
        if c not in df.columns: df[c]=0.0
    df[["adv_cvr_global","adv_pop_norm"]] = df[["adv_cvr_global","adv_pop_norm"]].fillna(0.0)

    if media_portfolio is not None:
        mp = media_portfolio.copy()
        cols = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp.columns]
        if "mda_idx" in cols: df = df.merge(mp[cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in df.columns: df[c]=0.0

    # ---- 라벨: TGT에서 클릭/전환
    tgt2 = tgt.copy()
    tgt2["clk"]  = _to_num(tgt2.get("rpt_time_clk", 0))
    tgt2["turn"] = _to_num(tgt2.get("rpt_time_turn",0))
    y_agg = (tgt2.groupby(["ads_idx","mda_idx"], as_index=False)
                   .agg(tgt_clk=("clk","sum"), tgt_turn=("turn","sum")))
    y_agg["tgt_cvr"] = (y_agg["tgt_turn"] / y_agg["tgt_clk"].replace(0,np.nan)).clip(0,1)

    df = df.merge(y_agg, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_clk":0.0,"tgt_turn":0.0})

    # 랭커용 relevance
    y = df["tgt_turn"].astype(float)
    rel = np.zeros(len(df), dtype=int)
    pos = y > 0
    if pos.sum()>0:
        ylog = np.log1p(y[pos])
        try:
            bins = pd.qcut(ylog, q=3, labels=False, duplicates="drop")
            rel[pos] = bins.astype(int)+1
        except ValueError:
            rel[pos] = 1
    df["relevance"] = rel
    return df, actx

# -------------------------------
# 6) 학습기들 (랭커 + 클릭회귀 + CVR회귀)
# -------------------------------
LABEL_GAIN = [0.0,1.0,4.0,12.0]

def ensure_feature_matrix(df, feat_cols):
    X = df.copy()
    for c in feat_cols:
        if c not in X.columns: X[c]=0.0
        if not np.issubdtype(X[c].dtype, np.number):
            X[c]=pd.to_numeric(X[c], errors="coerce")
    return X[feat_cols].fillna(0.0).astype(np.float32)

def downsample_negatives_per_ad(df, ratio=4, random_state=42):
    out, rng = [], np.random.RandomState(random_state)
    for ad, g in df.groupby("ads_idx", sort=False):
        pos = g[g["tgt_turn"]>0]; neg = g[g["tgt_turn"]<=0]
        if len(pos)==0: continue
        cap = len(pos)*ratio
        if len(neg)>cap: neg = neg.sample(n=cap, random_state=rng)
        out.append(pd.concat([pos,neg], ignore_index=True))
    return pd.concat(out, ignore_index=True) if out else df

def train_attr_ranker(train_df: pd.DataFrame):
    feat_cols = [c for c in FEAT_ALLOW if c in train_df.columns]
    X = ensure_feature_matrix(train_df, feat_cols)
    y = train_df["relevance"].astype(int)
    groups = train_df.groupby("ads_idx").size().astype(int).tolist()
    ranker = lgb.LGBMRanker(
        objective="lambdarank", metric="ndcg", eval_at=[5,10,20],
        learning_rate=0.05, n_estimators=1200, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        label_gain=LABEL_GAIN, random_state=42, n_jobs=-1, verbosity=-1
    )
    ranker.fit(X, y, group=groups, sample_weight=np.ones(len(train_df)))
    return ranker, feat_cols

def train_click_regressor(train_df: pd.DataFrame, feat_cols: List[str], tweedie_p: float = 1.1):
    X = ensure_feature_matrix(train_df, feat_cols)
    y = train_df["tgt_clk"].astype(float)
    reg = lgb.LGBMRegressor(
        objective="tweedie", tweedie_variance_power=tweedie_p,
        n_estimators=1200, learning_rate=0.05, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        random_state=42, n_jobs=-1, verbosity=-1
    )
    reg.fit(X, y)
    return reg, {"mode":"tweedie_click","twp":tweedie_p}

def _logit_clip(p, eps=1e-6):
    p = np.clip(p, eps, 1-eps)
    return np.log(p/(1-p))

def _sigmoid(x): return 1/(1+np.exp(-x))

def train_cvr_regressor(train_df: pd.DataFrame, feat_cols: List[str]):
    g = train_df[train_df["tgt_clk"]>0].copy()
    if g.empty:
        dummy = lgb.LGBMRegressor().fit(np.zeros((1, len(feat_cols))), [0.0])
        return dummy, {"mode":"cvr_logit","fallback":True, "mean_logit":-20.0}
    X = ensure_feature_matrix(g, feat_cols)
    y = g["tgt_turn"]/g["tgt_clk"]
    y = y.clip(0,1)
    y_logit = _logit_clip(y.values)
    reg = lgb.LGBMRegressor(
        objective="regression",
        n_estimators=1200, learning_rate=0.05, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        random_state=42, n_jobs=-1, verbosity=-1
    )
    reg.fit(X, y_logit)
    return reg, {"mode":"cvr_logit","fallback":False}

# -------------------------------
# 7) 예측 (랭커 스코어 + 클릭×CVR 예측)
# -------------------------------
def predict_all_mda_with_counts_batched(
    ranker, click_reg, cvr_reg, feat_cols, actx, ads_list_with_domain,
    K=20, batch_size=5000, max_predict_ads: Optional[int] = 20000,
    media_portfolio: Optional[pd.DataFrame] = None,
    excluded_mda_ids: Optional[set] = None  # ★ 추가: 예측 교차조인에서도 제외
) -> Dict[int, pd.DataFrame]:
    mda_all, cat_mda, dom_mda = actx["mda_all"], actx["cat_mda"], actx["dom_mda"]
    time_sig, adv_ctx = actx["time_sig"], actx["adv_ctx"]

    mda_active = mda_all[(mda_all["mda_clk_global"]>=200) | (mda_all["mda_conv_global"]>=20)]
    if mda_active.empty: mda_active = mda_all
    if excluded_mda_ids:
        mda_active = mda_active[~mda_active["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    mda_active = mda_active[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]].copy()
    mda_active["mda_idx"]=mda_active["mda_idx"].astype(int)
    mda_active = mda_active.merge(time_sig, on="mda_idx", how="left").fillna({"mda_day_ratio":0.0,"mda_weekend_ratio":0.0})

    if media_portfolio is not None:
        mp = media_portfolio.copy()
        cols = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp.columns]
        if "mda_idx" in cols: mda_active = mda_active.merge(mp[cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in mda_active.columns: mda_active[c]=0.0

    ads_meta = ads_list_with_domain[["ads_idx","ads_category","ads_reward_price","ads_contract_price","ads_domain","adv_idx"]].drop_duplicates("ads_idx").copy()
    if max_predict_ads is not None: ads_meta = ads_meta.head(int(max_predict_ads))
    ads_meta = _norm_cat_col(ads_meta, "ads_category")
    ads_meta = _ensure_col(ads_meta, "ads_domain", ["ads_domain","domain"])
    ads_meta["ads_idx"] = ads_meta["ads_idx"].astype(int)
    ads_meta["ads_domain"] = _norm_cat(ads_meta["ads_domain"])
    ads_meta["ads_reward_price"]   = _to_num(ads_meta["ads_reward_price"])
    ads_meta["ads_contract_price"] = _to_num(ads_meta["ads_contract_price"])
    ads_meta["price_reward_log"]   = np.log1p(ads_meta["ads_reward_price"])
    ads_meta["price_contract_log"] = np.log1p(ads_meta["ads_contract_price"])
    ads_meta["price_ratio"]        = (ads_meta["ads_reward_price"] / ads_meta["ads_contract_price"].replace(0,np.nan)).fillna(0.0)
    if adv_ctx is not None:
        ads_meta = ads_meta.merge(adv_ctx[["adv_idx","adv_cvr_global","adv_pop_norm"]], on="adv_idx", how="left")
    for c in ["adv_cvr_global","adv_pop_norm"]:
        if c not in ads_meta.columns: ads_meta[c]=0.0
    ads_meta[["adv_cvr_global","adv_pop_norm"]] = ads_meta[["adv_cvr_global","adv_pop_norm"]].fillna(0.0)

    cat_mda_local = _norm_cat_col(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]].copy(), "ads_category")
    dom_mda_local = dom_mda.copy(); dom_mda_local["ads_domain"] = _norm_cat(dom_mda_local["ads_domain"])

    out={}
    ad_ids = ads_meta["ads_idx"].tolist()
    for i in range(0, len(ad_ids), batch_size):
        ad_batch = ads_meta.iloc[i:i+batch_size]
        grid = ad_batch[["ads_idx","ads_category","ads_domain","price_reward_log","price_contract_log","price_ratio",
                         "adv_idx","adv_cvr_global","adv_pop_norm"]] \
               .merge(mda_active, how="cross")

        grid = grid.merge(cat_mda_local, on=["mda_idx","ads_category"], how="left") \
                   .fillna({"cat_cvr":0.0,"cat_share":0.0})
        grid = grid.merge(dom_mda_local[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                          on=["mda_idx","ads_domain"], how="left") \
                   .fillna({"dom_cvr":0.0,"dom_share":0.0})

        X = ensure_feature_matrix(grid, feat_cols)
        score = ranker.predict(X)  # 랭커 점수

        yhat_clk = np.maximum(0.0, click_reg.predict(X))
        yhat_cvr_logit = cvr_reg.predict(X)
        yhat_cvr = np.clip(_sigmoid(yhat_cvr_logit), 0.0, 1.0)
        yhat_turn = yhat_clk * yhat_cvr

        grid2 = grid[["ads_idx","mda_idx","ads_category","mda_cvr_global","mda_pop_norm","mda_clk_global","mda_conv_global"]].copy()
        grid2["score"] = score.astype(float)
        grid2["yhat_turn"] = yhat_turn.astype(float)

        for ad, g in grid2.groupby("ads_idx", sort=False):
            top = g.nlargest(K, "score")
            out[int(ad)] = top.sort_values("score", ascending=False).reset_index(drop=True)
        print(f"[Predict+Count] processed ads {i+len(ad_batch)}/{len(ad_ids)}")
    return out

# -------------------------------
# 8) 평가
# -------------------------------
def _precision_recall_hit_at_k(preds_dict, gt_dict, k=10):
    P,R,H=[],[],[]
    for ad, df in preds_dict.items():
        if ad not in gt_dict: continue
        pred = df["mda_idx"].astype(int).tolist()[:k]
        true = set(gt_dict[ad]); 
        if not true: continue
        inter = len(set(pred)&true)
        P.append(inter/max(1,len(pred))); R.append(inter/len(true)); H.append(1.0 if inter>0 else 0.0)
    return (float(np.mean(P)) if P else 0.0,
            float(np.mean(R)) if R else 0.0,
            float(np.mean(H)) if H else 0.0)

def _map_at_k(preds_dict, gt_dict, k=10):
    APs=[]
    for ad, df in preds_dict.items():
        if ad not in gt_dict: continue
        true=set(gt_dict[ad]); 
        if not true: continue
        ranked = df["mda_idx"].astype(int).tolist()[:k]
        hits=cum_prec=0
        for i,m in enumerate(ranked, start=1):
            if m in true:
                hits+=1; cum_prec += hits/i
        APs.append(cum_prec/min(len(true),k) if hits>0 else 0.0)
    return float(np.mean(APs)) if APs else 0.0

def _auc_flat(preds_dict, gt_pairs_dict, score_col="score"):
    y_true, y_score = [], []
    for ad, df in preds_dict.items():
        true_mdas = set(gt_pairs_dict.get(ad, []))
        for _, r in df.iterrows():
            y_true.append(1 if int(r["mda_idx"]) in true_mdas else 0)
            y_score.append(float(r[score_col]))
    if len(set(y_true))<2: return 0.5
    try: return float(roc_auc_score(y_true, y_score))
    except Exception: return 0.5

def evaluate_all(preds_with_counts: Dict[int, pd.DataFrame], tgt: pd.DataFrame, k=10):
    gt_pairs = tgt.groupby("ads_idx")["mda_idx"].apply(lambda s: list(pd.Series(s).dropna().astype(int).unique())).to_dict()
    prec, rec, hit = _precision_recall_hit_at_k(preds_with_counts, gt_pairs, k=k)
    mapk = _map_at_k(preds_with_counts, gt_pairs, k=k)
    auc_rank = _auc_flat(preds_with_counts, gt_pairs, score_col="score")
    auc_reg  = _auc_flat(preds_with_counts, gt_pairs, score_col="yhat_turn")
    return {f"precision@{k}":prec, f"recall@{k}":rec, f"map@{k}":mapk,
            f"hit_rate@{k}":hit, "auc_ranker":auc_rank, "auc_regressor":auc_reg}

# -------------------------------
# 9) 편의
# -------------------------------
def get_expected_conversions_for_ad(preds_with_counts: Dict[int, pd.DataFrame],
                                    ad_id: int, sort_by="yhat_turn", top=20) -> pd.DataFrame:
    df = preds_with_counts.get(int(ad_id))
    if df is None or df.empty:
        return pd.DataFrame(columns=["mda_idx","score","yhat_turn"])
    sort_by = "yhat_turn" if sort_by not in ("yhat_turn","score") else sort_by
    out = df.sort_values(sort_by, ascending=False).copy()
    return out.head(top) if top is not None else out

# =========================================================
# ==============   메인 플로우 (예시)   ====================
# =========================================================
# 요구: ads_time, ads_list, ads_segment(ads_size, domain), media_performance_classification, media_portfolio(옵션)

# (A) 윈도우
obs, tgt, start, end, cut = split_obs_tgt_by_dates(
    ads_time, obs_start="2025-07-25", tgt_start="2025-08-20"
)
print(f"[Window] OBS: {start.date()} → {cut.date()}  |  TGT: {(cut+pd.Timedelta(days=1)).date()} → {end.date()}")

# (A-1) 계약종료형 매체 목록 (이미 앞에서 만든 경우 이 블록 생략 가능)
# banned_mda = set(
#     media_performance_classification
#       .loc[media_performance_classification["basic_classification"].astype(str).str.strip() == "계약종료형", "mda_idx"]
#       .dropna().astype(int).unique()
# )
print(f"[Exclude] 계약종료형 mda 개수 = {len(banned_mda)}")

# (A-2) OBS/TGT에서 계약종료형 제거 (컨텍스트/학습/평가 전 과정 반영)
if banned_mda:
    obs = obs[~obs["mda_idx"].astype(int).isin(banned_mda)].copy()
    tgt = tgt[~tgt["mda_idx"].astype(int).isin(banned_mda)].copy()

# (B) ads_list 확장 (ads_size, domain 병합)
ads_list_ext = ads_list.merge(
    ads_segment[["ads_idx","ads_size","domain"]].rename(columns={"domain":"ads_domain"}),
    on="ads_idx", how="left"
)

# (C) MEGA/LARGE/MEDIUM만
size_whitelist = {"MEGA","LARGE","MEDIUM"}
ads_list_sel = ads_list_ext[
    ads_list_ext["ads_size"].astype(str).str.strip().str.upper().isin(size_whitelist)
].copy()
sel_ids = set(ads_list_sel["ads_idx"].astype(int))

tgt_sel  = tgt[tgt["ads_idx"].astype(int).isin(sel_ids)].copy()

print(f"[Filter] sizes kept = {sorted(size_whitelist)} | ads={ads_list_sel['ads_idx'].nunique():,} | tgt pairs={len(tgt_sel):,}")

# (D) 학습 데이터 (계약종료형 제외 세이프가드 포함)
train_df, actx = prepare_training_attr_only(
    obs, tgt_sel, ads_list_sel,
    topC=10, add_random=2, cat_topM=5,
    min_active_clk=200, min_active_conv=20,
    max_train_ads=20000, min_pos_in_tgt=1,
    media_portfolio=None,
    excluded_mda_ids=banned_mda  # ★ 유지
)
print("train_df size:", len(train_df))

# (E) 학습 (랭커 + 클릭 + CVR)
train_df_ds = downsample_negatives_per_ad(train_df, ratio=4)
ranker, feat_cols   = train_attr_ranker(train_df_ds)
click_reg, click_md = train_click_regressor(train_df_ds, feat_cols, tweedie_p=1.1)
cvr_reg, cvr_md     = train_cvr_regressor(train_df_ds, feat_cols)
print("feat_cols used:", feat_cols)

# (F) 예측 & 평가 (예측 교차조인에서도 계약종료형 제거)
preds_with_counts = predict_all_mda_with_counts_batched(
    ranker, click_reg, cvr_reg, feat_cols, actx, ads_list_sel,
    K=20, batch_size=5000, max_predict_ads=20000, media_portfolio=None,
    excluded_mda_ids=banned_mda  # ★ 유지
)
metrics_k5  = evaluate_all(preds_with_counts, tgt_sel, k=5)
metrics_k10 = evaluate_all(preds_with_counts, tgt_sel, k=10)
print("Eval@5:", metrics_k5)
print("Eval@10:", metrics_k10)

# (G) Sanity Check: 계약종료형 유입 여부
leftover_tgt = tgt_sel["mda_idx"].astype(int).isin(banned_mda).sum()
print(f"[Sanity] TGT(selected sizes) 내 계약종료형 잔여: {leftover_tgt}")
any_banned_in_preds = any(
    int(m) in banned_mda
    for df in preds_with_counts.values()
    for m in df["mda_idx"].astype(int).tolist()
)
print(f"[Sanity] 예측결과에 계약종료형 포함 여부: {any_banned_in_preds}")

# (H) 예: 특정 광고 top-5 (예상 전환수 기준)
# ad_id = 446406
# top5 = get_expected_conversions_for_ad(preds_with_counts, ad_id, sort_by="yhat_turn", top=5)
# print(top5[["mda_idx","yhat_turn","score"]])



In [ ]:
# =========================================================
#  Ad × Media 추천 + 전환수 예측 (Decomposition: Click × CVR, OS 제외)
#  - 누수 방지: 모든 피처는 OBS에서만 생성, 라벨은 TGT
#  - 클릭: Tweedie 회귀(권장), CVR: logit 회귀
#  - 예측: yhat_turn = yhat_click × yhat_cvr
#  - 계약종료형(banned_mda) 매체 전 과정 제외
# =========================================================
# 전제: import pandas as pd, numpy as np, lightgbm as lgb, warnings
#       from typing import *
#       from sklearn.metrics import roc_auc_score
warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm")

# -------------------------------
# 유틸
# -------------------------------
def _to_num(s): return pd.to_numeric(s, errors="coerce").fillna(0)
def _to_dt(s):  return pd.to_datetime(s, errors="coerce")

def _norm_cat_col(df: pd.DataFrame, col: str) -> pd.DataFrame:
    if col in df.columns:
        df = df.copy()
        df[col] = df[col].astype(str).str.strip().str.lower()
        df[col] = df[col].replace({"<na>":"__na__", "nan":"__na__", "none":"__na__", "nat":"__na__"})
    return df

def _norm_cat(s): return pd.Series(s, copy=False).astype("string").str.strip().str.lower()

def _pick_col(cols, cand):
    for c in cand:
        if c in cols: return c
    return None

def _ensure_col(df: pd.DataFrame, out_col: str, cands: List[str]) -> pd.DataFrame:
    col = _pick_col(df.columns, cands)
    df = df.copy()
    if col is None: df[out_col] = "__na__"
    else:           df[out_col] = _norm_cat(df[col])
    return df

# -------------------------------
# 1) 윈도우
# -------------------------------
def split_obs_tgt_by_dates(
    ads_time: pd.DataFrame,
    obs_start: str = "2025-07-25",
    tgt_start: str = "2025-08-20",
    obs_end: Optional[str] = None,
    tgt_end: Optional[str] = None,
    tgt_max_days: int = 6
):
    t = ads_time.copy()
    t["date"] = _to_dt(t["rpt_time_date"])
    if t["date"].isna().all():
        raise ValueError("ads_time['rpt_time_date']에 유효한 날짜가 없습니다.")
    obs_start = pd.to_datetime(obs_start).normalize()
    tgt_start = pd.to_datetime(tgt_start).normalize()
    obs_end = (pd.to_datetime(obs_end).normalize() if obs_end is not None
               else (tgt_start - pd.Timedelta(days=1)).normalize())
    last_date = t["date"].max().normalize()
    tgt_end = (pd.to_datetime(tgt_end).normalize() if tgt_end is not None
               else min(last_date, (tgt_start + pd.Timedelta(days=tgt_max_days)).normalize()))
    obs = t[(t["date"] >= obs_start) & (t["date"] <= obs_end)].copy()
    tgt = t[(t["date"] >= tgt_start) & (t["date"] <= tgt_end)].copy()
    return obs, tgt, obs_start, tgt_end, (tgt_start - pd.Timedelta(days=1)).normalize()

# -------------------------------
# 2) 시간/매체/카테고리/도메인 컨텍스트 (OBS만)
# -------------------------------
def _build_time_signals(obs: pd.DataFrame) -> pd.DataFrame:
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["weekday"] = _to_dt(t["rpt_time_date"]).dt.weekday  # 0~6
    wd = t.groupby(["mda_idx","weekday"], as_index=False).agg(clk=("clk","sum"))
    wd["is_weekend"] = wd["weekday"].isin([5,6]).astype(int)
    g = wd.groupby("mda_idx", as_index=False).agg(clk_total=("clk","sum"))
    w = wd.merge(g, on="mda_idx", how="left")
    day_ratio = (w.assign(ratio=w["clk"]/w["clk_total"].replace(0,np.nan))
                   .sort_values(["mda_idx","ratio"], ascending=[True,False])
                   .groupby("mda_idx").head(1)[["mda_idx","ratio"]]
                   .rename(columns={"ratio":"mda_day_ratio"}))
    weekend_ratio = (w.groupby(["mda_idx","is_weekend"], as_index=False)
                       .agg(clk=("clk","sum")).merge(g,on="mda_idx",how="left"))
    weekend_ratio["mda_weekend_ratio"] = (weekend_ratio["clk"] /
                                          weekend_ratio["clk_total"].replace(0,np.nan)).fillna(0)
    weekend_ratio = weekend_ratio[weekend_ratio["is_weekend"]==1][["mda_idx","mda_weekend_ratio"]]
    out = day_ratio.merge(weekend_ratio, on="mda_idx", how="outer").fillna(0.0)
    out["mda_idx"] = out["mda_idx"].astype(int)
    return out

def build_attr_context(obs: pd.DataFrame, ads_list_with_domain: pd.DataFrame):
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["conv"] = _to_num(t.get("rpt_time_turn", 0))

    # mda 전역
    mda_all = (t.groupby("mda_idx", as_index=False)
                 .agg(mda_clk_global=("clk","sum"), mda_conv_global=("conv","sum")))
    mda_all["mda_cvr_global"] = (mda_all["mda_conv_global"] /
                                 mda_all["mda_clk_global"].replace(0,np.nan)).fillna(0.0)
    mu, sd = mda_all["mda_conv_global"].mean(), mda_all["mda_conv_global"].std(ddof=0)
    mda_all["mda_pop_norm"] = (mda_all["mda_conv_global"] - mu) / (sd + 1e-9)

    # 카테고리×매체
    al = ads_list_with_domain[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    al["ads_category"] = _norm_cat(al["ads_category"])
    t2 = t.merge(al, on="ads_idx", how="left")
    cat_mda = (t2.groupby(["mda_idx","ads_category"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    cat_mda["cat_cvr"] = (cat_mda["conv"]/cat_mda["clk"].replace(0,np.nan)).fillna(0.0)
    mda_tot = cat_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    cat_mda = cat_mda.merge(mda_tot, on="mda_idx", how="left")
    cat_mda["cat_share"] = (cat_mda["conv"]/cat_mda["mda_conv_total"].replace(0,np.nan)).fillna(0.0)

    # 도메인×매체 (ads_list 도메인 기반)
    t_dom = t.merge(
        ads_list_with_domain[["ads_idx","ads_domain"]].drop_duplicates("ads_idx"),
        on="ads_idx", how="left"
    )
    t_dom["ads_domain"] = _norm_cat(t_dom["ads_domain"])
    dom_mda = (t_dom.groupby(["mda_idx","ads_domain"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    dom_mda["dom_cvr"] = (dom_mda["conv"]/dom_mda["clk"].replace(0,np.nan)).fillna(0.0)
    mda_tot2 = dom_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    dom_mda = dom_mda.merge(mda_tot2, on="mda_idx", how="left")
    dom_mda["dom_share"] = (dom_mda["conv"]/dom_mda["mda_conv_total"].replace(0,np.nan)).fillna(0.0)

    # 시간 신호
    time_sig = _build_time_signals(obs)

    # (선택) 광고주 신호
    adv_ctx = None
    if "adv_idx" in ads_list_with_domain.columns:
        t_adv = t.merge(ads_list_with_domain[["ads_idx","adv_idx"]].drop_duplicates("ads_idx"),
                        on="ads_idx", how="left")
        adv_all = (t_adv.groupby("adv_idx", as_index=False)
                        .agg(adv_clk_global=("clk","sum"),
                             adv_conv_global=("conv","sum")))
        adv_all["adv_cvr_global"] = (adv_all["adv_conv_global"] /
                                     adv_all["adv_clk_global"].replace(0,np.nan)).fillna(0.0)
        mu2, sd2 = adv_all["adv_conv_global"].mean(), adv_all["adv_conv_global"].std(ddof=0)
        adv_all["adv_pop_norm"] = (adv_all["adv_conv_global"] - mu2) / (sd2 + 1e-9)
        adv_ctx = adv_all

    return {"mda_all": mda_all, "cat_mda": cat_mda, "dom_mda": dom_mda,
            "time_sig": time_sig, "adv_ctx": adv_ctx}

# -------------------------------
# 3) 학습 대상 광고
# -------------------------------
def pick_train_ads_from_tgt(tgt: pd.DataFrame, min_pos_in_tgt: int = 1,
                            max_train_ads: Optional[int] = 20000, rng_seed: int = 42) -> np.ndarray:
    t = tgt.copy()
    t["turn"] = _to_num(t.get("rpt_time_turn", 0))
    pos = (t.groupby("ads_idx")["turn"].apply(lambda s: (s > 0).sum())
             .reset_index(name="pos_cnt"))
    ok = pos[pos["pos_cnt"] >= max(1,int(min_pos_in_tgt))]["ads_idx"].astype(int).values
    if max_train_ads is not None and len(ok) > max_train_ads:
        rng = np.random.RandomState(rng_seed)
        ok = rng.choice(ok, size=max_train_ads, replace=False)
    return np.sort(ok)

# -------------------------------
# 4) 후보 생성
# -------------------------------
def create_candidates(ads_list: pd.DataFrame, mda_all: pd.DataFrame, cat_mda: pd.DataFrame,
                      topC=10, add_random=2, cat_topM=5,
                      min_active_clk=200, min_active_conv=20,
                      rng_seed=42, ad_ids_filter=None, max_per_ad=20):
    mda_active = mda_all[(mda_all["mda_clk_global"]>=min_active_clk) |
                         (mda_all["mda_conv_global"]>=min_active_conv)].copy()
    if mda_active.empty: mda_active = mda_all.copy()
    mda_ids_global = (mda_active.sort_values("mda_conv_global", ascending=False)["mda_idx"]
                      .astype(int).values)
    cat_top = (cat_mda.sort_values(["ads_category","cat_cvr"], ascending=[True,False])
                      .groupby("ads_category").head(cat_topM)[["ads_category","mda_idx"]])
    cat2top = {c: g["mda_idx"].astype(int).tolist() for c,g in cat_top.groupby("ads_category")}
    rng = np.random.RandomState(rng_seed)
    ad_ids = (np.array(ad_ids_filter, dtype=int) if (ad_ids_filter is not None and len(ad_ids_filter)>0)
              else ads_list["ads_idx"].dropna().astype(int).unique())
    ad_meta = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    ad_meta["ads_idx"] = ad_meta["ads_idx"].astype(int)
    ad_meta["ads_category"] = ad_meta["ads_category"].astype(str).str.strip().str.lower()
    ad2cat = ad_meta.set_index("ads_idx")["ads_category"].to_dict()

    pairs=[]
    for ad in ad_ids:
        base=[]
        cat = ad2cat.get(int(ad), None)
        if cat in cat2top: base.extend(cat2top[cat])
        base.extend(mda_ids_global[:min(topC,len(mda_ids_global))])
        base=list(dict.fromkeys(base))
        remain=np.setdiff1d(mda_ids_global, np.array(base, dtype=int))
        if len(remain)>0 and add_random>0:
            base.extend(rng.choice(remain, size=min(add_random,len(remain)), replace=False).tolist())
        if len(base)>max_per_ad: base=base[:max_per_ad]
        pairs += [(int(ad), int(m)) for m in base]
    out = pd.DataFrame(pairs, columns=["ads_idx","mda_idx"]).drop_duplicates()
    print(f"[Candidates] pairs={len(out):,} | ads={len(ad_ids):,} | active_mda={len(mda_active):,} | topC={topC}, rnd={add_random}, cat_topM={cat_topM}, per_ad≤{max_per_ad}")
    return out

# -------------------------------
# 5) 피처 & 라벨 (OBS 피처 / TGT 라벨)
# -------------------------------
FEAT_ALLOW = [
    # mda 전역
    "mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm",
    # 카테고리×매체
    "cat_cvr","cat_share",
    # 광고 가격
    "price_reward_log","price_contract_log","price_ratio",
    # 도메인×매체
    "dom_cvr","dom_share",
    # 시간 신호
    "mda_day_ratio","mda_weekend_ratio",
    # 광고주 신호
    "adv_cvr_global","adv_pop_norm",
    # 외부 포트폴리오가 있으면
    "mda_size","active_ratio","day_ratio","weekend_ratio"
]

def prepare_training_attr_only(
    obs: pd.DataFrame, tgt: pd.DataFrame, ads_list_with_domain: pd.DataFrame,
    topC=10, add_random=2, cat_topM=5, min_active_clk=200, min_active_conv=20,
    max_train_ads: Optional[int] = 20000, min_pos_in_tgt: int = 1,
    media_portfolio: Optional[pd.DataFrame] = None,
    excluded_mda_ids: Optional[set] = None  # ★ 추가: 학습 데이터에서 제외
):
    # ★ mda 제외(OBS/TGT에서 이미 제거했더라도 안전망)
    if excluded_mda_ids:
        obs = obs[~obs["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()
        tgt = tgt[~tgt["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    train_ads = pick_train_ads_from_tgt(tgt, min_pos_in_tgt=min_pos_in_tgt, max_train_ads=max_train_ads)
    actx = build_attr_context(obs, ads_list_with_domain)
    mda_all, cat_mda, dom_mda = actx["mda_all"], actx["cat_mda"], actx["dom_mda"]

    cand = create_candidates(
        ads_list=ads_list_with_domain, mda_all=mda_all, cat_mda=cat_mda,
        topC=topC, add_random=add_random, cat_topM=cat_topM,
        min_active_clk=min_active_clk, min_active_conv=min_active_conv,
        ad_ids_filter=train_ads, max_per_ad=20
    )

    # ★ 후보 그리드에서도 제외
    if excluded_mda_ids:
        cand = cand[~cand["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    df = cand.merge(
        mda_all[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]],
        on="mda_idx", how="left"
    )

    ad_meta = ads_list_with_domain[["ads_idx","ads_category","ads_reward_price","ads_contract_price","ads_domain","adv_idx"]].drop_duplicates("ads_idx").copy()
    ad_meta = _norm_cat_col(ad_meta, "ads_category")
    ad_meta = _ensure_col(ad_meta, "ads_domain", ["ads_domain","domain"])
    for c in ["ads_reward_price","ads_contract_price"]:
        ad_meta[c] = _to_num(ad_meta[c])
    ad_meta["price_reward_log"]   = np.log1p(ad_meta["ads_reward_price"])
    ad_meta["price_contract_log"] = np.log1p(ad_meta["ads_contract_price"])
    ad_meta["price_ratio"]        = (ad_meta["ads_reward_price"] /
                                     ad_meta["ads_contract_price"].replace(0,np.nan)).fillna(0.0)

    df = df.merge(ad_meta[["ads_idx","ads_category","ads_domain","adv_idx",
                           "price_reward_log","price_contract_log","price_ratio"]],
                  on="ads_idx", how="left")

    df      = _norm_cat_col(df, "ads_category")
    cat_mda = _norm_cat_col(cat_mda, "ads_category")
    df = df.merge(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]],
                  on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0,"cat_share":0.0})

    dom_mda = dom_mda.copy()
    dom_mda["ads_domain"] = _norm_cat(dom_mda["ads_domain"])
    df["ads_domain"] = _norm_cat(df["ads_domain"])
    df = df.merge(dom_mda[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                  on=["mda_idx","ads_domain"], how="left").fillna({"dom_cvr":0.0,"dom_share":0.0})

    time_sig = actx["time_sig"]
    df = df.merge(time_sig, on="mda_idx", how="left").fillna({"mda_day_ratio":0.0,"mda_weekend_ratio":0.0})

    if actx["adv_ctx"] is not None:
        adv_ctx = actx["adv_ctx"].copy()
        df = df.merge(adv_ctx[["adv_idx","adv_cvr_global","adv_pop_norm"]], on="adv_idx", how="left")
    for c in ["adv_cvr_global","adv_pop_norm"]:
        if c not in df.columns: df[c]=0.0
    df[["adv_cvr_global","adv_pop_norm"]] = df[["adv_cvr_global","adv_pop_norm"]].fillna(0.0)

    if media_portfolio is not None:
        mp = media_portfolio.copy()
        cols = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp.columns]
        if "mda_idx" in cols: df = df.merge(mp[cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in df.columns: df[c]=0.0

    # ---- 라벨: TGT에서 클릭/전환
    tgt2 = tgt.copy()
    tgt2["clk"]  = _to_num(tgt2.get("rpt_time_clk", 0))
    tgt2["turn"] = _to_num(tgt2.get("rpt_time_turn",0))
    y_agg = (tgt2.groupby(["ads_idx","mda_idx"], as_index=False)
                   .agg(tgt_clk=("clk","sum"), tgt_turn=("turn","sum")))
    y_agg["tgt_cvr"] = (y_agg["tgt_turn"] / y_agg["tgt_clk"].replace(0,np.nan)).clip(0,1)

    df = df.merge(y_agg, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_clk":0.0,"tgt_turn":0.0})

    # 랭커용 relevance
    y = df["tgt_turn"].astype(float)
    rel = np.zeros(len(df), dtype=int)
    pos = y > 0
    if pos.sum()>0:
        ylog = np.log1p(y[pos])
        try:
            bins = pd.qcut(ylog, q=3, labels=False, duplicates="drop")
            rel[pos] = bins.astype(int)+1
        except ValueError:
            rel[pos] = 1
    df["relevance"] = rel
    return df, actx

# -------------------------------
# 6) 학습기들 (랭커 + 클릭회귀 + CVR회귀)
# -------------------------------
LABEL_GAIN = [0.0,1.0,4.0,12.0]

def ensure_feature_matrix(df, feat_cols):
    X = df.copy()
    for c in feat_cols:
        if c not in X.columns: X[c]=0.0
        if not np.issubdtype(X[c].dtype, np.number):
            X[c]=pd.to_numeric(X[c], errors="coerce")
    return X[feat_cols].fillna(0.0).astype(np.float32)

def downsample_negatives_per_ad(df, ratio=4, random_state=42):
    out, rng = [], np.random.RandomState(random_state)
    for ad, g in df.groupby("ads_idx", sort=False):
        pos = g[g["tgt_turn"]>0]; neg = g[g["tgt_turn"]<=0]
        if len(pos)==0: continue
        cap = len(pos)*ratio
        if len(neg)>cap: neg = neg.sample(n=cap, random_state=rng)
        out.append(pd.concat([pos,neg], ignore_index=True))
    return pd.concat(out, ignore_index=True) if out else df

def train_attr_ranker(train_df: pd.DataFrame):
    feat_cols = [c for c in FEAT_ALLOW if c in train_df.columns]
    X = ensure_feature_matrix(train_df, feat_cols)
    y = train_df["relevance"].astype(int)
    groups = train_df.groupby("ads_idx").size().astype(int).tolist()
    ranker = lgb.LGBMRanker(
        objective="lambdarank", metric="ndcg", eval_at=[5,10,20],
        learning_rate=0.05, n_estimators=1200, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        label_gain=LABEL_GAIN, random_state=42, n_jobs=-1, verbosity=-1
    )
    ranker.fit(X, y, group=groups, sample_weight=np.ones(len(train_df)))
    return ranker, feat_cols

def train_click_regressor(train_df: pd.DataFrame, feat_cols: List[str], tweedie_p: float = 1.1):
    X = ensure_feature_matrix(train_df, feat_cols)
    y = train_df["tgt_clk"].astype(float)
    reg = lgb.LGBMRegressor(
        objective="tweedie", tweedie_variance_power=tweedie_p,
        n_estimators=1200, learning_rate=0.05, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        random_state=42, n_jobs=-1, verbosity=-1
    )
    reg.fit(X, y)
    return reg, {"mode":"tweedie_click","twp":tweedie_p}

def _logit_clip(p, eps=1e-6):
    p = np.clip(p, eps, 1-eps)
    return np.log(p/(1-p))

def _sigmoid(x): return 1/(1+np.exp(-x))

def train_cvr_regressor(train_df: pd.DataFrame, feat_cols: List[str]):
    g = train_df[train_df["tgt_clk"]>0].copy()
    if g.empty:
        dummy = lgb.LGBMRegressor().fit(np.zeros((1, len(feat_cols))), [0.0])
        return dummy, {"mode":"cvr_logit","fallback":True, "mean_logit":-20.0}
    X = ensure_feature_matrix(g, feat_cols)
    y = g["tgt_turn"]/g["tgt_clk"]
    y = y.clip(0,1)
    y_logit = _logit_clip(y.values)
    reg = lgb.LGBMRegressor(
        objective="regression",
        n_estimators=1200, learning_rate=0.05, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        random_state=42, n_jobs=-1, verbosity=-1
    )
    reg.fit(X, y_logit)
    return reg, {"mode":"cvr_logit","fallback":False}

# -------------------------------
# 7) 예측 (랭커 스코어 + 클릭×CVR 예측)
# -------------------------------
def predict_all_mda_with_counts_batched(
    ranker, click_reg, cvr_reg, feat_cols, actx, ads_list_with_domain,
    K=20, batch_size=5000, max_predict_ads: Optional[int] = 20000,
    media_portfolio: Optional[pd.DataFrame] = None,
    excluded_mda_ids: Optional[set] = None  # ★ 추가: 예측 교차조인에서도 제외
) -> Dict[int, pd.DataFrame]:
    mda_all, cat_mda, dom_mda = actx["mda_all"], actx["cat_mda"], actx["dom_mda"]
    time_sig, adv_ctx = actx["time_sig"], actx["adv_ctx"]

    mda_active = mda_all[(mda_all["mda_clk_global"]>=200) | (mda_all["mda_conv_global"]>=20)]
    if mda_active.empty: mda_active = mda_all
    if excluded_mda_ids:
        mda_active = mda_active[~mda_active["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    mda_active = mda_active[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]].copy()
    mda_active["mda_idx"]=mda_active["mda_idx"].astype(int)
    mda_active = mda_active.merge(time_sig, on="mda_idx", how="left").fillna({"mda_day_ratio":0.0,"mda_weekend_ratio":0.0})

    if media_portfolio is not None:
        mp = media_portfolio.copy()
        cols = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp.columns]
        if "mda_idx" in cols: mda_active = mda_active.merge(mp[cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in mda_active.columns: mda_active[c]=0.0

    ads_meta = ads_list_with_domain[["ads_idx","ads_category","ads_reward_price","ads_contract_price","ads_domain","adv_idx"]].drop_duplicates("ads_idx").copy()
    if max_predict_ads is not None: ads_meta = ads_meta.head(int(max_predict_ads))
    ads_meta = _norm_cat_col(ads_meta, "ads_category")
    ads_meta = _ensure_col(ads_meta, "ads_domain", ["ads_domain","domain"])
    ads_meta["ads_idx"] = ads_meta["ads_idx"].astype(int)
    ads_meta["ads_domain"] = _norm_cat(ads_meta["ads_domain"])
    ads_meta["ads_reward_price"]   = _to_num(ads_meta["ads_reward_price"])
    ads_meta["ads_contract_price"] = _to_num(ads_meta["ads_contract_price"])
    ads_meta["price_reward_log"]   = np.log1p(ads_meta["ads_reward_price"])
    ads_meta["price_contract_log"] = np.log1p(ads_meta["ads_contract_price"])
    ads_meta["price_ratio"]        = (ads_meta["ads_reward_price"] / ads_meta["ads_contract_price"].replace(0,np.nan)).fillna(0.0)
    if adv_ctx is not None:
        ads_meta = ads_meta.merge(adv_ctx[["adv_idx","adv_cvr_global","adv_pop_norm"]], on="adv_idx", how="left")
    for c in ["adv_cvr_global","adv_pop_norm"]:
        if c not in ads_meta.columns: ads_meta[c]=0.0
    ads_meta[["adv_cvr_global","adv_pop_norm"]] = ads_meta[["adv_cvr_global","adv_pop_norm"]].fillna(0.0)

    cat_mda_local = _norm_cat_col(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]].copy(), "ads_category")
    dom_mda_local = dom_mda.copy(); dom_mda_local["ads_domain"] = _norm_cat(dom_mda_local["ads_domain"])

    out={}
    ad_ids = ads_meta["ads_idx"].tolist()
    for i in range(0, len(ad_ids), batch_size):
        ad_batch = ads_meta.iloc[i:i+batch_size]
        grid = ad_batch[["ads_idx","ads_category","ads_domain","price_reward_log","price_contract_log","price_ratio",
                         "adv_idx","adv_cvr_global","adv_pop_norm"]] \
               .merge(mda_active, how="cross")

        grid = grid.merge(cat_mda_local, on=["mda_idx","ads_category"], how="left") \
                   .fillna({"cat_cvr":0.0,"cat_share":0.0})
        grid = grid.merge(dom_mda_local[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                          on=["mda_idx","ads_domain"], how="left") \
                   .fillna({"dom_cvr":0.0,"dom_share":0.0})

        X = ensure_feature_matrix(grid, feat_cols)
        score = ranker.predict(X)  # 랭커 점수

        yhat_clk = np.maximum(0.0, click_reg.predict(X))
        yhat_cvr_logit = cvr_reg.predict(X)
        yhat_cvr = np.clip(_sigmoid(yhat_cvr_logit), 0.0, 1.0)
        yhat_turn = yhat_clk * yhat_cvr

        grid2 = grid[["ads_idx","mda_idx","ads_category","mda_cvr_global","mda_pop_norm","mda_clk_global","mda_conv_global"]].copy()
        grid2["score"] = score.astype(float)
        grid2["yhat_turn"] = yhat_turn.astype(float)

        for ad, g in grid2.groupby("ads_idx", sort=False):
            top = g.nlargest(K, "score")
            out[int(ad)] = top.sort_values("score", ascending=False).reset_index(drop=True)
        print(f"[Predict+Count] processed ads {i+len(ad_batch)}/{len(ad_ids)}")
    return out

# -------------------------------
# 8) 평가
# -------------------------------
def _precision_recall_hit_at_k(preds_dict, gt_dict, k=10):
    P,R,H=[],[],[]
    for ad, df in preds_dict.items():
        if ad not in gt_dict: continue
        pred = df["mda_idx"].astype(int).tolist()[:k]
        true = set(gt_dict[ad]); 
        if not true: continue
        inter = len(set(pred)&true)
        P.append(inter/max(1,len(pred))); R.append(inter/len(true)); H.append(1.0 if inter>0 else 0.0)
    return (float(np.mean(P)) if P else 0.0,
            float(np.mean(R)) if R else 0.0,
            float(np.mean(H)) if H else 0.0)

def _map_at_k(preds_dict, gt_dict, k=10):
    APs=[]
    for ad, df in preds_dict.items():
        if ad not in gt_dict: continue
        true=set(gt_dict[ad]); 
        if not true: continue
        ranked = df["mda_idx"].astype(int).tolist()[:k]
        hits=cum_prec=0
        for i,m in enumerate(ranked, start=1):
            if m in true:
                hits+=1; cum_prec += hits/i
        APs.append(cum_prec/min(len(true),k) if hits>0 else 0.0)
    return float(np.mean(APs)) if APs else 0.0

def _auc_flat(preds_dict, gt_pairs_dict, score_col="score"):
    y_true, y_score = [], []
    for ad, df in preds_dict.items():
        true_mdas = set(gt_pairs_dict.get(ad, []))
        for _, r in df.iterrows():
            y_true.append(1 if int(r["mda_idx"]) in true_mdas else 0)
            y_score.append(float(r[score_col]))
    if len(set(y_true))<2: return 0.5
    try: return float(roc_auc_score(y_true, y_score))
    except Exception: return 0.5

def evaluate_all(preds_with_counts: Dict[int, pd.DataFrame], tgt: pd.DataFrame, k=10):
    gt_pairs = tgt.groupby("ads_idx")["mda_idx"].apply(lambda s: list(pd.Series(s).dropna().astype(int).unique())).to_dict()
    prec, rec, hit = _precision_recall_hit_at_k(preds_with_counts, gt_pairs, k=k)
    mapk = _map_at_k(preds_with_counts, gt_pairs, k=k)
    auc_rank = _auc_flat(preds_with_counts, gt_pairs, score_col="score")
    auc_reg  = _auc_flat(preds_with_counts, gt_pairs, score_col="yhat_turn")
    return {f"precision@{k}":prec, f"recall@{k}":rec, f"map@{k}":mapk,
            f"hit_rate@{k}":hit, "auc_ranker":auc_rank, "auc_regressor":auc_reg}

# -------------------------------
# 9) 편의
# -------------------------------
def get_expected_conversions_for_ad(preds_with_counts: Dict[int, pd.DataFrame],
                                    ad_id: int, sort_by="yhat_turn", top=20) -> pd.DataFrame:
    df = preds_with_counts.get(int(ad_id))
    if df is None or df.empty:
        return pd.DataFrame(columns=["mda_idx","score","yhat_turn"])
    sort_by = "yhat_turn" if sort_by not in ("yhat_turn","score") else sort_by
    out = df.sort_values(sort_by, ascending=False).copy()
    return out.head(top) if top is not None else out

# =========================================================
# ==============   메인 플로우 (예시)   ====================
# =========================================================
# 요구: ads_time, ads_list, ads_segment(ads_size, domain), media_performance_classification, media_portfolio(옵션)

# (A) 윈도우
obs, tgt, start, end, cut = split_obs_tgt_by_dates(
    ads_time, obs_start="2025-07-25", tgt_start="2025-08-20"
)
print(f"[Window] OBS: {start.date()} → {cut.date()}  |  TGT: {(cut+pd.Timedelta(days=1)).date()} → {end.date()}")

# (A-1) 계약종료형 매체 목록 (이미 앞에서 만든 경우 이 블록 생략 가능)
# banned_mda = set(
#     media_performance_classification
#       .loc[media_performance_classification["basic_classification"].astype(str).str.strip() == "계약종료형", "mda_idx"]
#       .dropna().astype(int).unique()
# )
print(f"[Exclude] 계약종료형 mda 개수 = {len(banned_mda)}")

# (A-2) OBS/TGT에서 계약종료형 제거 (컨텍스트/학습/평가 전 과정 반영)
if banned_mda:
    obs = obs[~obs["mda_idx"].astype(int).isin(banned_mda)].copy()
    tgt = tgt[~tgt["mda_idx"].astype(int).isin(banned_mda)].copy()

# (B) ads_list 확장 (ads_size, domain 병합)
ads_list_ext = ads_list.merge(
    ads_segment[["ads_idx","ads_size","domain"]].rename(columns={"domain":"ads_domain"}),
    on="ads_idx", how="left"
)

# (C) LARGE만
ads_list_large = ads_list_ext[ads_list_ext["ads_size"].astype(str).str.strip().str.upper()=="LARGE"].copy()
large_ids = set(ads_list_large["ads_idx"].astype(int))
tgt_large  = tgt[tgt["ads_idx"].astype(int).isin(large_ids)].copy()

# (D) 학습 데이터 (계약종료형 제외 세이프가드 포함)
train_df, actx = prepare_training_attr_only(
    obs, tgt_large, ads_list_large,
    topC=10, add_random=2, cat_topM=5,
    min_active_clk=200, min_active_conv=20,
    max_train_ads=20000, min_pos_in_tgt=1,
    media_portfolio=None,
    excluded_mda_ids=banned_mda  # ★ 추가
)
print("train_df size:", len(train_df))

# (E) 학습 (랭커 + 클릭 + CVR)
train_df_ds = downsample_negatives_per_ad(train_df, ratio=4)
ranker, feat_cols   = train_attr_ranker(train_df_ds)
click_reg, click_md = train_click_regressor(train_df_ds, feat_cols, tweedie_p=1.1)
cvr_reg, cvr_md     = train_cvr_regressor(train_df_ds, feat_cols)
print("feat_cols used:", feat_cols)

# (F) 예측 & 평가 (예측 교차조인에서도 계약종료형 제거)
preds_with_counts = predict_all_mda_with_counts_batched(
    ranker, click_reg, cvr_reg, feat_cols, actx, ads_list_large,
    K=20, batch_size=5000, max_predict_ads=20000, media_portfolio=None,
    excluded_mda_ids=banned_mda  # ★ 추가
)
metrics_k5  = evaluate_all(preds_with_counts, tgt_large, k=5)
metrics_k10 = evaluate_all(preds_with_counts, tgt_large, k=10)
print("Eval@5:", metrics_k5)
print("Eval@10:", metrics_k10)

# (G) Sanity Check: 계약종료형 유입 여부
leftover_tgt = tgt_large["mda_idx"].astype(int).isin(banned_mda).sum()
print(f"[Sanity] TGT(LARGE) 내 계약종료형 잔여: {leftover_tgt}")
any_banned_in_preds = any(
    int(m) in banned_mda
    for df in preds_with_counts.values()
    for m in df["mda_idx"].astype(int).tolist()
)
print(f"[Sanity] 예측결과에 계약종료형 포함 여부: {any_banned_in_preds}")

# (H) 예: 특정 광고 top-5 (예상 전환수 기준)
# ad_id = 446406
# top5 = get_expected_conversions_for_ad(preds_with_counts, ad_id, sort_by="yhat_turn", top=5)
# print(top5[["mda_idx","yhat_turn","score"]])



In [ ]:
# =========================================================
#  Ad × Media 추천 + 전환수 예측 (Decomposition: Click × CVR, OS 제외)
#  - 누수 방지: 모든 피처는 OBS에서만 생성, 라벨은 TGT
#  - 클릭: Tweedie 회귀(권장), CVR: logit 회귀
#  - 예측: yhat_turn = yhat_click × yhat_cvr
#  - 계약종료형(banned_mda) 매체 전 과정 제외
# =========================================================
# 전제: import pandas as pd, numpy as np, lightgbm as lgb, warnings
#       from typing import *
#       from sklearn.metrics import roc_auc_score
warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm")

# -------------------------------
# 유틸
# -------------------------------
def _to_num(s): return pd.to_numeric(s, errors="coerce").fillna(0)
def _to_dt(s):  return pd.to_datetime(s, errors="coerce")

def _norm_cat_col(df: pd.DataFrame, col: str) -> pd.DataFrame:
    if col in df.columns:
        df = df.copy()
        df[col] = df[col].astype(str).str.strip().str.lower()
        df[col] = df[col].replace({"<na>":"__na__", "nan":"__na__", "none":"__na__", "nat":"__na__"})
    return df

def _norm_cat(s): return pd.Series(s, copy=False).astype("string").str.strip().str.lower()

def _pick_col(cols, cand):
    for c in cand:
        if c in cols: return c
    return None

def _ensure_col(df: pd.DataFrame, out_col: str, cands: List[str]) -> pd.DataFrame:
    col = _pick_col(df.columns, cands)
    df = df.copy()
    if col is None: df[out_col] = "__na__"
    else:           df[out_col] = _norm_cat(df[col])
    return df

# -------------------------------
# 1) 윈도우
# -------------------------------
def split_obs_tgt_by_dates(
    ads_time: pd.DataFrame,
    obs_start: str = "2025-07-25",
    tgt_start: str = "2025-08-20",
    obs_end: Optional[str] = None,
    tgt_end: Optional[str] = None,
    tgt_max_days: int = 6
):
    t = ads_time.copy()
    t["date"] = _to_dt(t["rpt_time_date"])
    if t["date"].isna().all():
        raise ValueError("ads_time['rpt_time_date']에 유효한 날짜가 없습니다.")
    obs_start = pd.to_datetime(obs_start).normalize()
    tgt_start = pd.to_datetime(tgt_start).normalize()
    obs_end = (pd.to_datetime(obs_end).normalize() if obs_end is not None
               else (tgt_start - pd.Timedelta(days=1)).normalize())
    last_date = t["date"].max().normalize()
    tgt_end = (pd.to_datetime(tgt_end).normalize() if tgt_end is not None
               else min(last_date, (tgt_start + pd.Timedelta(days=tgt_max_days)).normalize()))
    obs = t[(t["date"] >= obs_start) & (t["date"] <= obs_end)].copy()
    tgt = t[(t["date"] >= tgt_start) & (t["date"] <= tgt_end)].copy()
    return obs, tgt, obs_start, tgt_end, (tgt_start - pd.Timedelta(days=1)).normalize()

# -------------------------------
# 2) 시간/매체/카테고리/도메인 컨텍스트 (OBS만)
# -------------------------------
def _build_time_signals(obs: pd.DataFrame) -> pd.DataFrame:
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["weekday"] = _to_dt(t["rpt_time_date"]).dt.weekday  # 0~6
    wd = t.groupby(["mda_idx","weekday"], as_index=False).agg(clk=("clk","sum"))
    wd["is_weekend"] = wd["weekday"].isin([5,6]).astype(int)
    g = wd.groupby("mda_idx", as_index=False).agg(clk_total=("clk","sum"))
    w = wd.merge(g, on="mda_idx", how="left")
    day_ratio = (w.assign(ratio=w["clk"]/w["clk_total"].replace(0,np.nan))
                   .sort_values(["mda_idx","ratio"], ascending=[True,False])
                   .groupby("mda_idx").head(1)[["mda_idx","ratio"]]
                   .rename(columns={"ratio":"mda_day_ratio"}))
    weekend_ratio = (w.groupby(["mda_idx","is_weekend"], as_index=False)
                       .agg(clk=("clk","sum")).merge(g,on="mda_idx",how="left"))
    weekend_ratio["mda_weekend_ratio"] = (weekend_ratio["clk"] /
                                          weekend_ratio["clk_total"].replace(0,np.nan)).fillna(0)
    weekend_ratio = weekend_ratio[weekend_ratio["is_weekend"]==1][["mda_idx","mda_weekend_ratio"]]
    out = day_ratio.merge(weekend_ratio, on="mda_idx", how="outer").fillna(0.0)
    out["mda_idx"] = out["mda_idx"].astype(int)
    return out

def build_attr_context(obs: pd.DataFrame, ads_list_with_domain: pd.DataFrame):
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["conv"] = _to_num(t.get("rpt_time_turn", 0))

    # mda 전역
    mda_all = (t.groupby("mda_idx", as_index=False)
                 .agg(mda_clk_global=("clk","sum"), mda_conv_global=("conv","sum")))
    mda_all["mda_cvr_global"] = (mda_all["mda_conv_global"] /
                                 mda_all["mda_clk_global"].replace(0,np.nan)).fillna(0.0)
    mu, sd = mda_all["mda_conv_global"].mean(), mda_all["mda_conv_global"].std(ddof=0)
    mda_all["mda_pop_norm"] = (mda_all["mda_conv_global"] - mu) / (sd + 1e-9)

    # 카테고리×매체
    al = ads_list_with_domain[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    al["ads_category"] = _norm_cat(al["ads_category"])
    t2 = t.merge(al, on="ads_idx", how="left")
    cat_mda = (t2.groupby(["mda_idx","ads_category"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    cat_mda["cat_cvr"] = (cat_mda["conv"]/cat_mda["clk"].replace(0,np.nan)).fillna(0.0)
    mda_tot = cat_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    cat_mda = cat_mda.merge(mda_tot, on="mda_idx", how="left")
    cat_mda["cat_share"] = (cat_mda["conv"]/cat_mda["mda_conv_total"].replace(0,np.nan)).fillna(0.0)

    # 도메인×매체 (ads_list 도메인 기반)
    t_dom = t.merge(
        ads_list_with_domain[["ads_idx","ads_domain"]].drop_duplicates("ads_idx"),
        on="ads_idx", how="left"
    )
    t_dom["ads_domain"] = _norm_cat(t_dom["ads_domain"])
    dom_mda = (t_dom.groupby(["mda_idx","ads_domain"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    dom_mda["dom_cvr"] = (dom_mda["conv"]/dom_mda["clk"].replace(0,np.nan)).fillna(0.0)
    mda_tot2 = dom_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    dom_mda = dom_mda.merge(mda_tot2, on="mda_idx", how="left")
    dom_mda["dom_share"] = (dom_mda["conv"]/dom_mda["mda_conv_total"].replace(0,np.nan)).fillna(0.0)

    # 시간 신호
    time_sig = _build_time_signals(obs)

    # (선택) 광고주 신호
    adv_ctx = None
    if "adv_idx" in ads_list_with_domain.columns:
        t_adv = t.merge(ads_list_with_domain[["ads_idx","adv_idx"]].drop_duplicates("ads_idx"),
                        on="ads_idx", how="left")
        adv_all = (t_adv.groupby("adv_idx", as_index=False)
                        .agg(adv_clk_global=("clk","sum"),
                             adv_conv_global=("conv","sum")))
        adv_all["adv_cvr_global"] = (adv_all["adv_conv_global"] /
                                     adv_all["adv_clk_global"].replace(0,np.nan)).fillna(0.0)
        mu2, sd2 = adv_all["adv_conv_global"].mean(), adv_all["adv_conv_global"].std(ddof=0)
        adv_all["adv_pop_norm"] = (adv_all["adv_conv_global"] - mu2) / (sd2 + 1e-9)
        adv_ctx = adv_all

    return {"mda_all": mda_all, "cat_mda": cat_mda, "dom_mda": dom_mda,
            "time_sig": time_sig, "adv_ctx": adv_ctx}

# -------------------------------
# 3) 학습 대상 광고
# -------------------------------
def pick_train_ads_from_tgt(tgt: pd.DataFrame, min_pos_in_tgt: int = 1,
                            max_train_ads: Optional[int] = 20000, rng_seed: int = 42) -> np.ndarray:
    t = tgt.copy()
    t["turn"] = _to_num(t.get("rpt_time_turn", 0))
    pos = (t.groupby("ads_idx")["turn"].apply(lambda s: (s > 0).sum())
             .reset_index(name="pos_cnt"))
    ok = pos[pos["pos_cnt"] >= max(1,int(min_pos_in_tgt))]["ads_idx"].astype(int).values
    if max_train_ads is not None and len(ok) > max_train_ads:
        rng = np.random.RandomState(rng_seed)
        ok = rng.choice(ok, size=max_train_ads, replace=False)
    return np.sort(ok)

# -------------------------------
# 4) 후보 생성
# -------------------------------
def create_candidates(ads_list: pd.DataFrame, mda_all: pd.DataFrame, cat_mda: pd.DataFrame,
                      topC=10, add_random=2, cat_topM=5,
                      min_active_clk=200, min_active_conv=20,
                      rng_seed=42, ad_ids_filter=None, max_per_ad=20):
    mda_active = mda_all[(mda_all["mda_clk_global"]>=min_active_clk) |
                         (mda_all["mda_conv_global"]>=min_active_conv)].copy()
    if mda_active.empty: mda_active = mda_all.copy()
    mda_ids_global = (mda_active.sort_values("mda_conv_global", ascending=False)["mda_idx"]
                      .astype(int).values)
    cat_top = (cat_mda.sort_values(["ads_category","cat_cvr"], ascending=[True,False])
                      .groupby("ads_category").head(cat_topM)[["ads_category","mda_idx"]])
    cat2top = {c: g["mda_idx"].astype(int).tolist() for c,g in cat_top.groupby("ads_category")}
    rng = np.random.RandomState(rng_seed)
    ad_ids = (np.array(ad_ids_filter, dtype=int) if (ad_ids_filter is not None and len(ad_ids_filter)>0)
              else ads_list["ads_idx"].dropna().astype(int).unique())
    ad_meta = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    ad_meta["ads_idx"] = ad_meta["ads_idx"].astype(int)
    ad_meta["ads_category"] = ad_meta["ads_category"].astype(str).str.strip().str.lower()
    ad2cat = ad_meta.set_index("ads_idx")["ads_category"].to_dict()

    pairs=[]
    for ad in ad_ids:
        base=[]
        cat = ad2cat.get(int(ad), None)
        if cat in cat2top: base.extend(cat2top[cat])
        base.extend(mda_ids_global[:min(topC,len(mda_ids_global))])
        base=list(dict.fromkeys(base))
        remain=np.setdiff1d(mda_ids_global, np.array(base, dtype=int))
        if len(remain)>0 and add_random>0:
            base.extend(rng.choice(remain, size=min(add_random,len(remain)), replace=False).tolist())
        if len(base)>max_per_ad: base=base[:max_per_ad]
        pairs += [(int(ad), int(m)) for m in base]
    out = pd.DataFrame(pairs, columns=["ads_idx","mda_idx"]).drop_duplicates()
    print(f"[Candidates] pairs={len(out):,} | ads={len(ad_ids):,} | active_mda={len(mda_active):,} | topC={topC}, rnd={add_random}, cat_topM={cat_topM}, per_ad≤{max_per_ad}")
    return out

# -------------------------------
# 5) 피처 & 라벨 (OBS 피처 / TGT 라벨)
# -------------------------------
FEAT_ALLOW = [
    # mda 전역
    "mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm",
    # 카테고리×매체
    "cat_cvr","cat_share",
    # 광고 가격
    "price_reward_log","price_contract_log","price_ratio",
    # 도메인×매체
    "dom_cvr","dom_share",
    # 시간 신호
    "mda_day_ratio","mda_weekend_ratio",
    # 광고주 신호
    "adv_cvr_global","adv_pop_norm",
    # 외부 포트폴리오가 있으면
    "mda_size","active_ratio","day_ratio","weekend_ratio"
]

def prepare_training_attr_only(
    obs: pd.DataFrame, tgt: pd.DataFrame, ads_list_with_domain: pd.DataFrame,
    topC=10, add_random=2, cat_topM=5, min_active_clk=200, min_active_conv=20,
    max_train_ads: Optional[int] = 20000, min_pos_in_tgt: int = 1,
    media_portfolio: Optional[pd.DataFrame] = None,
    excluded_mda_ids: Optional[set] = None  # ★ 추가: 학습 데이터에서 제외
):
    # ★ mda 제외(OBS/TGT에서 이미 제거했더라도 안전망)
    if excluded_mda_ids:
        obs = obs[~obs["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()
        tgt = tgt[~tgt["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    train_ads = pick_train_ads_from_tgt(tgt, min_pos_in_tgt=min_pos_in_tgt, max_train_ads=max_train_ads)
    actx = build_attr_context(obs, ads_list_with_domain)
    mda_all, cat_mda, dom_mda = actx["mda_all"], actx["cat_mda"], actx["dom_mda"]

    cand = create_candidates(
        ads_list=ads_list_with_domain, mda_all=mda_all, cat_mda=cat_mda,
        topC=topC, add_random=add_random, cat_topM=cat_topM,
        min_active_clk=min_active_clk, min_active_conv=min_active_conv,
        ad_ids_filter=train_ads, max_per_ad=20
    )

    # ★ 후보 그리드에서도 제외
    if excluded_mda_ids:
        cand = cand[~cand["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    df = cand.merge(
        mda_all[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]],
        on="mda_idx", how="left"
    )

    ad_meta = ads_list_with_domain[["ads_idx","ads_category","ads_reward_price","ads_contract_price","ads_domain","adv_idx"]].drop_duplicates("ads_idx").copy()
    ad_meta = _norm_cat_col(ad_meta, "ads_category")
    ad_meta = _ensure_col(ad_meta, "ads_domain", ["ads_domain","domain"])
    for c in ["ads_reward_price","ads_contract_price"]:
        ad_meta[c] = _to_num(ad_meta[c])
    ad_meta["price_reward_log"]   = np.log1p(ad_meta["ads_reward_price"])
    ad_meta["price_contract_log"] = np.log1p(ad_meta["ads_contract_price"])
    ad_meta["price_ratio"]        = (ad_meta["ads_reward_price"] /
                                     ad_meta["ads_contract_price"].replace(0,np.nan)).fillna(0.0)

    df = df.merge(ad_meta[["ads_idx","ads_category","ads_domain","adv_idx",
                           "price_reward_log","price_contract_log","price_ratio"]],
                  on="ads_idx", how="left")

    df      = _norm_cat_col(df, "ads_category")
    cat_mda = _norm_cat_col(cat_mda, "ads_category")
    df = df.merge(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]],
                  on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0,"cat_share":0.0})

    dom_mda = dom_mda.copy()
    dom_mda["ads_domain"] = _norm_cat(dom_mda["ads_domain"])
    df["ads_domain"] = _norm_cat(df["ads_domain"])
    df = df.merge(dom_mda[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                  on=["mda_idx","ads_domain"], how="left").fillna({"dom_cvr":0.0,"dom_share":0.0})

    time_sig = actx["time_sig"]
    df = df.merge(time_sig, on="mda_idx", how="left").fillna({"mda_day_ratio":0.0,"mda_weekend_ratio":0.0})

    if actx["adv_ctx"] is not None:
        adv_ctx = actx["adv_ctx"].copy()
        df = df.merge(adv_ctx[["adv_idx","adv_cvr_global","adv_pop_norm"]], on="adv_idx", how="left")
    for c in ["adv_cvr_global","adv_pop_norm"]:
        if c not in df.columns: df[c]=0.0
    df[["adv_cvr_global","adv_pop_norm"]] = df[["adv_cvr_global","adv_pop_norm"]].fillna(0.0)

    if media_portfolio is not None:
        mp = media_portfolio.copy()
        cols = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp.columns]
        if "mda_idx" in cols: df = df.merge(mp[cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in df.columns: df[c]=0.0

    # ---- 라벨: TGT에서 클릭/전환
    tgt2 = tgt.copy()
    tgt2["clk"]  = _to_num(tgt2.get("rpt_time_clk", 0))
    tgt2["turn"] = _to_num(tgt2.get("rpt_time_turn",0))
    y_agg = (tgt2.groupby(["ads_idx","mda_idx"], as_index=False)
                   .agg(tgt_clk=("clk","sum"), tgt_turn=("turn","sum")))
    y_agg["tgt_cvr"] = (y_agg["tgt_turn"] / y_agg["tgt_clk"].replace(0,np.nan)).clip(0,1)

    df = df.merge(y_agg, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_clk":0.0,"tgt_turn":0.0})

    # 랭커용 relevance
    y = df["tgt_turn"].astype(float)
    rel = np.zeros(len(df), dtype=int)
    pos = y > 0
    if pos.sum()>0:
        ylog = np.log1p(y[pos])
        try:
            bins = pd.qcut(ylog, q=3, labels=False, duplicates="drop")
            rel[pos] = bins.astype(int)+1
        except ValueError:
            rel[pos] = 1
    df["relevance"] = rel
    return df, actx

# -------------------------------
# 6) 학습기들 (랭커 + 클릭회귀 + CVR회귀)
# -------------------------------
LABEL_GAIN = [0.0,1.0,4.0,12.0]

def ensure_feature_matrix(df, feat_cols):
    X = df.copy()
    for c in feat_cols:
        if c not in X.columns: X[c]=0.0
        if not np.issubdtype(X[c].dtype, np.number):
            X[c]=pd.to_numeric(X[c], errors="coerce")
    return X[feat_cols].fillna(0.0).astype(np.float32)

def downsample_negatives_per_ad(df, ratio=4, random_state=42):
    out, rng = [], np.random.RandomState(random_state)
    for ad, g in df.groupby("ads_idx", sort=False):
        pos = g[g["tgt_turn"]>0]; neg = g[g["tgt_turn"]<=0]
        if len(pos)==0: continue
        cap = len(pos)*ratio
        if len(neg)>cap: neg = neg.sample(n=cap, random_state=rng)
        out.append(pd.concat([pos,neg], ignore_index=True))
    return pd.concat(out, ignore_index=True) if out else df

def train_attr_ranker(train_df: pd.DataFrame):
    feat_cols = [c for c in FEAT_ALLOW if c in train_df.columns]
    X = ensure_feature_matrix(train_df, feat_cols)
    y = train_df["relevance"].astype(int)
    groups = train_df.groupby("ads_idx").size().astype(int).tolist()
    ranker = lgb.LGBMRanker(
        objective="lambdarank", metric="ndcg", eval_at=[5,10,20],
        learning_rate=0.05, n_estimators=1200, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        label_gain=LABEL_GAIN, random_state=42, n_jobs=-1, verbosity=-1
    )
    ranker.fit(X, y, group=groups, sample_weight=np.ones(len(train_df)))
    return ranker, feat_cols

def train_click_regressor(train_df: pd.DataFrame, feat_cols: List[str], tweedie_p: float = 1.1):
    X = ensure_feature_matrix(train_df, feat_cols)
    y = train_df["tgt_clk"].astype(float)
    reg = lgb.LGBMRegressor(
        objective="tweedie", tweedie_variance_power=tweedie_p,
        n_estimators=1200, learning_rate=0.05, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        random_state=42, n_jobs=-1, verbosity=-1
    )
    reg.fit(X, y)
    return reg, {"mode":"tweedie_click","twp":tweedie_p}

def _logit_clip(p, eps=1e-6):
    p = np.clip(p, eps, 1-eps)
    return np.log(p/(1-p))

def _sigmoid(x): return 1/(1+np.exp(-x))

def train_cvr_regressor(train_df: pd.DataFrame, feat_cols: List[str]):
    g = train_df[train_df["tgt_clk"]>0].copy()
    if g.empty:
        dummy = lgb.LGBMRegressor().fit(np.zeros((1, len(feat_cols))), [0.0])
        return dummy, {"mode":"cvr_logit","fallback":True, "mean_logit":-20.0}
    X = ensure_feature_matrix(g, feat_cols)
    y = g["tgt_turn"]/g["tgt_clk"]
    y = y.clip(0,1)
    y_logit = _logit_clip(y.values)
    reg = lgb.LGBMRegressor(
        objective="regression",
        n_estimators=1200, learning_rate=0.05, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        random_state=42, n_jobs=-1, verbosity=-1
    )
    reg.fit(X, y_logit)
    return reg, {"mode":"cvr_logit","fallback":False}

# -------------------------------
# 7) 예측 (랭커 스코어 + 클릭×CVR 예측)
# -------------------------------
def predict_all_mda_with_counts_batched(
    ranker, click_reg, cvr_reg, feat_cols, actx, ads_list_with_domain,
    K=20, batch_size=5000, max_predict_ads: Optional[int] = 20000,
    media_portfolio: Optional[pd.DataFrame] = None,
    excluded_mda_ids: Optional[set] = None  # ★ 추가: 예측 교차조인에서도 제외
) -> Dict[int, pd.DataFrame]:
    mda_all, cat_mda, dom_mda = actx["mda_all"], actx["cat_mda"], actx["dom_mda"]
    time_sig, adv_ctx = actx["time_sig"], actx["adv_ctx"]

    mda_active = mda_all[(mda_all["mda_clk_global"]>=200) | (mda_all["mda_conv_global"]>=20)]
    if mda_active.empty: mda_active = mda_all
    if excluded_mda_ids:
        mda_active = mda_active[~mda_active["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    mda_active = mda_active[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]].copy()
    mda_active["mda_idx"]=mda_active["mda_idx"].astype(int)
    mda_active = mda_active.merge(time_sig, on="mda_idx", how="left").fillna({"mda_day_ratio":0.0,"mda_weekend_ratio":0.0})

    if media_portfolio is not None:
        mp = media_portfolio.copy()
        cols = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp.columns]
        if "mda_idx" in cols: mda_active = mda_active.merge(mp[cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in mda_active.columns: mda_active[c]=0.0

    ads_meta = ads_list_with_domain[["ads_idx","ads_category","ads_reward_price","ads_contract_price","ads_domain","adv_idx"]].drop_duplicates("ads_idx").copy()
    if max_predict_ads is not None: ads_meta = ads_meta.head(int(max_predict_ads))
    ads_meta = _norm_cat_col(ads_meta, "ads_category")
    ads_meta = _ensure_col(ads_meta, "ads_domain", ["ads_domain","domain"])
    ads_meta["ads_idx"] = ads_meta["ads_idx"].astype(int)
    ads_meta["ads_domain"] = _norm_cat(ads_meta["ads_domain"])
    ads_meta["ads_reward_price"]   = _to_num(ads_meta["ads_reward_price"])
    ads_meta["ads_contract_price"] = _to_num(ads_meta["ads_contract_price"])
    ads_meta["price_reward_log"]   = np.log1p(ads_meta["ads_reward_price"])
    ads_meta["price_contract_log"] = np.log1p(ads_meta["ads_contract_price"])
    ads_meta["price_ratio"]        = (ads_meta["ads_reward_price"] / ads_meta["ads_contract_price"].replace(0,np.nan)).fillna(0.0)
    if adv_ctx is not None:
        ads_meta = ads_meta.merge(adv_ctx[["adv_idx","adv_cvr_global","adv_pop_norm"]], on="adv_idx", how="left")
    for c in ["adv_cvr_global","adv_pop_norm"]:
        if c not in ads_meta.columns: ads_meta[c]=0.0
    ads_meta[["adv_cvr_global","adv_pop_norm"]] = ads_meta[["adv_cvr_global","adv_pop_norm"]].fillna(0.0)

    cat_mda_local = _norm_cat_col(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]].copy(), "ads_category")
    dom_mda_local = dom_mda.copy(); dom_mda_local["ads_domain"] = _norm_cat(dom_mda_local["ads_domain"])

    out={}
    ad_ids = ads_meta["ads_idx"].tolist()
    for i in range(0, len(ad_ids), batch_size):
        ad_batch = ads_meta.iloc[i:i+batch_size]
        grid = ad_batch[["ads_idx","ads_category","ads_domain","price_reward_log","price_contract_log","price_ratio",
                         "adv_idx","adv_cvr_global","adv_pop_norm"]] \
               .merge(mda_active, how="cross")

        grid = grid.merge(cat_mda_local, on=["mda_idx","ads_category"], how="left") \
                   .fillna({"cat_cvr":0.0,"cat_share":0.0})
        grid = grid.merge(dom_mda_local[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                          on=["mda_idx","ads_domain"], how="left") \
                   .fillna({"dom_cvr":0.0,"dom_share":0.0})

        X = ensure_feature_matrix(grid, feat_cols)
        score = ranker.predict(X)  # 랭커 점수

        yhat_clk = np.maximum(0.0, click_reg.predict(X))
        yhat_cvr_logit = cvr_reg.predict(X)
        yhat_cvr = np.clip(_sigmoid(yhat_cvr_logit), 0.0, 1.0)
        yhat_turn = yhat_clk * yhat_cvr

        grid2 = grid[["ads_idx","mda_idx","ads_category","mda_cvr_global","mda_pop_norm","mda_clk_global","mda_conv_global"]].copy()
        grid2["score"] = score.astype(float)
        grid2["yhat_turn"] = yhat_turn.astype(float)

        for ad, g in grid2.groupby("ads_idx", sort=False):
            top = g.nlargest(K, "score")
            out[int(ad)] = top.sort_values("score", ascending=False).reset_index(drop=True)
        print(f"[Predict+Count] processed ads {i+len(ad_batch)}/{len(ad_ids)}")
    return out

# -------------------------------
# 8) 평가
# -------------------------------
def _precision_recall_hit_at_k(preds_dict, gt_dict, k=10):
    P,R,H=[],[],[]
    for ad, df in preds_dict.items():
        if ad not in gt_dict: continue
        pred = df["mda_idx"].astype(int).tolist()[:k]
        true = set(gt_dict[ad]); 
        if not true: continue
        inter = len(set(pred)&true)
        P.append(inter/max(1,len(pred))); R.append(inter/len(true)); H.append(1.0 if inter>0 else 0.0)
    return (float(np.mean(P)) if P else 0.0,
            float(np.mean(R)) if R else 0.0,
            float(np.mean(H)) if H else 0.0)

def _map_at_k(preds_dict, gt_dict, k=10):
    APs=[]
    for ad, df in preds_dict.items():
        if ad not in gt_dict: continue
        true=set(gt_dict[ad]); 
        if not true: continue
        ranked = df["mda_idx"].astype(int).tolist()[:k]
        hits=cum_prec=0
        for i,m in enumerate(ranked, start=1):
            if m in true:
                hits+=1; cum_prec += hits/i
        APs.append(cum_prec/min(len(true),k) if hits>0 else 0.0)
    return float(np.mean(APs)) if APs else 0.0

def _auc_flat(preds_dict, gt_pairs_dict, score_col="score"):
    y_true, y_score = [], []
    for ad, df in preds_dict.items():
        true_mdas = set(gt_pairs_dict.get(ad, []))
        for _, r in df.iterrows():
            y_true.append(1 if int(r["mda_idx"]) in true_mdas else 0)
            y_score.append(float(r[score_col]))
    if len(set(y_true))<2: return 0.5
    try: return float(roc_auc_score(y_true, y_score))
    except Exception: return 0.5

def evaluate_all(preds_with_counts: Dict[int, pd.DataFrame], tgt: pd.DataFrame, k=10):
    gt_pairs = tgt.groupby("ads_idx")["mda_idx"].apply(lambda s: list(pd.Series(s).dropna().astype(int).unique())).to_dict()
    prec, rec, hit = _precision_recall_hit_at_k(preds_with_counts, gt_pairs, k=k)
    mapk = _map_at_k(preds_with_counts, gt_pairs, k=k)
    auc_rank = _auc_flat(preds_with_counts, gt_pairs, score_col="score")
    auc_reg  = _auc_flat(preds_with_counts, gt_pairs, score_col="yhat_turn")
    return {f"precision@{k}":prec, f"recall@{k}":rec, f"map@{k}":mapk,
            f"hit_rate@{k}":hit, "auc_ranker":auc_rank, "auc_regressor":auc_reg}

# -------------------------------
# 9) 편의
# -------------------------------
def get_expected_conversions_for_ad(preds_with_counts: Dict[int, pd.DataFrame],
                                    ad_id: int, sort_by="yhat_turn", top=20) -> pd.DataFrame:
    df = preds_with_counts.get(int(ad_id))
    if df is None or df.empty:
        return pd.DataFrame(columns=["mda_idx","score","yhat_turn"])
    sort_by = "yhat_turn" if sort_by not in ("yhat_turn","score") else sort_by
    out = df.sort_values(sort_by, ascending=False).copy()
    return out.head(top) if top is not None else out

# =========================================================
# ==============   메인 플로우 (예시)   ====================
# =========================================================
# 요구: ads_time, ads_list, ads_segment(ads_size, domain), media_performance_classification, media_portfolio(옵션)

# (A) 윈도우
obs, tgt, start, end, cut = split_obs_tgt_by_dates(
    ads_time, obs_start="2025-07-25", tgt_start="2025-08-20"
)
print(f"[Window] OBS: {start.date()} → {cut.date()}  |  TGT: {(cut+pd.Timedelta(days=1)).date()} → {end.date()}")

# (A-1) 계약종료형 매체 목록 (이미 앞에서 만든 경우 이 블록 생략 가능)
# banned_mda = set(
#     media_performance_classification
#       .loc[media_performance_classification["basic_classification"].astype(str).str.strip() == "계약종료형", "mda_idx"]
#       .dropna().astype(int).unique()
# )
print(f"[Exclude] 계약종료형 mda 개수 = {len(banned_mda)}")

# (A-2) OBS/TGT에서 계약종료형 제거 (컨텍스트/학습/평가 전 과정 반영)
if banned_mda:
    obs = obs[~obs["mda_idx"].astype(int).isin(banned_mda)].copy()
    tgt = tgt[~tgt["mda_idx"].astype(int).isin(banned_mda)].copy()

# (B) ads_list 확장 (ads_size, domain 병합)
ads_list_ext = ads_list.merge(
    ads_segment[["ads_idx","ads_size","domain"]].rename(columns={"domain":"ads_domain"}),
    on="ads_idx", how="left"
)

# (C) MEGA/LARGE/MEDIUM만
size_whitelist = {"MEGA","LARGE","MEDIUM"}
ads_list_sel = ads_list_ext[
    ads_list_ext["ads_size"].astype(str).str.strip().str.upper().isin(size_whitelist)
].copy()
sel_ids = set(ads_list_sel["ads_idx"].astype(int))

tgt_sel  = tgt[tgt["ads_idx"].astype(int).isin(sel_ids)].copy()

print(f"[Filter] sizes kept = {sorted(size_whitelist)} | ads={ads_list_sel['ads_idx'].nunique():,} | tgt pairs={len(tgt_sel):,}")

# (D) 학습 데이터 (계약종료형 제외 세이프가드 포함)
train_df, actx = prepare_training_attr_only(
    obs, tgt_sel, ads_list_sel,
    topC=10, add_random=2, cat_topM=5,
    min_active_clk=200, min_active_conv=20,
    max_train_ads=20000, min_pos_in_tgt=1,
    media_portfolio=None,
    excluded_mda_ids=banned_mda  # ★ 유지
)
print("train_df size:", len(train_df))

# (E) 학습 (랭커 + 클릭 + CVR)
train_df_ds = downsample_negatives_per_ad(train_df, ratio=4)
ranker, feat_cols   = train_attr_ranker(train_df_ds)
click_reg, click_md = train_click_regressor(train_df_ds, feat_cols, tweedie_p=1.1)
cvr_reg, cvr_md     = train_cvr_regressor(train_df_ds, feat_cols)
print("feat_cols used:", feat_cols)

# (F) 예측 & 평가 (예측 교차조인에서도 계약종료형 제거)
preds_with_counts = predict_all_mda_with_counts_batched(
    ranker, click_reg, cvr_reg, feat_cols, actx, ads_list_sel,
    K=20, batch_size=5000, max_predict_ads=20000, media_portfolio=None,
    excluded_mda_ids=banned_mda  # ★ 유지
)
metrics_k5  = evaluate_all(preds_with_counts, tgt_sel, k=5)
metrics_k10 = evaluate_all(preds_with_counts, tgt_sel, k=10)
print("Eval@5:", metrics_k5)
print("Eval@10:", metrics_k10)

# (G) Sanity Check: 계약종료형 유입 여부
leftover_tgt = tgt_sel["mda_idx"].astype(int).isin(banned_mda).sum()
print(f"[Sanity] TGT(selected sizes) 내 계약종료형 잔여: {leftover_tgt}")
any_banned_in_preds = any(
    int(m) in banned_mda
    for df in preds_with_counts.values()
    for m in df["mda_idx"].astype(int).tolist()
)
print(f"[Sanity] 예측결과에 계약종료형 포함 여부: {any_banned_in_preds}")

# (H) 예: 특정 광고 top-5 (예상 전환수 기준)
# ad_id = 446406
# top5 = get_expected_conversions_for_ad(preds_with_counts, ad_id, sort_by="yhat_turn", top=5)
# print(top5[["mda_idx","yhat_turn","score"]])


[Window] OBS: 2025-07-25 → 2025-08-19  |  TGT: 2025-08-20 → 2025-08-25
[Exclude] 계약종료형 mda 개수 = 51
[Filter] sizes kept = ['LARGE', 'MEDIUM', 'MEGA'] | ads=7,169 | tgt pairs=63,274
[Candidates] pairs=26,263 | ads=1,565 | active_mda=128 | topC=10, rnd=2, cat_topM=5, per_ad≤20
train_df size: 26263
feat_cols used: ['mda_clk_global', 'mda_conv_global', 'mda_cvr_global', 'mda_pop_norm', 'cat_cvr', 'cat_share', 'price_reward_log', 'price_contract_log', 'price_ratio', 'dom_cvr', 'dom_share', 'mda_day_ratio', 'mda_weekend_ratio', 'adv_cvr_global', 'adv_pop_norm', 'mda_size', 'active_ratio', 'day_ratio', 'weekend_ratio']
[Predict+Count] processed ads 5000/7169
[Predict+Count] processed ads 7169/7169
Eval@5: {'precision@5': 0.2850102669404518, 'recall@5': 0.7922062310387551, 'map@5': 0.6797374800365047, 'hit_rate@5': 0.9040041067761807, 'auc_ranker': 0.8820701576967978, 'auc_regressor': 0.8492052808958677}
Eval@10: {'precision@10': 0.17109856262833675, 'recall@10': 0.875213873423343, 'map@10': 0.

In [ ]:
def evaluate_regression(preds_with_counts: Dict[int, pd.DataFrame], tgt: pd.DataFrame):
    # 실제 라벨 (tgt) 준비
    y_true, y_pred = [], []
    gt = (tgt.groupby(["ads_idx","mda_idx"], as_index=False)
              .agg(tgt_turn=("rpt_time_turn","sum")))
    
    for ad, df in preds_with_counts.items():
        df = df[["ads_idx","mda_idx","yhat_turn"]].copy()
        merged = df.merge(gt, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_turn":0})
        y_true.extend(merged["tgt_turn"].astype(float).tolist())
        y_pred.extend(merged["yhat_turn"].astype(float).tolist())
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-9))) * 100  # % 단위
    
    return {"MAE": mae, "RMSE": rmse, "R2": r2, "MAPE(%)": mape}


In [ ]:
reg_metrics = evaluate_regression(preds_with_counts, tgt_sel)
print("Regression Metrics:", reg_metrics)


Regression Metrics: {'MAE': 3.9282948000188247, 'RMSE': np.float64(176.320776602396), 'R2': 0.24203177838951195, 'MAPE(%)': np.float64(316630311004.45374)}


# 이 모델 사용

In [123]:
# =========================================================
#  Ad × Media 추천 + 전환수 예측 (Decomposition: Click × CVR, OS 제외)
#  - 누수 방지: 모든 피처는 OBS에서만 생성, 라벨은 TGT
#  - 클릭: Tweedie 회귀(권장), CVR: logit 회귀
#  - 예측: yhat_turn = yhat_click × yhat_cvr
#  - 계약종료형(banned_mda) 매체 & 현재 진행중 페어(running_pairs) 전 과정 제외
# =========================================================
# 전제 패키지:
#   pip install pandas numpy lightgbm pyarrow scikit-learn
# =========================================================

warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm")

# -------------------------------
# 유틸
# -------------------------------
def _to_num(s): return pd.to_numeric(s, errors="coerce").fillna(0)
def _to_dt(s):  return pd.to_datetime(s, errors="coerce")

def _norm_cat_col(df: pd.DataFrame, col: str) -> pd.DataFrame:
    if col in df.columns:
        df = df.copy()
        df[col] = df[col].astype(str).str.strip().str.lower()
        df[col] = df[col].replace({"<na>":"__na__", "nan":"__na__", "none":"__na__", "nat":"__na__"})
    return df

def _norm_cat(s): return pd.Series(s, copy=False).astype("string").str.strip().str.lower()

def _pick_col(cols, cand):
    for c in cand:
        if c in cols: return c
    return None

def _ensure_col(df: pd.DataFrame, out_col: str, cands: List[str]) -> pd.DataFrame:
    col = _pick_col(df.columns, cands)
    df = df.copy()
    if col is None: df[out_col] = "__na__"
    else:           df[out_col] = _norm_cat(df[col])
    return df

def _dedup_running_pairs(running_pairs: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
    if running_pairs is None or running_pairs.empty: 
        return None
    rp = running_pairs.copy()
    rp["ads_idx"] = rp["ads_idx"].astype(int)
    rp["mda_idx"] = rp["mda_idx"].astype(int)
    rp = rp.drop_duplicates(["ads_idx","mda_idx"])
    return rp

# -------------------------------
# 1) 윈도우
# -------------------------------
def split_obs_tgt_by_dates(
    ads_time: pd.DataFrame,
    obs_start: str = "2025-07-25",
    tgt_start: str = "2025-08-20",
    obs_end: Optional[str] = None,
    tgt_end: Optional[str] = None,
    tgt_max_days: int = 6
):
    t = ads_time.copy()
    t["date"] = _to_dt(t["rpt_time_date"])
    if t["date"].isna().all():
        raise ValueError("ads_time['rpt_time_date']에 유효한 날짜가 없습니다.")
    obs_start = pd.to_datetime(obs_start).normalize()
    tgt_start = pd.to_datetime(tgt_start).normalize()
    obs_end = (pd.to_datetime(obs_end).normalize() if obs_end is not None
               else (tgt_start - pd.Timedelta(days=1)).normalize())
    last_date = t["date"].max().normalize()
    tgt_end = (pd.to_datetime(tgt_end).normalize() if tgt_end is not None
               else min(last_date, (tgt_start + pd.Timedelta(days=tgt_max_days)).normalize()))
    obs = t[(t["date"] >= obs_start) & (t["date"] <= obs_end)].copy()
    tgt = t[(t["date"] >= tgt_start) & (t["date"] <= tgt_end)].copy()
    return obs, tgt, obs_start, tgt_end, (tgt_start - pd.Timedelta(days=1)).normalize()

# -------------------------------
# 2) 시간/매체/카테고리/도메인 컨텍스트 (OBS만)
# -------------------------------
def _build_time_signals(obs: pd.DataFrame) -> pd.DataFrame:
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["weekday"] = _to_dt(t["rpt_time_date"]).dt.weekday  # 0~6
    wd = t.groupby(["mda_idx","weekday"], as_index=False).agg(clk=("clk","sum"))
    wd["is_weekend"] = wd["weekday"].isin([5,6]).astype(int)
    g = wd.groupby("mda_idx", as_index=False).agg(clk_total=("clk","sum"))
    w = wd.merge(g, on="mda_idx", how="left")
    day_ratio = (w.assign(ratio=w["clk"]/w["clk_total"].replace(0,np.nan))
                   .sort_values(["mda_idx","ratio"], ascending=[True,False])
                   .groupby("mda_idx").head(1)[["mda_idx","ratio"]]
                   .rename(columns={"ratio":"mda_day_ratio"}))
    weekend_ratio = (w.groupby(["mda_idx","is_weekend"], as_index=False)
                       .agg(clk=("clk","sum")).merge(g,on="mda_idx",how="left"))
    weekend_ratio["mda_weekend_ratio"] = (weekend_ratio["clk"] /
                                          weekend_ratio["clk_total"].replace(0,np.nan)).fillna(0)
    weekend_ratio = weekend_ratio[weekend_ratio["is_weekend"]==1][["mda_idx","mda_weekend_ratio"]]
    out = day_ratio.merge(weekend_ratio, on="mda_idx", how="outer").fillna(0.0)
    out["mda_idx"] = out["mda_idx"].astype(int)
    return out

def build_attr_context(obs: pd.DataFrame, ads_list_with_domain: pd.DataFrame):
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["conv"] = _to_num(t.get("rpt_time_turn", 0))

    # mda 전역
    mda_all = (t.groupby("mda_idx", as_index=False)
                 .agg(mda_clk_global=("clk","sum"), mda_conv_global=("conv","sum")))
    mda_all["mda_cvr_global"] = (mda_all["mda_conv_global"] /
                                 mda_all["mda_clk_global"].replace(0,np.nan)).fillna(0.0)
    mu, sd = mda_all["mda_conv_global"].mean(), mda_all["mda_conv_global"].std(ddof=0)
    mda_all["mda_pop_norm"] = (mda_all["mda_conv_global"] - mu) / (sd + 1e-9)

    # 카테고리×매체
    al = ads_list_with_domain[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    al["ads_category"] = _norm_cat(al["ads_category"])
    t2 = t.merge(al, on="ads_idx", how="left")
    cat_mda = (t2.groupby(["mda_idx","ads_category"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    cat_mda["cat_cvr"] = (cat_mda["conv"]/cat_mda["clk"].replace(0,np.nan)).fillna(0.0)
    mda_tot = cat_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    cat_mda = cat_mda.merge(mda_tot, on="mda_idx", how="left")
    cat_mda["cat_share"] = (cat_mda["conv"]/cat_mda["mda_conv_total"].replace(0,np.nan)).fillna(0.0)

    # 도메인×매체
    t_dom = t.merge(
        ads_list_with_domain[["ads_idx","ads_domain"]].drop_duplicates("ads_idx"),
        on="ads_idx", how="left"
    )
    t_dom["ads_domain"] = _norm_cat(t_dom["ads_domain"])
    dom_mda = (t_dom.groupby(["mda_idx","ads_domain"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    dom_mda["dom_cvr"] = (dom_mda["conv"]/dom_mda["clk"].replace(0,np.nan)).fillna(0.0)
    mda_tot2 = dom_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    dom_mda = dom_mda.merge(mda_tot2, on="mda_idx", how="left")
    dom_mda["dom_share"] = (dom_mda["conv"]/dom_mda["mda_conv_total"].replace(0,np.nan)).fillna(0.0)

    # 시간 신호
    time_sig = _build_time_signals(obs)

    # (선택) 광고주 신호
    adv_ctx = None
    if "adv_idx" in ads_list_with_domain.columns:
        t_adv = t.merge(ads_list_with_domain[["ads_idx","adv_idx"]].drop_duplicates("ads_idx"),
                        on="ads_idx", how="left")
        adv_all = (t_adv.groupby("adv_idx", as_index=False)
                        .agg(adv_clk_global=("clk","sum"),
                             adv_conv_global=("conv","sum")))
        adv_all["adv_cvr_global"] = (adv_all["adv_conv_global"] /
                                     adv_all["adv_clk_global"].replace(0,np.nan)).fillna(0.0)
        mu2, sd2 = adv_all["adv_conv_global"].mean(), adv_all["adv_conv_global"].std(ddof=0)
        adv_all["adv_pop_norm"] = (adv_all["adv_conv_global"] - mu2) / (sd2 + 1e-9)
        adv_ctx = adv_all

    return {"mda_all": mda_all, "cat_mda": cat_mda, "dom_mda": dom_mda,
            "time_sig": time_sig, "adv_ctx": adv_ctx}

# -------------------------------
# 3) 학습 대상 광고
# -------------------------------
def pick_train_ads_from_tgt(tgt: pd.DataFrame, min_pos_in_tgt: int = 1,
                            max_train_ads: Optional[int] = 20000, rng_seed: int = 42) -> np.ndarray:
    t = tgt.copy()
    t["turn"] = _to_num(t.get("rpt_time_turn", 0))
    pos = (t.groupby("ads_idx")["turn"].apply(lambda s: (s > 0).sum())
             .reset_index(name="pos_cnt"))
    ok = pos[pos["pos_cnt"] >= max(1,int(min_pos_in_tgt))]["ads_idx"].astype(int).values
    if max_train_ads is not None and len(ok) > max_train_ads:
        rng = np.random.RandomState(rng_seed)
        ok = rng.choice(ok, size=max_train_ads, replace=False)
    return np.sort(ok)

# -------------------------------
# 4) 후보 생성
# -------------------------------
def create_candidates(ads_list: pd.DataFrame, mda_all: pd.DataFrame, cat_mda: pd.DataFrame,
                      topC=10, add_random=2, cat_topM=5,
                      min_active_clk=200, min_active_conv=20,
                      rng_seed=42, ad_ids_filter=None, max_per_ad=20):
    mda_active = mda_all[(mda_all["mda_clk_global"]>=min_active_clk) |
                         (mda_all["mda_conv_global"]>=min_active_conv)].copy()
    if mda_active.empty: mda_active = mda_all.copy()
    mda_ids_global = (mda_active.sort_values("mda_conv_global", ascending=False)["mda_idx"]
                      .astype(int).values)
    cat_top = (cat_mda.sort_values(["ads_category","cat_cvr"], ascending=[True,False])
                      .groupby("ads_category").head(cat_topM)[["ads_category","mda_idx"]])
    cat2top = {c: g["mda_idx"].astype(int).tolist() for c,g in cat_top.groupby("ads_category")}
    rng = np.random.RandomState(rng_seed)
    ad_ids = (np.array(ad_ids_filter, dtype=int) if (ad_ids_filter is not None and len(ad_ids_filter)>0)
              else ads_list["ads_idx"].dropna().astype(int).unique())
    ad_meta = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    ad_meta["ads_idx"] = ad_meta["ads_idx"].astype(int)
    ad_meta["ads_category"] = ad_meta["ads_category"].astype(str).str.strip().str.lower()
    ad2cat = ad_meta.set_index("ads_idx")["ads_category"].to_dict()

    pairs=[]
    for ad in ad_ids:
        base=[]
        cat = ad2cat.get(int(ad), None)
        if cat in cat2top: base.extend(cat2top[cat])
        base.extend(mda_ids_global[:min(topC,len(mda_ids_global))])
        base=list(dict.fromkeys(base))
        remain=np.setdiff1d(mda_ids_global, np.array(base, dtype=int))
        if len(remain)>0 and add_random>0:
            base.extend(rng.choice(remain, size=min(add_random,len(remain)), replace=False).tolist())
        if len(base)>max_per_ad: base=base[:max_per_ad]
        pairs += [(int(ad), int(m)) for m in base]
    out = pd.DataFrame(pairs, columns=["ads_idx","mda_idx"]).drop_duplicates()
    print(f"[Candidates] pairs={len(out):,} | ads={len(ad_ids):,} | active_mda={len(mda_active):,} | topC={topC}, rnd={add_random}, cat_topM={cat_topM}, per_ad≤{max_per_ad}")
    return out

# -------------------------------
# 5) 피처 & 라벨 (OBS 피처 / TGT 라벨)
# -------------------------------
FEAT_ALLOW = [
    # mda 전역
    "mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm",
    # 카테고리×매체
    "cat_cvr","cat_share",
    # 광고 가격
    "price_reward_log","price_contract_log","price_ratio",
    # 도메인×매체
    "dom_cvr","dom_share",
    # 시간 신호
    "mda_day_ratio","mda_weekend_ratio",
    # 광고주 신호
    "adv_cvr_global","adv_pop_norm",
    # 외부 포트폴리오가 있으면
    "mda_size","active_ratio","day_ratio","weekend_ratio"
]

def prepare_training_attr_only(
    obs: pd.DataFrame, tgt: pd.DataFrame, ads_list_with_domain: pd.DataFrame,
    topC=10, add_random=2, cat_topM=5, min_active_clk=200, min_active_conv=20,
    max_train_ads: Optional[int] = 20000, min_pos_in_tgt: int = 1,
    media_portfolio: Optional[pd.DataFrame] = None,
    excluded_mda_ids: Optional[set] = None,          # ★ 계약종료형 제외
    running_pairs: Optional[pd.DataFrame] = None     # ★ 현재 진행중 페어 제외
):
    running_pairs = _dedup_running_pairs(running_pairs)

    # ★ mda 제외(OBS/TGT에서 안전망)
    if excluded_mda_ids:
        obs = obs[~obs["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()
        tgt = tgt[~tgt["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    train_ads = pick_train_ads_from_tgt(tgt, min_pos_in_tgt=min_pos_in_tgt, max_train_ads=max_train_ads)
    actx = build_attr_context(obs, ads_list_with_domain)
    mda_all, cat_mda, dom_mda = actx["mda_all"], actx["cat_mda"], actx["dom_mda"]

    cand = create_candidates(
        ads_list=ads_list_with_domain, mda_all=mda_all, cat_mda=cat_mda,
        topC=topC, add_random=add_random, cat_topM=cat_topM,
        min_active_clk=min_active_clk, min_active_conv=min_active_conv,
        ad_ids_filter=train_ads, max_per_ad=20
    )

    # ★ 후보 그리드에서 현재 진행중 페어 제외
    if running_pairs is not None and not running_pairs.empty:
        tmp = running_pairs.assign(_ex=1)
        cand = cand.merge(tmp, on=["ads_idx","mda_idx"], how="left")
        cand = cand[cand["_ex"].isna()].drop(columns=["_ex"])

    # ★ 후보 그리드에서도 계약종료형 mda 제외
    if excluded_mda_ids:
        cand = cand[~cand["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    df = cand.merge(
        mda_all[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]],
        on="mda_idx", how="left"
    )

    ad_meta = ads_list_with_domain[["ads_idx","ads_category","ads_reward_price","ads_contract_price","ads_domain","adv_idx"]].drop_duplicates("ads_idx").copy()
    ad_meta = _norm_cat_col(ad_meta, "ads_category")
    ad_meta = _ensure_col(ad_meta, "ads_domain", ["ads_domain","domain"])
    for c in ["ads_reward_price","ads_contract_price"]:
        ad_meta[c] = _to_num(ad_meta[c])
    ad_meta["price_reward_log"]   = np.log1p(ad_meta["ads_reward_price"])
    ad_meta["price_contract_log"] = np.log1p(ad_meta["ads_contract_price"])
    ad_meta["price_ratio"]        = (ad_meta["ads_reward_price"] /
                                     ad_meta["ads_contract_price"].replace(0,np.nan)).fillna(0.0)

    df = df.merge(ad_meta[["ads_idx","ads_category","ads_domain","adv_idx",
                           "price_reward_log","price_contract_log","price_ratio"]],
                  on="ads_idx", how="left")

    df      = _norm_cat_col(df, "ads_category")
    cat_mda = _norm_cat_col(cat_mda, "ads_category")
    df = df.merge(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]],
                  on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0,"cat_share":0.0})

    dom_mda = dom_mda.copy()
    dom_mda["ads_domain"] = _norm_cat(dom_mda["ads_domain"])
    df["ads_domain"] = _norm_cat(df["ads_domain"])
    df = df.merge(dom_mda[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                  on=["mda_idx","ads_domain"], how="left").fillna({"dom_cvr":0.0,"dom_share":0.0})

    time_sig = actx["time_sig"]
    df = df.merge(time_sig, on="mda_idx", how="left").fillna({"mda_day_ratio":0.0,"mda_weekend_ratio":0.0})

    if actx["adv_ctx"] is not None:
        adv_ctx = actx["adv_ctx"].copy()
        df = df.merge(adv_ctx[["adv_idx","adv_cvr_global","adv_pop_norm"]], on="adv_idx", how="left")
    for c in ["adv_cvr_global","adv_pop_norm"]:
        if c not in df.columns: df[c]=0.0
    df[["adv_cvr_global","adv_pop_norm"]] = df[["adv_cvr_global","adv_pop_norm"]].fillna(0.0)

    if media_portfolio is not None:
        mp = media_portfolio.copy()
        cols = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp.columns]
        if "mda_idx" in cols: df = df.merge(mp[cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in df.columns: df[c]=0.0

    # ---- 라벨: TGT에서 클릭/전환
    tgt2 = tgt.copy()
    tgt2["clk"]  = _to_num(tgt2.get("rpt_time_clk", 0))
    tgt2["turn"] = _to_num(tgt2.get("rpt_time_turn",0))
    y_agg = (tgt2.groupby(["ads_idx","mda_idx"], as_index=False)
                   .agg(tgt_clk=("clk","sum"), tgt_turn=("turn","sum")))
    y_agg["tgt_cvr"] = (y_agg["tgt_turn"] / y_agg["tgt_clk"].replace(0,np.nan)).clip(0,1)

    df = df.merge(y_agg, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_clk":0.0,"tgt_turn":0.0})

    # 랭커용 relevance
    y = df["tgt_turn"].astype(float)
    rel = np.zeros(len(df), dtype=int)
    pos = y > 0
    if pos.sum()>0:
        ylog = np.log1p(y[pos])
        try:
            bins = pd.qcut(ylog, q=3, labels=False, duplicates="drop")
            rel[pos] = bins.astype(int)+1
        except ValueError:
            rel[pos] = 1
    df["relevance"] = rel
    return df, actx

# -------------------------------
# 6) 학습기들 (랭커 + 클릭회귀 + CVR회귀)
# -------------------------------
LABEL_GAIN = [0.0,1.0,4.0,12.0]

def ensure_feature_matrix(df, feat_cols):
    X = df.copy()
    for c in feat_cols:
        if c not in X.columns: X[c]=0.0
        if not np.issubdtype(X[c].dtype, np.number):
            X[c]=pd.to_numeric(X[c], errors="coerce")
    return X[feat_cols].fillna(0.0).astype(np.float32)

def downsample_negatives_per_ad(df, ratio=4, random_state=42):
    out, rng = [], np.random.RandomState(random_state)
    for ad, g in df.groupby("ads_idx", sort=False):
        pos = g[g["tgt_turn"]>0]; neg = g[g["tgt_turn"]<=0]
        if len(pos)==0: continue
        cap = len(pos)*ratio
        if len(neg)>cap: neg = neg.sample(n=cap, random_state=rng)
        out.append(pd.concat([pos,neg], ignore_index=True))
    return pd.concat(out, ignore_index=True) if out else df

def train_attr_ranker(train_df: pd.DataFrame):
    feat_cols = [c for c in FEAT_ALLOW if c in train_df.columns]
    X = ensure_feature_matrix(train_df, feat_cols)
    y = train_df["relevance"].astype(int)
    groups = train_df.groupby("ads_idx").size().astype(int).tolist()
    ranker = lgb.LGBMRanker(
        objective="lambdarank", metric="ndcg", eval_at=[5,10,20],
        learning_rate=0.05, n_estimators=1200, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        label_gain=LABEL_GAIN, random_state=42, n_jobs=-1, verbosity=-1
    )
    ranker.fit(X, y, group=groups, sample_weight=np.ones(len(train_df)))
    return ranker, feat_cols

def train_click_regressor(train_df: pd.DataFrame, feat_cols: List[str], tweedie_p: float = 1.1):
    X = ensure_feature_matrix(train_df, feat_cols)
    y = train_df["tgt_clk"].astype(float)
    reg = lgb.LGBMRegressor(
        objective="tweedie", tweedie_variance_power=tweedie_p,
        n_estimators=1200, learning_rate=0.05, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        random_state=42, n_jobs=-1, verbosity=-1
    )
    reg.fit(X, y)
    return reg, {"mode":"tweedie_click","twp":tweedie_p}

def _logit_clip(p, eps=1e-6):
    p = np.clip(p, eps, 1-eps)
    return np.log(p/(1-p))

def _sigmoid(x): return 1/(1+np.exp(-x))

def train_cvr_regressor(train_df: pd.DataFrame, feat_cols: List[str]):
    g = train_df[train_df["tgt_clk"]>0].copy()
    if g.empty:
        dummy = lgb.LGBMRegressor().fit(np.zeros((1, len(feat_cols))), [0.0])
        return dummy, {"mode":"cvr_logit","fallback":True, "mean_logit":-20.0}
    X = ensure_feature_matrix(g, feat_cols)
    y = g["tgt_turn"]/g["tgt_clk"]
    y = y.clip(0,1)
    y_logit = _logit_clip(y.values)
    reg = lgb.LGBMRegressor(
        objective="regression",
        n_estimators=1200, learning_rate=0.05, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        random_state=42, n_jobs=-1, verbosity=-1
    )
    reg.fit(X, y_logit)
    return reg, {"mode":"cvr_logit","fallback":False}

# -------------------------------
# 7) 예측 (랭커 스코어 + 클릭×CVR 예측)
# -------------------------------
def predict_all_mda_with_counts_batched(
    ranker, click_reg, cvr_reg, feat_cols, actx, ads_list_with_domain,
    K=20, batch_size=5000, max_predict_ads: Optional[int] = 20000,
    media_portfolio: Optional[pd.DataFrame] = None,
    excluded_mda_ids: Optional[set] = None,          # ★ 계약종료형 제외
    running_pairs: Optional[pd.DataFrame] = None     # ★ 현재 진행중 페어 제외
) -> Dict[int, pd.DataFrame]:
    running_pairs = _dedup_running_pairs(running_pairs)

    mda_all, cat_mda, dom_mda = actx["mda_all"], actx["cat_mda"], actx["dom_mda"]
    time_sig, adv_ctx = actx["time_sig"], actx["adv_ctx"]

    mda_active = mda_all[(mda_all["mda_clk_global"]>=200) | (mda_all["mda_conv_global"]>=20)]
    if mda_active.empty: mda_active = mda_all
    if excluded_mda_ids:
        mda_active = mda_active[~mda_active["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    mda_active = mda_active[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]].copy()
    mda_active["mda_idx"]=mda_active["mda_idx"].astype(int)
    mda_active = mda_active.merge(time_sig, on="mda_idx", how="left").fillna({"mda_day_ratio":0.0,"mda_weekend_ratio":0.0})

    if media_portfolio is not None:
        mp = media_portfolio.copy()
        cols = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp.columns]
        if "mda_idx" in cols: mda_active = mda_active.merge(mp[cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in mda_active.columns: mda_active[c]=0.0

    ads_meta = ads_list_with_domain[["ads_idx","ads_category","ads_reward_price","ads_contract_price","ads_domain","adv_idx"]].drop_duplicates("ads_idx").copy()
    if max_predict_ads is not None: ads_meta = ads_meta.head(int(max_predict_ads))
    ads_meta = _norm_cat_col(ads_meta, "ads_category")
    ads_meta = _ensure_col(ads_meta, "ads_domain", ["ads_domain","domain"])
    ads_meta["ads_idx"] = ads_meta["ads_idx"].astype(int)
    ads_meta["ads_domain"] = _norm_cat(ads_meta["ads_domain"])
    ads_meta["ads_reward_price"]   = _to_num(ads_meta["ads_reward_price"])
    ads_meta["ads_contract_price"] = _to_num(ads_meta["ads_contract_price"])
    ads_meta["price_reward_log"]   = np.log1p(ads_meta["ads_reward_price"])
    ads_meta["price_contract_log"] = np.log1p(ads_meta["ads_contract_price"])
    ads_meta["price_ratio"]        = (ads_meta["ads_reward_price"] / ads_meta["ads_contract_price"].replace(0,np.nan)).fillna(0.0)
    if adv_ctx is not None:
        ads_meta = ads_meta.merge(adv_ctx[["adv_idx","adv_cvr_global","adv_pop_norm"]], on="adv_idx", how="left")
    for c in ["adv_cvr_global","adv_pop_norm"]:
        if c not in ads_meta.columns: ads_meta[c]=0.0
    ads_meta[["adv_cvr_global","adv_pop_norm"]] = ads_meta[["adv_cvr_global","adv_pop_norm"]].fillna(0.0)

    cat_mda_local = _norm_cat_col(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]].copy(), "ads_category")
    dom_mda_local = dom_mda.copy(); dom_mda_local["ads_domain"] = _norm_cat(dom_mda_local["ads_domain"])

    out={}
    ad_ids = ads_meta["ads_idx"].tolist()
    for i in range(0, len(ad_ids), batch_size):
        ad_batch = ads_meta.iloc[i:i+batch_size]
        grid = ad_batch[["ads_idx","ads_category","ads_domain","price_reward_log","price_contract_log","price_ratio",
                         "adv_idx","adv_cvr_global","adv_pop_norm"]] \
               .merge(mda_active, how="cross")

        # ★ 현재 진행중 페어 제외 (안티조인)
        if running_pairs is not None and not running_pairs.empty:
            tmp = running_pairs.assign(_ex=1)
            grid = grid.merge(tmp, on=["ads_idx","mda_idx"], how="left")
            grid = grid[grid["_ex"].isna()].drop(columns=["_ex"])

        grid = grid.merge(cat_mda_local, on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0,"cat_share":0.0})
        grid = grid.merge(dom_mda_local[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                          on=["mda_idx","ads_domain"], how="left").fillna({"dom_cvr":0.0,"dom_share":0.0})

        X = ensure_feature_matrix(grid, feat_cols)
        score = ranker.predict(X)  # 랭커 점수

        # Decomposition: click × cvr
        yhat_clk = np.maximum(0.0, click_reg.predict(X))
        yhat_cvr_logit = cvr_reg.predict(X)
        yhat_cvr = np.clip(_sigmoid(yhat_cvr_logit), 0.0, 1.0)
        yhat_turn = yhat_clk * yhat_cvr

        grid2 = grid[["ads_idx","mda_idx","ads_category","mda_cvr_global","mda_pop_norm","mda_clk_global","mda_conv_global"]].copy()
        grid2["score"] = score.astype(float)
        grid2["yhat_turn"] = yhat_turn.astype(float)

        for ad, g in grid2.groupby("ads_idx", sort=False):
            top = g.nlargest(K, "score")
            out[int(ad)] = top.sort_values("score", ascending=False).reset_index(drop=True)
        print(f"[Predict+Count] processed ads {i+len(ad_batch)}/{len(ad_ids)}")
    return out

# -------------------------------
# 8) 평가
# -------------------------------
def _precision_recall_hit_at_k(preds_dict, gt_dict, k=10):
    P,R,H=[],[],[]
    for ad, df in preds_dict.items():
        if ad not in gt_dict: continue
        pred = df["mda_idx"].astype(int).tolist()[:k]
        true = set(gt_dict[ad]); 
        if not true: continue
        inter = len(set(pred)&true)
        P.append(inter/max(1,len(pred))); R.append(inter/len(true)); H.append(1.0 if inter>0 else 0.0)
    return (float(np.mean(P)) if P else 0.0,
            float(np.mean(R)) if R else 0.0,
            float(np.mean(H)) if H else 0.0)

def _map_at_k(preds_dict, gt_dict, k=10):
    APs=[]
    for ad, df in preds_dict.items():
        if ad not in gt_dict: continue
        true=set(gt_dict[ad]); 
        if not true: continue
        ranked = df["mda_idx"].astype(int).tolist()[:k]
        hits=cum_prec=0
        for i,m in enumerate(ranked, start=1):
            if m in true:
                hits+=1; cum_prec += hits/i
        APs.append(cum_prec/min(len(true),k) if hits>0 else 0.0)
    return float(np.mean(APs)) if APs else 0.0

def _auc_flat(preds_dict, gt_pairs_dict, score_col="score"):
    y_true, y_score = [], []
    for ad, df in preds_dict.items():
        true_mdas = set(gt_pairs_dict.get(ad, []))
        for _, r in df.iterrows():
            y_true.append(1 if int(r["mda_idx"]) in true_mdas else 0)
            y_score.append(float(r[score_col]))
    if len(set(y_true))<2: return 0.5
    try: return float(roc_auc_score(y_true, y_score))
    except Exception: return 0.5

def evaluate_all(preds_with_counts: Dict[int, pd.DataFrame], tgt: pd.DataFrame, k=10):
    gt_pairs = tgt.groupby("ads_idx")["mda_idx"].apply(lambda s: list(pd.Series(s).dropna().astype(int).unique())).to_dict()
    prec, rec, hit = _precision_recall_hit_at_k(preds_with_counts, gt_pairs, k=k)
    mapk = _map_at_k(preds_with_counts, gt_pairs, k=k)
    auc_rank = _auc_flat(preds_with_counts, gt_pairs, score_col="score")
    auc_reg  = _auc_flat(preds_with_counts, gt_pairs, score_col="yhat_turn")
    return {f"precision@{k}":prec, f"recall@{k}":rec, f"map@{k}":mapk,
            f"hit_rate@{k}":hit, "auc_ranker":auc_rank, "auc_regressor":auc_reg}

# -------------------------------
# 9) 편의
# -------------------------------
def get_expected_conversions_for_ad(preds_with_counts: Dict[int, pd.DataFrame],
                                    ad_id: int, sort_by="yhat_turn", top=20) -> pd.DataFrame:
    df = preds_with_counts.get(int(ad_id))
    if df is None or df.empty:
        return pd.DataFrame(columns=["mda_idx","score","yhat_turn"])
    sort_by = "yhat_turn" if sort_by not in ("yhat_turn","score") else sort_by
    out = df.sort_values(sort_by, ascending=False).copy()
    return out.head(top) if top is not None else out

# =========================================================
# ==============   메인 플로우 (예시)   ====================
# =========================================================
# 요구 입력 예시:
#   ads_time, ads_list, ads_segment(ads_size, domain),
#   media_performance_classification(옵션), running_pairs(옵션),
#   media_portfolio(옵션)

# (A) 윈도우
obs, tgt, start, end, cut = split_obs_tgt_by_dates(
    ads_time, obs_start="2025-07-25", tgt_start="2025-08-20"
)
print(f"[Window] OBS: {start.date()} → {cut.date()}  |  TGT: {(cut+pd.Timedelta(days=1)).date()} → {end.date()}")

# (A-1) 계약종료형 매체 목록
if 'media_performance_classification' in globals():
    banned_mda = set(
        media_performance_classification
          .loc[media_performance_classification["basic_classification"].astype(str).str.strip() == "계약종료형", "mda_idx"]
          .dropna().astype(int).unique()
    )
else:
    banned_mda = set()
print(f"[Exclude] 계약종료형 mda 개수 = {len(banned_mda)}")

# (A-2) OBS/TGT에서 계약종료형 제거
if banned_mda:
    obs = obs[~obs["mda_idx"].astype(int).isin(banned_mda)].copy()
    tgt = tgt[~tgt["mda_idx"].astype(int).isin(banned_mda)].copy()

# (A-3) 현재 진행중 페어 준비(옵션)
if 'running_pairs' in globals():
    running_pairs = _dedup_running_pairs(running_pairs)
    print(f"[Exclude] running_pairs (현재 진행중 페어) = {0 if running_pairs is None else len(running_pairs)}")
else:
    running_pairs = None

# (B) ads_list 확장 (ads_size, domain 병합)
ads_list_ext = ads_list.merge(
    ads_segment[["ads_idx","ads_size","domain"]].rename(columns={"domain":"ads_domain"}),
    on="ads_idx", how="left"
)

# (C) 사이즈 필터(예: MEGA/LARGE/MEDIUM만)
size_whitelist = {"MEGA","LARGE","MEDIUM"}
ads_list_sel = ads_list_ext[
    ads_list_ext["ads_size"].astype(str).str.strip().str.upper().isin(size_whitelist)
].copy()
sel_ids = set(ads_list_sel["ads_idx"].astype(int))
tgt_sel  = tgt[tgt["ads_idx"].astype(int).isin(sel_ids)].copy()
print(f"[Filter] sizes kept = {sorted(size_whitelist)} | ads={ads_list_sel['ads_idx'].nunique():,} | tgt pairs={len(tgt_sel):,}")

# (D) 학습 데이터 (계약종료형 + 현재진행 제외 세이프가드 포함)
train_df, actx = prepare_training_attr_only(
    obs, tgt_sel, ads_list_sel,
    topC=10, add_random=2, cat_topM=5,
    min_active_clk=200, min_active_conv=20,
    max_train_ads=20000, min_pos_in_tgt=1,
    media_portfolio=None,
    excluded_mda_ids=banned_mda,      # ★ 유지
    running_pairs=running_pairs       # ★ 추가
)
print("train_df size:", len(train_df))

# (E) 학습 (랭커 + 클릭 + CVR)
train_df_ds = downsample_negatives_per_ad(train_df, ratio=4)
ranker, feat_cols   = train_attr_ranker(train_df_ds)
click_reg, click_md = train_click_regressor(train_df_ds, feat_cols, tweedie_p=1.1)
cvr_reg, cvr_md     = train_cvr_regressor(train_df_ds, feat_cols)
print("feat_cols used:", feat_cols)

# (F) 예측 & 평가 (예측 교차조인에서도 계약종료형 + 현재진행 제외)
preds_with_counts = predict_all_mda_with_counts_batched(
    ranker, click_reg, cvr_reg, feat_cols, actx, ads_list_sel,
    K=20, batch_size=5000, max_predict_ads=20000, media_portfolio=None,
    excluded_mda_ids=banned_mda,      # ★ 유지
    running_pairs=running_pairs       # ★ 추가
)
metrics_k5  = evaluate_all(preds_with_counts, tgt_sel, k=5)
metrics_k10 = evaluate_all(preds_with_counts, tgt_sel, k=10)
print("Eval@5:", metrics_k5)
print("Eval@10:", metrics_k10)

# (G) Sanity Check
leftover_tgt = tgt_sel["mda_idx"].astype(int).isin(banned_mda).sum()
print(f"[Sanity] TGT(selected sizes) 내 계약종료형 잔여: {leftover_tgt}")
any_banned_in_preds = any(
    int(m) in banned_mda
    for df in preds_with_counts.values()
    for m in df["mda_idx"].astype(int).tolist()
)
print(f"[Sanity] 예측결과에 계약종료형 포함 여부: {any_banned_in_preds}")

# (H) 특정 광고 Top-K (예상 전환수 기준)
# ad_id = 446406
# top5 = get_expected_conversions_for_ad(preds_with_counts, ad_id, sort_by="yhat_turn", top=5)
# print(top5[["mda_idx","yhat_turn","score"]])



[Window] OBS: 2025-07-25 → 2025-08-19  |  TGT: 2025-08-20 → 2025-08-25
[Exclude] 계약종료형 mda 개수 = 51
[Exclude] running_pairs (현재 진행중 페어) = 21026
[Filter] sizes kept = ['LARGE', 'MEDIUM', 'MEGA'] | ads=7,169 | tgt pairs=63,274
[Candidates] pairs=26,263 | ads=1,565 | active_mda=128 | topC=10, rnd=2, cat_topM=5, per_ad≤20
train_df size: 23932


[LightGBM] [Fatal] [tweedie]: sum of labels is zero


LightGBMError: [tweedie]: sum of labels is zero

In [ ]:
# =======================
# LGBM 회귀 지표 유틸
# =======================

def _safe_rmse(y_true, y_pred):
    try:
        return mean_squared_error(y_true, y_pred, squared=False)
    except TypeError:
        return np.sqrt(mean_squared_error(y_true, y_pred))

def _safe_mape(y_true, y_pred, eps=1e-8, only_positive=True):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    if only_positive:
        mask = y_true > 0
        if mask.sum() == 0:
            return np.nan
        y_true = y_true[mask]
        y_pred = y_pred[mask]
    denom = np.where(np.abs(y_true) < eps, eps, np.abs(y_true))
    return float(np.mean(np.abs((y_true - y_pred) / denom)) * 100.0)

def _collect_true_vs_pred(preds_with_counts: dict,
                          tgt_df: pd.DataFrame,
                          topK: int | None = None) -> pd.DataFrame:
    rows=[]
    for ad, df in preds_with_counts.items():
        g = df if topK is None else df.head(int(topK))
        g2 = g[["mda_idx","yhat_turn"]].copy()
        g2["ads_idx"] = int(ad)
        rows.append(g2)
    pred_all = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame(columns=["ads_idx","mda_idx","yhat_turn"])
    pred_all["ads_idx"] = pred_all["ads_idx"].astype(int)
    pred_all["mda_idx"] = pred_all["mda_idx"].astype(int)

    t = tgt_df.copy()
    t["tgt_turn"] = pd.to_numeric(t.get("rpt_time_turn", 0), errors="coerce").fillna(0.0)
    true_agg = (t.groupby(["ads_idx","mda_idx"], as_index=False)
                  .agg(tgt_turn=("tgt_turn","sum")))

    df = pred_all.merge(true_agg, on=["ads_idx","mda_idx"], how="outer") \
                 .fillna({"yhat_turn":0.0, "tgt_turn":0.0})
    return df[["ads_idx","mda_idx","tgt_turn","yhat_turn"]]

def regression_metrics_from_preds(preds_with_counts: dict,
                                  tgt_df: pd.DataFrame,
                                  topK: int | None = None) -> dict:
    df = _collect_true_vs_pred(preds_with_counts, tgt_df, topK=topK)
    if df.empty:
        return {"MAE": np.nan, "RMSE": np.nan, "R2": np.nan, "MAPE(%)": np.nan, "n_pairs": 0}
    y_true = df["tgt_turn"].values
    y_pred = df["yhat_turn"].values
    return {
        "MAE": float(mean_absolute_error(y_true, y_pred)),
        "RMSE": float(_safe_rmse(y_true, y_pred)),
        "R2": float(r2_score(y_true, y_pred)),
        "MAPE(%)": float(_safe_mape(y_true, y_pred, eps=1e-8, only_positive=True)),
        "n_pairs": int(len(df)),
    }

def print_regression_report(preds_with_counts: dict,
                            tgt_df: pd.DataFrame,
                            ks=(None, 5, 10)):
    for k in ks:
        tag = "ALL" if k is None else f"Top-{k}"
        m = regression_metrics_from_preds(preds_with_counts, tgt_df, topK=k)
        print(f"[{tag}]  MAE: {m['MAE']:.2f} | RMSE: {m['RMSE']:.2f} | R²: {m['R2']:.3f} | "
              f"MAPE: {m['MAPE(%)']:.2f}% | n={m['n_pairs']:,}")


In [ ]:
# (F)에서 preds_with_counts 생성 후 바로 실행
print_regression_report(preds_with_counts, tgt_sel, ks=(None, 5, 10))

# 필요하면 단일 값으로도
m_all  = regression_metrics_from_preds(preds_with_counts, tgt_sel, topK=None)
m_top5 = regression_metrics_from_preds(preds_with_counts, tgt_sel, topK=5)
m_top10= regression_metrics_from_preds(preds_with_counts, tgt_sel, topK=10)
print(m_all)


[ALL]  MAE: 4.52 | RMSE: 180.05 | R²: 0.233 | MAPE: 252.39% | n=144,636
[Top-5]  MAE: 16.09 | RMSE: 365.12 | R²: 0.166 | MAPE: 252.90% | n=38,231
[Top-10]  MAE: 8.40 | RMSE: 255.05 | R²: 0.217 | MAPE: 252.55% | n=73,519
{'MAE': 4.517249567373953, 'RMSE': 180.0526411666147, 'R2': 0.23280486632038466, 'MAPE(%)': 252.38606085625034, 'n_pairs': 144636}


In [ ]:

def _build_eval_frame(preds_with_counts: dict, tgt: pd.DataFrame) -> pd.DataFrame:
    """
    preds_with_counts: predict_all_mda_with_counts_batched 결과(dict: ad_id -> df)
    tgt: TGT 윈도우 원본 로그(DataFrame; rpt_time_turn 포함)
    반환: ads_idx, mda_idx, yhat_turn, score, tgt_turn 컬럼 갖는 평가용 프레임
    """
    # 1) 예측 합치기
    pred_list=[]
    for ad, df in preds_with_counts.items():
        if df is None or df.empty:
            continue
        tmp = df[["mda_idx","yhat_turn","score"]].copy()
        tmp["ads_idx"] = int(ad)
        pred_list.append(tmp)
    pred_df = (pd.concat(pred_list, ignore_index=True)
               if pred_list else pd.DataFrame(columns=["ads_idx","mda_idx","yhat_turn","score"]))
    pred_df["ads_idx"] = pred_df["ads_idx"].astype(int)
    pred_df["mda_idx"] = pred_df["mda_idx"].astype(int)

    # 2) 실제 전환 집계(TGT)
    t = tgt.copy()
    t["turn"] = pd.to_numeric(t.get("rpt_time_turn", 0), errors="coerce").fillna(0)
    gt = (t.groupby(["ads_idx","mda_idx"], as_index=False)
            .agg(tgt_turn=("turn","sum")))
    gt["ads_idx"] = gt["ads_idx"].astype(int)
    gt["mda_idx"] = gt["mda_idx"].astype(int)

    # 3) 예측 기준 left join (예측 없는 조합은 평가 제외)
    eval_df = pred_df.merge(gt, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_turn":0.0})
    return eval_df

def regression_metrics_overall(eval_df: pd.DataFrame) -> dict:
    y_true = eval_df["tgt_turn"].values.astype(float)
    y_pred = eval_df["yhat_turn"].values.astype(float)

    mae  = float(mean_absolute_error(y_true, y_pred))
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))  # 버전 독립적 RMSE
    r2   = float(r2_score(y_true, y_pred))

    # MAPE (0 나누기 방지: 0인 구간 제외하고 평균)
    eps = 1e-8
    denom = np.where(np.abs(y_true) < eps, np.nan, np.abs(y_true))
    mape = float(np.nanmean(np.abs((y_true - y_pred) / denom)) * 100.0)

    return {"MAE": mae, "RMSE": rmse, "R2": r2, "MAPE(%)": mape}

def print_metrics(m: dict, title: str = "Overall regression metrics"):
    print(f"\n=== {title} ===")
    print(f"MAE : {m['MAE']:.2f}")
    print(f"RMSE: {m['RMSE']:.2f}")
    print(f"R²  : {m['R2']:.3f}")
    print(f"MAPE: {m['MAPE(%)']:.2f}%")


In [ ]:
# 회귀 지표
eval_df = _build_eval_frame(preds_with_counts, tgt_sel)  # 필요 시 tgt로 교체 가능
metrics = regression_metrics_overall(eval_df)
print_metrics(metrics, title="LightGBM (banned 제외 + running_pairs 제외)")



=== LightGBM (banned 제외 + running_pairs 제외) ===
MAE : 3.93
RMSE: 176.32
R²  : 0.242
MAPE: 295.65%


In [ ]:
def get_expected_vs_actual_for_ad(
    preds_with_counts: Dict[int, pd.DataFrame],
    tgt_df: pd.DataFrame,
    ad_id: int,
    sort_by: str = "yhat_turn",
    top: int = 10
) -> pd.DataFrame:
    """
    특정 광고(ad_id)에 대해 예측된 매체 상위 K를 뽑고,
    TGT에서의 실제 전환수(tgt_turn)를 합쳐 비교 테이블을 반환합니다.
    """
    # 1) 예측 가져오기
    pred_df = preds_with_counts.get(int(ad_id))
    if pred_df is None or pred_df.empty:
        return pd.DataFrame(columns=["mda_idx", "yhat_turn", "tgt_turn", "score"])

    # 2) 실제 전환 집계(TGT)
    t = tgt_df.copy()
    t["ads_idx"] = pd.to_numeric(t["ads_idx"], errors="coerce").astype("Int64")
    t["mda_idx"] = pd.to_numeric(t["mda_idx"], errors="coerce").astype("Int64")
    t["rpt_time_turn"] = pd.to_numeric(t.get("rpt_time_turn", 0), errors="coerce").fillna(0)

    actual = (
        t[t["ads_idx"] == int(ad_id)]
        .groupby("mda_idx", as_index=False)
        .agg(tgt_turn=("rpt_time_turn", "sum"))
    )
    actual["mda_idx"] = actual["mda_idx"].astype(int)

    # 3) 예측과 실제 머지
    view = pred_df.merge(actual, on="mda_idx", how="left")
    view["tgt_turn"] = view["tgt_turn"].fillna(0.0)

    # 4) 정렬 & 상위 N
    sort_by = sort_by if sort_by in ("yhat_turn", "score") else "yhat_turn"
    view = view.sort_values(sort_by, ascending=False).reset_index(drop=True)
    if top is not None:
        view = view.head(int(top))

    # 5) 필요한 컬럼만 깔끔하게
    out = view[["mda_idx", "yhat_turn", "tgt_turn", "score"]].copy()
    return out


In [ ]:
ad_id = 34045
top5_cmp = get_expected_vs_actual_for_ad(
    preds_with_counts, tgt_large,  # <- 여기서 tgt_large 대신 tgt_sel 등 사용 가능
    ad_id=ad_id, sort_by="yhat_turn", top=10
)

print(f"=== Ad {ad_id} 추천 매체사 Top-10 (예상 전환수 vs 실제 전환수) ===")
print(top5_cmp[["mda_idx","yhat_turn","tgt_turn","score"]])

NameError: name 'tgt_large' is not defined

In [ ]:

df = top5_cmp.copy()

y_true = df["tgt_turn"].values
y_pred = df["yhat_turn"].values

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))  # 버전 호환 안전
r2 = r2_score(y_true, y_pred)

# MAPE (0 나누기 방지)
mask = y_true > 0
mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

print(f"MAE : {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²  : {r2:.3f}")
print(f"MAPE: {mape:.2f}%")


MAE : 2.86
RMSE: 7.62
R²  : 0.983
MAPE: 9.22%


In [ ]:
ad_id = 34045
top5 = get_expected_conversions_for_ad(preds_with_counts, ad_id, sort_by="yhat_turn", top=5)
print(top5[["mda_idx","yhat_turn","score"]])

   mda_idx  yhat_turn     score
1      756  82.984114  2.847827
0      757  79.846323  3.591785
2      678  60.676631  2.692724
8      796  28.824818  1.493661
5     1025   9.715345  1.659074


In [ ]:
import joblib

bundle = { 'preds_with_counts' : preds_with_counts,'tgt_large' : tgt_large }

# 저장
joblib.dump(bundle, "model.pkl")


['model.pkl']

In [ ]:
# -------------------------------
# 9-추가) 신규 매체사만 필터링 헬퍼
# -------------------------------
def _build_excluded_pairs_for_ad(
    ad_id: int,
    ads_time: pd.DataFrame,
    running_pairs: Optional[pd.DataFrame] = None,
    date_from: Optional[str] = None,
    date_to: Optional[str] = None,
    exclude_click_ge1: bool = True,
    exclude_any_history: bool = True
) -> Set[int]:
    """
    특정 광고(ad_id)에 대해 '제외해야 할 mda_idx' 집합을 만든다.
      - 과거 로그에 등장한 모든 페어(ads_idx=ad_id)  [exclude_any_history]
      - 과거 클릭수가 1 이상인 페어                   [exclude_click_ge1]
      - 현재 진행중 페어(running_pairs)
    """
    t = ads_time.copy()
    # 날짜 필터
    if date_from or date_to:
        t["date"] = pd.to_datetime(t["rpt_time_date"], errors="coerce")
        if date_from: t = t[t["date"] >= pd.to_datetime(date_from)]
        if date_to:   t = t[t["date"] <= pd.to_datetime(date_to)]
    # ad 고정
    t = t[pd.to_numeric(t["ads_idx"], errors="coerce").fillna(-1).astype(int) == int(ad_id)].copy()
    t["mda_idx"] = pd.to_numeric(t["mda_idx"], errors="coerce").fillna(-1).astype(int)

    excluded = set()
    if exclude_any_history and not t.empty:
        excluded.update(t["mda_idx"].dropna().astype(int).tolist())
    if exclude_click_ge1 and not t.empty:
        clk = pd.to_numeric(t.get("rpt_time_clk", 0), errors="coerce").fillna(0)
        excluded.update(t.loc[clk >= 1, "mda_idx"].dropna().astype(int).tolist())

    if running_pairs is not None and not running_pairs.empty:
        rp = running_pairs.copy()
        rp["ads_idx"] = pd.to_numeric(rp["ads_idx"], errors="coerce").fillna(-1).astype(int)
        rp["mda_idx"] = pd.to_numeric(rp["mda_idx"], errors="coerce").fillna(-1).astype(int)
        excluded.update(rp.loc[rp["ads_idx"] == int(ad_id), "mda_idx"].tolist())

    return {int(m) for m in excluded if m != -1}

def get_new_media_for_ad(
    preds_with_counts: Dict[int, pd.DataFrame],
    ad_id: int,
    ads_time: pd.DataFrame,
    running_pairs: Optional[pd.DataFrame] = None,
    date_from: Optional[str] = None,
    date_to: Optional[str] = None,
    sort_by: str = "yhat_turn",
    top: int = 20,
    media_meta: Optional[pd.DataFrame] = None  # 예: media_portfolio[["mda_idx","mda_name"]]
) -> pd.DataFrame:
    """
    예측 결과에서 해당 광고의 '신규 매체사'만 반환.
      - 신규 정의: 이력 無, 클릭≥1 無, 현재 진행중 아님
    """
    # 1) 예측 상위 rows
    cand = get_expected_conversions_for_ad(preds_with_counts, ad_id, sort_by=sort_by, top=None)
    if cand.empty:
        return cand

    # 2) 제외할 mda 집합 구성
    excluded_mdas = _build_excluded_pairs_for_ad(
        ad_id=ad_id,
        ads_time=ads_time,
        running_pairs=running_pairs,
        date_from=date_from,
        date_to=date_to,
        exclude_click_ge1=True,
        exclude_any_history=True
    )

    # 3) 신규만 필터
    out = cand[~cand["mda_idx"].astype(int).isin(excluded_mdas)].copy()

    # 4) 이름 붙이기(옵션)
    if media_meta is not None:
        cols = [c for c in ["mda_idx","mda_name"] if c in media_meta.columns]
        if "mda_idx" in cols and len(cols) >= 1:
            out = out.merge(media_meta[cols].drop_duplicates("mda_idx"), on="mda_idx", how="left")

    # 5) top N
    if top is not None:
        out = out.head(int(top))
    return out


In [ ]:
# running_pairs 만들기 예시
running_pairs = ads_time.groupby(["ads_idx", "mda_idx"]).size().reset_index(name="cnt")

print(running_pairs.head())


   ads_idx  mda_idx  cnt
0     6508      854    1
1     6985       22   28
2     6985      401    9
3     8327      761    9
4     9264      772    1


In [ ]:
ad_id = 349591

In [ ]:
# (I) 특정 광고 Top-K - '신규 매체사'만
ad_id = 349591
# 필요하면 과거 이력 기준 기간을 제한할 수도 있음 (예: 2024-01-01 ~ 2025-08-19)
# date_from, date_to = "2024-01-01", "2025-08-19"
date_from, date_to = None, None

top5_new = get_new_media_for_ad(
    preds_with_counts=preds_with_counts,
    ad_id=ad_id,
    ads_time=ads_time,                 # 전체 로그
    running_pairs=running_pairs,       # 현재 진행중 페어
    date_from=date_from,
    date_to=date_to,
    sort_by="yhat_turn",
    top=5,
    # 매체명 붙이고 싶으면 아래 전달 (없으면 None)
    media_meta=(media_portfolio[["mda_idx","mda_name"]] if "media_portfolio" in globals() and "mda_name" in media_portfolio.columns else None)
)
print(top5_new[["mda_idx","yhat_turn","score", *(["mda_name"] if "mda_name" in top5_new.columns else [])]])


    mda_idx  yhat_turn     score
13       58   8.017398  0.044323
3        14   3.079165  2.098122
5       371   2.626794  1.608369
18      667   1.318032 -1.444518
10      398   0.885277  0.412876


In [ ]:
# =========================================================
#  Ad × Media 추천 + 전환수 예측 (Decomposition: Click × CVR, OS 제외)
#  - 누수 방지: 모든 피처는 OBS에서만 생성, 라벨은 TGT
#  - 클릭: Tweedie 회귀(권장), CVR: logit 회귀
#  - 예측: yhat_turn = yhat_click × yhat_cvr
#  - ★ 진행중 페어는 포함, 계약종료형(banned_mda)만 전 과정에서 제거
# =========================================================
# 전제 패키지:
#   pip install pandas numpy lightgbm pyarrow scikit-learn
# 전제 import:
#   import pandas as pd, numpy as np, lightgbm as lgb, warnings
#   from typing import *
#   from sklearn.metrics import roc_auc_score
warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm")

# -------------------------------
# 유틸
# -------------------------------
def _to_num(s): return pd.to_numeric(s, errors="coerce").fillna(0)
def _to_dt(s):  return pd.to_datetime(s, errors="coerce")

def _norm_cat_col(df: pd.DataFrame, col: str) -> pd.DataFrame:
    if col in df.columns:
        df = df.copy()
        df[col] = df[col].astype(str).str.strip().str.lower()
        df[col] = df[col].replace({"<na>":"__na__", "nan":"__na__", "none":"__na__", "nat":"__na__"})
    return df

def _norm_cat(s): return pd.Series(s, copy=False).astype("string").str.strip().str.lower()

def _pick_col(cols, cand):
    for c in cand:
        if c in cols: return c
    return None

def _ensure_col(df: pd.DataFrame, out_col: str, cands: List[str]) -> pd.DataFrame:
    col = _pick_col(df.columns, cands)
    df = df.copy()
    if col is None: df[out_col] = "__na__"
    else:           df[out_col] = _norm_cat(df[col])
    return df

# -------------------------------
# 1) 윈도우
# -------------------------------
def split_obs_tgt_by_dates(
    ads_time: pd.DataFrame,
    obs_start: str = "2025-07-25",
    tgt_start: str = "2025-08-20",
    obs_end: Optional[str] = None,
    tgt_end: Optional[str] = None,
    tgt_max_days: int = 6
):
    t = ads_time.copy()
    t["date"] = _to_dt(t["rpt_time_date"])
    if t["date"].isna().all():
        raise ValueError("ads_time['rpt_time_date']에 유효한 날짜가 없습니다.")
    obs_start = pd.to_datetime(obs_start).normalize()
    tgt_start = pd.to_datetime(tgt_start).normalize()
    obs_end = (pd.to_datetime(obs_end).normalize() if obs_end is not None
               else (tgt_start - pd.Timedelta(days=1)).normalize())
    last_date = t["date"].max().normalize()
    tgt_end = (pd.to_datetime(tgt_end).normalize() if tgt_end is not None
               else min(last_date, (tgt_start + pd.Timedelta(days=tgt_max_days)).normalize()))
    obs = t[(t["date"] >= obs_start) & (t["date"] <= obs_end)].copy()
    tgt = t[(t["date"] >= tgt_start) & (t["date"] <= tgt_end)].copy()
    return obs, tgt, obs_start, tgt_end, (tgt_start - pd.Timedelta(days=1)).normalize()

# -------------------------------
# 2) 시간/매체/카테고리/도메인 컨텍스트 (OBS만)
# -------------------------------
def _build_time_signals(obs: pd.DataFrame) -> pd.DataFrame:
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["weekday"] = _to_dt(t["rpt_time_date"]).dt.weekday  # 0~6
    wd = t.groupby(["mda_idx","weekday"], as_index=False).agg(clk=("clk","sum"))
    wd["is_weekend"] = wd["weekday"].isin([5,6]).astype(int)
    g = wd.groupby("mda_idx", as_index=False).agg(clk_total=("clk","sum"))
    w = wd.merge(g, on="mda_idx", how="left")
    day_ratio = (w.assign(ratio=w["clk"]/w["clk_total"].replace(0,np.nan))
                   .sort_values(["mda_idx","ratio"], ascending=[True,False])
                   .groupby("mda_idx").head(1)[["mda_idx","ratio"]]
                   .rename(columns={"ratio":"mda_day_ratio"}))
    weekend_ratio = (w.groupby(["mda_idx","is_weekend"], as_index=False)
                       .agg(clk=("clk","sum")).merge(g,on="mda_idx",how="left"))
    weekend_ratio["mda_weekend_ratio"] = (weekend_ratio["clk"] /
                                          weekend_ratio["clk_total"].replace(0,np.nan)).fillna(0)
    weekend_ratio = weekend_ratio[weekend_ratio["is_weekend"]==1][["mda_idx","mda_weekend_ratio"]]
    out = day_ratio.merge(weekend_ratio, on="mda_idx", how="outer").fillna(0.0)
    out["mda_idx"] = out["mda_idx"].astype(int)
    return out

def build_attr_context(obs: pd.DataFrame, ads_list_with_domain: pd.DataFrame):
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["conv"] = _to_num(t.get("rpt_time_turn", 0))

    # mda 전역
    mda_all = (t.groupby("mda_idx", as_index=False)
                 .agg(mda_clk_global=("clk","sum"), mda_conv_global=("conv","sum")))
    mda_all["mda_cvr_global"] = (mda_all["mda_conv_global"] /
                                 mda_all["mda_clk_global"].replace(0,np.nan)).fillna(0.0)
    mu, sd = mda_all["mda_conv_global"].mean(), mda_all["mda_conv_global"].std(ddof=0)
    mda_all["mda_pop_norm"] = (mda_all["mda_conv_global"] - mu) / (sd + 1e-9)

    # 카테고리×매체
    al = ads_list_with_domain[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    al["ads_category"] = _norm_cat(al["ads_category"])
    t2 = t.merge(al, on="ads_idx", how="left")
    cat_mda = (t2.groupby(["mda_idx","ads_category"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    cat_mda["cat_cvr"] = (cat_mda["conv"]/cat_mda["clk"].replace(0,np.nan)).fillna(0.0)
    mda_tot = cat_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    cat_mda = cat_mda.merge(mda_tot, on="mda_idx", how="left")
    cat_mda["cat_share"] = (cat_mda["conv"]/cat_mda["mda_conv_total"].replace(0,np.nan)).fillna(0.0)

    # 도메인×매체
    t_dom = t.merge(
        ads_list_with_domain[["ads_idx","ads_domain"]].drop_duplicates("ads_idx"),
        on="ads_idx", how="left"
    )
    t_dom["ads_domain"] = _norm_cat(t_dom["ads_domain"])
    dom_mda = (t_dom.groupby(["mda_idx","ads_domain"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    dom_mda["dom_cvr"] = (dom_mda["conv"]/dom_mda["clk"].replace(0,np.nan)).fillna(0.0)
    mda_tot2 = dom_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    dom_mda = dom_mda.merge(mda_tot2, on="mda_idx", how="left")
    dom_mda["dom_share"] = (dom_mda["conv"]/dom_mda["mda_conv_total"].replace(0,np.nan)).fillna(0.0)

    # 시간 신호
    time_sig = _build_time_signals(obs)

    # (선택) 광고주 신호
    adv_ctx = None
    if "adv_idx" in ads_list_with_domain.columns:
        t_adv = t.merge(ads_list_with_domain[["ads_idx","adv_idx"]].drop_duplicates("ads_idx"),
                        on="ads_idx", how="left")
        adv_all = (t_adv.groupby("adv_idx", as_index=False)
                        .agg(adv_clk_global=("clk","sum"),
                             adv_conv_global=("conv","sum")))
        adv_all["adv_cvr_global"] = (adv_all["adv_conv_global"] /
                                     adv_all["adv_clk_global"].replace(0,np.nan)).fillna(0.0)
        mu2, sd2 = adv_all["adv_conv_global"].mean(), adv_all["adv_conv_global"].std(ddof=0)
        adv_all["adv_pop_norm"] = (adv_all["adv_conv_global"] - mu2) / (sd2 + 1e-9)
        adv_ctx = adv_all

    return {"mda_all": mda_all, "cat_mda": cat_mda, "dom_mda": dom_mda,
            "time_sig": time_sig, "adv_ctx": adv_ctx}

# -------------------------------
# 3) 학습 대상 광고
# -------------------------------
def pick_train_ads_from_tgt(tgt: pd.DataFrame, min_pos_in_tgt: int = 1,
                            max_train_ads: Optional[int] = 20000, rng_seed: int = 42) -> np.ndarray:
    t = tgt.copy()
    t["turn"] = _to_num(t.get("rpt_time_turn", 0))
    pos = (t.groupby("ads_idx")["turn"].apply(lambda s: (s > 0).sum())
             .reset_index(name="pos_cnt"))
    ok = pos[pos["pos_cnt"] >= max(1,int(min_pos_in_tgt))]["ads_idx"].astype(int).values
    if max_train_ads is not None and len(ok) > max_train_ads:
        rng = np.random.RandomState(rng_seed)
        ok = rng.choice(ok, size=max_train_ads, replace=False)
    return np.sort(ok)

# -------------------------------
# 4) 후보 생성
# -------------------------------
def create_candidates(ads_list: pd.DataFrame, mda_all: pd.DataFrame, cat_mda: pd.DataFrame,
                      topC=10, add_random=2, cat_topM=5,
                      min_active_clk=200, min_active_conv=20,
                      rng_seed=42, ad_ids_filter=None, max_per_ad=20):
    mda_active = mda_all[(mda_all["mda_clk_global"]>=min_active_clk) |
                         (mda_all["mda_conv_global"]>=min_active_conv)].copy()
    if mda_active.empty: mda_active = mda_all.copy()
    mda_ids_global = (mda_active.sort_values("mda_conv_global", ascending=False)["mda_idx"]
                      .astype(int).values)
    cat_top = (cat_mda.sort_values(["ads_category","cat_cvr"], ascending=[True,False])
                      .groupby("ads_category").head(cat_topM)[["ads_category","mda_idx"]])
    cat2top = {c: g["mda_idx"].astype(int).tolist() for c,g in cat_top.groupby("ads_category")}
    rng = np.random.RandomState(rng_seed)
    ad_ids = (np.array(ad_ids_filter, dtype=int) if (ad_ids_filter is not None and len(ad_ids_filter)>0)
              else ads_list["ads_idx"].dropna().astype(int).unique())
    ad_meta = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    ad_meta["ads_idx"] = ad_meta["ads_idx"].astype(int)
    ad_meta["ads_category"] = ad_meta["ads_category"].astype(str).str.strip().str.lower()
    ad2cat = ad_meta.set_index("ads_idx")["ads_category"].to_dict()

    pairs=[]
    for ad in ad_ids:
        base=[]
        cat = ad2cat.get(int(ad), None)
        if cat in cat2top: base.extend(cat2top[cat])
        base.extend(mda_ids_global[:min(topC,len(mda_ids_global))])
        base=list(dict.fromkeys(base))
        remain=np.setdiff1d(mda_ids_global, np.array(base, dtype=int))
        if len(remain)>0 and add_random>0:
            base.extend(rng.choice(remain, size=min(add_random,len(remain)), replace=False).tolist())
        if len(base)>max_per_ad: base=base[:max_per_ad]
        pairs += [(int(ad), int(m)) for m in base]
    out = pd.DataFrame(pairs, columns=["ads_idx","mda_idx"]).drop_duplicates()
    print(f"[Candidates] pairs={len(out):,} | ads={len(ad_ids):,} | active_mda={len(mda_active):,} | topC={topC}, rnd={add_random}, cat_topM={cat_topM}, per_ad≤{max_per_ad}")
    return out

# -------------------------------
# 5) 피처 & 라벨 (OBS 피처 / TGT 라벨)
# -------------------------------
FEAT_ALLOW = [
    # mda 전역
    "mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm",
    # 카테고리×매체
    "cat_cvr","cat_share",
    # 광고 가격
    "price_reward_log","price_contract_log","price_ratio",
    # 도메인×매체
    "dom_cvr","dom_share",
    # 시간 신호
    "mda_day_ratio","mda_weekend_ratio",
    # 광고주 신호
    "adv_cvr_global","adv_pop_norm",
    # 외부 포트폴리오가 있으면
    "mda_size","active_ratio","day_ratio","weekend_ratio"
]

def prepare_training_attr_only(
    obs: pd.DataFrame, tgt: pd.DataFrame, ads_list_with_domain: pd.DataFrame,
    topC=10, add_random=2, cat_topM=5, min_active_clk=200, min_active_conv=20,
    max_train_ads: Optional[int] = 20000, min_pos_in_tgt: int = 1,
    media_portfolio: Optional[pd.DataFrame] = None,
    excluded_mda_ids: Optional[set] = None      # ★ 계약종료형만 제외
):
    # ★ 계약종료형 제외(OBS/TGT)
    if excluded_mda_ids:
        obs = obs[~obs["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()
        tgt = tgt[~tgt["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    train_ads = pick_train_ads_from_tgt(tgt, min_pos_in_tgt=min_pos_in_tgt, max_train_ads=max_train_ads)
    actx = build_attr_context(obs, ads_list_with_domain)
    mda_all, cat_mda, dom_mda = actx["mda_all"], actx["cat_mda"], actx["dom_mda"]

    cand = create_candidates(
        ads_list=ads_list_with_domain, mda_all=mda_all, cat_mda=cat_mda,
        topC=topC, add_random=add_random, cat_topM=cat_topM,
        min_active_clk=min_active_clk, min_active_conv=min_active_conv,
        ad_ids_filter=train_ads, max_per_ad=20
    )

    # ★ 후보 그리드에서도 계약종료형 제외 (진행중 페어는 포함)
    if excluded_mda_ids:
        cand = cand[~cand["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    df = cand.merge(
        mda_all[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]],
        on="mda_idx", how="left"
    )

    ad_meta = ads_list_with_domain[["ads_idx","ads_category","ads_reward_price","ads_contract_price","ads_domain","adv_idx"]].drop_duplicates("ads_idx").copy()
    ad_meta = _norm_cat_col(ad_meta, "ads_category")
    ad_meta = _ensure_col(ad_meta, "ads_domain", ["ads_domain","domain"])
    for c in ["ads_reward_price","ads_contract_price"]:
        ad_meta[c] = _to_num(ad_meta[c])
    ad_meta["price_reward_log"]   = np.log1p(ad_meta["ads_reward_price"])
    ad_meta["price_contract_log"] = np.log1p(ad_meta["ads_contract_price"])
    ad_meta["price_ratio"]        = (ad_meta["ads_reward_price"] /
                                     ad_meta["ads_contract_price"].replace(0,np.nan)).fillna(0.0)

    df = df.merge(ad_meta[["ads_idx","ads_category","ads_domain","adv_idx",
                           "price_reward_log","price_contract_log","price_ratio"]],
                  on="ads_idx", how="left")

    df      = _norm_cat_col(df, "ads_category")
    cat_mda = _norm_cat_col(cat_mda, "ads_category")
    df = df.merge(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]],
                  on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0,"cat_share":0.0})

    dom_mda = dom_mda.copy()
    dom_mda["ads_domain"] = _norm_cat(dom_mda["ads_domain"])
    df["ads_domain"] = _norm_cat(df["ads_domain"])
    df = df.merge(dom_mda[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                  on=["mda_idx","ads_domain"], how="left").fillna({"dom_cvr":0.0,"dom_share":0.0})

    time_sig = actx["time_sig"]
    df = df.merge(time_sig, on="mda_idx", how="left").fillna({"mda_day_ratio":0.0,"mda_weekend_ratio":0.0})

    if actx["adv_ctx"] is not None:
        adv_ctx = actx["adv_ctx"].copy()
        df = df.merge(adv_ctx[["adv_idx","adv_cvr_global","adv_pop_norm"]], on="adv_idx", how="left")
    for c in ["adv_cvr_global","adv_pop_norm"]:
        if c not in df.columns: df[c]=0.0
    df[["adv_cvr_global","adv_pop_norm"]] = df[["adv_cvr_global","adv_pop_norm"]].fillna(0.0)

    if media_portfolio is not None:
        mp = media_portfolio.copy()
        cols = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp.columns]
        if "mda_idx" in cols: df = df.merge(mp[cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in df.columns: df[c]=0.0

    # ---- 라벨: TGT에서 클릭/전환
    tgt2 = tgt.copy()
    tgt2["clk"]  = _to_num(tgt2.get("rpt_time_clk", 0))
    tgt2["turn"] = _to_num(tgt2.get("rpt_time_turn",0))
    y_agg = (tgt2.groupby(["ads_idx","mda_idx"], as_index=False)
                   .agg(tgt_clk=("clk","sum"), tgt_turn=("turn","sum")))
    y_agg["tgt_cvr"] = (y_agg["tgt_turn"] / y_agg["tgt_clk"].replace(0,np.nan)).clip(0,1)

    df = df.merge(y_agg, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_clk":0.0,"tgt_turn":0.0})

    # 랭커용 relevance
    y = df["tgt_turn"].astype(float)
    rel = np.zeros(len(df), dtype=int)
    pos = y > 0
    if pos.sum()>0:
        ylog = np.log1p(y[pos])
        try:
            bins = pd.qcut(ylog, q=3, labels=False, duplicates="drop")
            rel[pos] = bins.astype(int)+1
        except ValueError:
            rel[pos] = 1
    df["relevance"] = rel
    return df, actx

# -------------------------------
# 6) 학습기들 (랭커 + 클릭회귀 + CVR회귀)
# -------------------------------
LABEL_GAIN = [0.0,1.0,4.0,12.0]

def ensure_feature_matrix(df, feat_cols):
    X = df.copy()
    for c in feat_cols:
        if c not in X.columns: X[c]=0.0
        if not np.issubdtype(X[c].dtype, np.number):
            X[c]=pd.to_numeric(X[c], errors="coerce")
    return X[feat_cols].fillna(0.0).astype(np.float32)

def downsample_negatives_per_ad(df, ratio=4, random_state=42):
    out, rng = [], np.random.RandomState(random_state)
    for ad, g in df.groupby("ads_idx", sort=False):
        pos = g[g["tgt_turn"]>0]; neg = g[g["tgt_turn"]<=0]
        if len(pos)==0: continue
        cap = len(pos)*ratio
        if len(neg)>cap: neg = neg.sample(n=cap, random_state=rng)
        out.append(pd.concat([pos,neg], ignore_index=True))
    return pd.concat(out, ignore_index=True) if out else df

def train_attr_ranker(train_df: pd.DataFrame):
    feat_cols = [c for c in FEAT_ALLOW if c in train_df.columns]
    X = ensure_feature_matrix(train_df, feat_cols)
    y = train_df["relevance"].astype(int)
    groups = train_df.groupby("ads_idx").size().astype(int).tolist()
    ranker = lgb.LGBMRanker(
        objective="lambdarank", metric="ndcg", eval_at=[5,10,20],
        learning_rate=0.05, n_estimators=1200, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        label_gain=LABEL_GAIN, random_state=42, n_jobs=-1, verbosity=-1
    )
    ranker.fit(X, y, group=groups, sample_weight=np.ones(len(train_df)))
    return ranker, feat_cols

def train_click_regressor(train_df: pd.DataFrame, feat_cols: List[str], tweedie_p: float = 1.1):
    X = ensure_feature_matrix(train_df, feat_cols)
    y = train_df["tgt_clk"].astype(float)
    reg = lgb.LGBMRegressor(
        objective="tweedie", tweedie_variance_power=tweedie_p,
        n_estimators=1200, learning_rate=0.05, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        random_state=42, n_jobs=-1, verbosity=-1
    )
    reg.fit(X, y)
    return reg, {"mode":"tweedie_click","twp":tweedie_p}

def _logit_clip(p, eps=1e-6):
    p = np.clip(p, eps, 1-eps)
    return np.log(p/(1-p))

def _sigmoid(x): return 1/(1+np.exp(-x))

def train_cvr_regressor(train_df: pd.DataFrame, feat_cols: List[str]):
    g = train_df[train_df["tgt_clk"]>0].copy()
    if g.empty:
        dummy = lgb.LGBMRegressor().fit(np.zeros((1, len(feat_cols))), [0.0])
        return dummy, {"mode":"cvr_logit","fallback":True, "mean_logit":-20.0}
    X = ensure_feature_matrix(g, feat_cols)
    y = g["tgt_turn"]/g["tgt_clk"]
    y = y.clip(0,1)
    y_logit = _logit_clip(y.values)
    reg = lgb.LGBMRegressor(
        objective="regression",
        n_estimators=1200, learning_rate=0.05, num_leaves=47,
        min_data_in_leaf=40, lambda_l2=1.5, subsample=0.85, colsample_bytree=0.85,
        random_state=42, n_jobs=-1, verbosity=-1
    )
    reg.fit(X, y_logit)
    return reg, {"mode":"cvr_logit","fallback":False}

# -------------------------------
# 7) 예측 (랭커 스코어 + 클릭×CVR 예측)
# -------------------------------
def predict_all_mda_with_counts_batched(
    ranker, click_reg, cvr_reg, feat_cols, actx, ads_list_with_domain,
    K=20, batch_size=5000, max_predict_ads: Optional[int] = 20000,
    media_portfolio: Optional[pd.DataFrame] = None,
    excluded_mda_ids: Optional[set] = None      # ★ 계약종료형만 제외
) -> Dict[int, pd.DataFrame]:

    mda_all, cat_mda, dom_mda = actx["mda_all"], actx["cat_mda"], actx["dom_mda"]
    time_sig, adv_ctx = actx["time_sig"], actx["adv_ctx"]

    mda_active = mda_all[(mda_all["mda_clk_global"]>=200) | (mda_all["mda_conv_global"]>=20)]
    if mda_active.empty: mda_active = mda_all
    if excluded_mda_ids:
        mda_active = mda_active[~mda_active["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    mda_active = mda_active[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]].copy()
    mda_active["mda_idx"]=mda_active["mda_idx"].astype(int)
    mda_active = mda_active.merge(time_sig, on="mda_idx", how="left").fillna({"mda_day_ratio":0.0,"mda_weekend_ratio":0.0})

    if media_portfolio is not None:
        mp = media_portfolio.copy()
        cols = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp.columns]
        if "mda_idx" in cols: mda_active = mda_active.merge(mp[cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in mda_active.columns: mda_active[c]=0.0

    ads_meta = ads_list_with_domain[["ads_idx","ads_category","ads_reward_price","ads_contract_price","ads_domain","adv_idx"]].drop_duplicates("ads_idx").copy()
    if max_predict_ads is not None: ads_meta = ads_meta.head(int(max_predict_ads))
    ads_meta = _norm_cat_col(ads_meta, "ads_category")
    ads_meta = _ensure_col(ads_meta, "ads_domain", ["ads_domain","domain"])
    ads_meta["ads_idx"] = ads_meta["ads_idx"].astype(int)
    ads_meta["ads_domain"] = _norm_cat(ads_meta["ads_domain"])
    ads_meta["ads_reward_price"]   = _to_num(ads_meta["ads_reward_price"])
    ads_meta["ads_contract_price"] = _to_num(ads_meta["ads_contract_price"])
    ads_meta["price_reward_log"]   = np.log1p(ads_meta["ads_reward_price"])
    ads_meta["price_contract_log"] = np.log1p(ads_meta["ads_contract_price"])
    ads_meta["price_ratio"]        = (ads_meta["ads_reward_price"] / ads_meta["ads_contract_price"].replace(0,np.nan)).fillna(0.0)
    if adv_ctx is not None:
        ads_meta = ads_meta.merge(adv_ctx[["adv_idx","adv_cvr_global","adv_pop_norm"]], on="adv_idx", how="left")
    for c in ["adv_cvr_global","adv_pop_norm"]:
        if c not in ads_meta.columns: ads_meta[c]=0.0
    ads_meta[["adv_cvr_global","adv_pop_norm"]] = ads_meta[["adv_cvr_global","adv_pop_norm"]].fillna(0.0)

    cat_mda_local = _norm_cat_col(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]].copy(), "ads_category")
    dom_mda_local = dom_mda.copy(); dom_mda_local["ads_domain"] = _norm_cat(dom_mda_local["ads_domain"])

    out={}
    ad_ids = ads_meta["ads_idx"].tolist()
    for i in range(0, len(ad_ids), batch_size):
        ad_batch = ads_meta.iloc[i:i+batch_size]
        grid = ad_batch[["ads_idx","ads_category","ads_domain","price_reward_log","price_contract_log","price_ratio",
                         "adv_idx","adv_cvr_global","adv_pop_norm"]] \
               .merge(mda_active, how="cross")

        # ★ 진행중 페어 제외 없음 (포함)

        grid = grid.merge(cat_mda_local, on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0,"cat_share":0.0})
        grid = grid.merge(dom_mda_local[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                          on=["mda_idx","ads_domain"], how="left").fillna({"dom_cvr":0.0,"dom_share":0.0})

        X = ensure_feature_matrix(grid, feat_cols)
        score = ranker.predict(X)  # 랭커 점수

        # Decomposition: click × cvr
        yhat_clk = np.maximum(0.0, click_reg.predict(X))
        yhat_cvr_logit = cvr_reg.predict(X)
        yhat_cvr = np.clip(_sigmoid(yhat_cvr_logit), 0.0, 1.0)
        yhat_turn = yhat_clk * yhat_cvr

        grid2 = grid[["ads_idx","mda_idx","ads_category","mda_cvr_global","mda_pop_norm","mda_clk_global","mda_conv_global"]].copy()
        grid2["score"] = score.astype(float)
        grid2["yhat_turn"] = yhat_turn.astype(float)

        for ad, g in grid2.groupby("ads_idx", sort=False):
            top = g.nlargest(K, "score")
            out[int(ad)] = top.sort_values("score", ascending=False).reset_index(drop=True)
        print(f"[Predict+Count] processed ads {i+len(ad_batch)}/{len(ad_ids)}")
    return out

# -------------------------------
# 8) 평가
# -------------------------------
def _precision_recall_hit_at_k(preds_dict, gt_dict, k=10):
    P,R,H=[],[],[]
    for ad, df in preds_dict.items():
        if ad not in gt_dict: continue
        pred = df["mda_idx"].astype(int).tolist()[:k]
        true = set(gt_dict[ad]); 
        if not true: continue
        inter = len(set(pred)&true)
        P.append(inter/max(1,len(pred))); R.append(inter/len(true)); H.append(1.0 if inter>0 else 0.0)
    return (float(np.mean(P)) if P else 0.0,
            float(np.mean(R)) if R else 0.0,
            float(np.mean(H)) if H else 0.0)

def _map_at_k(preds_dict, gt_dict, k=10):
    APs=[]
    for ad, df in preds_dict.items():
        if ad not in gt_dict: continue
        true=set(gt_dict[ad]); 
        if not true: continue
        ranked = df["mda_idx"].astype(int).tolist()[:k]
        hits=cum_prec=0
        for i,m in enumerate(ranked, start=1):
            if m in true:
                hits+=1; cum_prec += hits/i
        APs.append(cum_prec/min(len(true),k) if hits>0 else 0.0)
    return float(np.mean(APs)) if APs else 0.0

def _auc_flat(preds_dict, gt_pairs_dict, score_col="score"):
    y_true, y_score = [], []
    for ad, df in preds_dict.items():
        true_mdas = set(gt_pairs_dict.get(ad, []))
        for _, r in df.iterrows():
            y_true.append(1 if int(r["mda_idx"]) in true_mdas else 0)
            y_score.append(float(r[score_col]))
    if len(set(y_true))<2: return 0.5
    try: return float(roc_auc_score(y_true, y_score))
    except Exception: return 0.5

def evaluate_all(preds_with_counts: Dict[int, pd.DataFrame], tgt: pd.DataFrame, k=10):
    gt_pairs = tgt.groupby("ads_idx")["mda_idx"].apply(lambda s: list(pd.Series(s).dropna().astype(int).unique())).to_dict()
    prec, rec, hit = _precision_recall_hit_at_k(preds_with_counts, gt_pairs, k=k)
    mapk = _map_at_k(preds_with_counts, gt_pairs, k=k)
    auc_rank = _auc_flat(preds_with_counts, gt_pairs, score_col="score")
    auc_reg  = _auc_flat(preds_with_counts, gt_pairs, score_col="yhat_turn")
    return {f"precision@{k}":prec, f"recall@{k}":rec, f"map@{k}":mapk,
            f"hit_rate@{k}":hit, "auc_ranker":auc_rank, "auc_regressor":auc_reg}

# -------------------------------
# 9) 편의
# -------------------------------
def get_expected_conversions_for_ad(preds_with_counts: Dict[int, pd.DataFrame],
                                    ad_id: int, sort_by="yhat_turn", top=20) -> pd.DataFrame:
    df = preds_with_counts.get(int(ad_id))
    if df is None or df.empty:
        return pd.DataFrame(columns=["mda_idx","score","yhat_turn"])
    sort_by = "yhat_turn" if sort_by not in ("yhat_turn","score") else sort_by
    out = df.sort_values(sort_by, ascending=False).copy()
    return out.head(top) if top is not None else out

# =========================================================
# ==============   메인 플로우 (예시)   ====================
# =========================================================
# 요구 입력 예시:
#   ads_time, ads_list, ads_segment(ads_size, domain),
#   media_performance_classification(옵션), media_portfolio(옵션)
#   ※ 진행중 페어는 전달하더라도 이 버전은 '제외하지 않음'

# (A) 윈도우
obs, tgt, start, end, cut = split_obs_tgt_by_dates(
    ads_time, obs_start="2025-07-25", tgt_start="2025-08-20"
)
print(f"[Window] OBS: {start.date()} → {cut.date()}  |  TGT: {(cut+pd.Timedelta(days=1)).date()} → {end.date()}")

# (A-1) 계약종료형 매체 목록
if 'media_performance_classification' in globals():
    banned_mda = set(
        media_performance_classification
          .loc[media_performance_classification["basic_classification"].astype(str).str.strip() == "계약종료형", "mda_idx"]
          .dropna().astype(int).unique()
    )
else:
    banned_mda = set()
print(f"[Exclude] 계약종료형 mda 개수 = {len(banned_mda)}")

# (A-2) OBS/TGT에서 계약종료형 제거
if banned_mda:
    obs = obs[~obs["mda_idx"].astype(int).isin(banned_mda)].copy()
    tgt = tgt[~tgt["mda_idx"].astype(int).isin(banned_mda)].copy()

# (B) ads_list 확장 (ads_size, domain 병합)
ads_list_ext = ads_list.merge(
    ads_segment[["ads_idx","ads_size","domain"]].rename(columns={"domain":"ads_domain"}),
    on="ads_idx", how="left"
)

# (C) 사이즈 필터(예: MEGA/LARGE/MEDIUM만)
size_whitelist = {"MEGA","LARGE","MEDIUM"}
ads_list_sel = ads_list_ext[
    ads_list_ext["ads_size"].astype(str).str.strip().str.upper().isin(size_whitelist)
].copy()
sel_ids = set(ads_list_sel["ads_idx"].astype(int))
tgt_sel  = tgt[tgt["ads_idx"].astype(int).isin(sel_ids)].copy()
print(f"[Filter] sizes kept = {sorted(size_whitelist)} | ads={ads_list_sel['ads_idx'].nunique():,} | tgt pairs={len(tgt_sel):,}")

# (D) 학습 데이터 (★ 계약종료형만 제외, 진행중 페어는 포함)
train_df, actx = prepare_training_attr_only(
    obs, tgt_sel, ads_list_sel,
    topC=10, add_random=2, cat_topM=5,
    min_active_clk=200, min_active_conv=20,
    max_train_ads=20000, min_pos_in_tgt=1,
    media_portfolio=None,
    excluded_mda_ids=banned_mda
)
print("train_df size:", len(train_df))

# (E) 학습 (랭커 + 클릭 + CVR)
train_df_ds = downsample_negatives_per_ad(train_df, ratio=4)
ranker, feat_cols   = train_attr_ranker(train_df_ds)
click_reg, click_md = train_click_regressor(train_df_ds, feat_cols, tweedie_p=1.1)
cvr_reg, cvr_md     = train_cvr_regressor(train_df_ds, feat_cols)
print("feat_cols used:", feat_cols)

# (F) 예측 & 평가 (★ 예측 교차조인에서도 계약종료형만 제외, 진행중 페어는 포함)
preds_with_counts = predict_all_mda_with_counts_batched(
    ranker, click_reg, cvr_reg, feat_cols, actx, ads_list_sel,
    K=20, batch_size=5000, max_predict_ads=20000, media_portfolio=None,
    excluded_mda_ids=banned_mda
)
metrics_k5  = evaluate_all(preds_with_counts, tgt_sel, k=5)
metrics_k10 = evaluate_all(preds_with_counts, tgt_sel, k=10)
print("Eval@5:", metrics_k5)
print("Eval@10:", metrics_k10)

# (G) Sanity Check
leftover_tgt = tgt_sel["mda_idx"].astype(int).isin(banned_mda).sum()
print(f"[Sanity] TGT(selected sizes) 내 계약종료형 잔여: {leftover_tgt}")
any_banned_in_preds = any(
    int(m) in banned_mda
    for df in preds_with_counts.values()
    for m in df["mda_idx"].astype(int).tolist()
)
print(f"[Sanity] 예측결과에 계약종료형 포함 여부: {any_banned_in_preds}")

# (H) 특정 광고 Top-K (예상 전환수 기준)
# ad_id = 446406
# top5 = get_expected_conversions_for_ad(preds_with_counts, ad_id, sort_by="yhat_turn", top=5)
# print(top5[["mda_idx","yhat_turn","score"]])


[Window] OBS: 2025-07-25 → 2025-08-19  |  TGT: 2025-08-20 → 2025-08-25
[Exclude] 계약종료형 mda 개수 = 51
[Filter] sizes kept = ['LARGE', 'MEDIUM', 'MEGA'] | ads=7,169 | tgt pairs=63,274
[Candidates] pairs=26,263 | ads=1,565 | active_mda=128 | topC=10, rnd=2, cat_topM=5, per_ad≤20
train_df size: 26263
feat_cols used: ['mda_clk_global', 'mda_conv_global', 'mda_cvr_global', 'mda_pop_norm', 'cat_cvr', 'cat_share', 'price_reward_log', 'price_contract_log', 'price_ratio', 'dom_cvr', 'dom_share', 'mda_day_ratio', 'mda_weekend_ratio', 'adv_cvr_global', 'adv_pop_norm', 'mda_size', 'active_ratio', 'day_ratio', 'weekend_ratio']
[Predict+Count] processed ads 5000/7169
[Predict+Count] processed ads 7169/7169
Eval@5: {'precision@5': 0.2850102669404518, 'recall@5': 0.7922062310387551, 'map@5': 0.6797374800365047, 'hit_rate@5': 0.9040041067761807, 'auc_ranker': 0.8820701576967978, 'auc_regressor': 0.8492052808958677}
Eval@10: {'precision@10': 0.17109856262833675, 'recall@10': 0.875213873423343, 'map@10': 0.

In [ ]:
ad_id = 440035
top5 = get_expected_conversions_for_ad(preds_with_counts, ad_id, sort_by="yhat_turn", top=10)
print(top5[["mda_idx","yhat_turn","score"]])


    mda_idx  yhat_turn     score
1        58   9.248167  2.072742
9        56   1.206707 -0.085316
14      371   0.786343 -0.553437
6       540   0.658247  0.249208
4        54   0.624396  1.075022
3        14   0.276957  1.097826
8       761   0.271803  0.078182
5       281   0.244026  0.664229
19      270   0.129636 -1.306427
2       583   0.101911  1.489639


In [ ]:
preds_with_counts

{9935:     ads_idx  mda_idx ads_category  mda_cvr_global  mda_pop_norm  \
 0      9935      583            8        0.041567     -0.234137   
 1      9935       14            8        0.271225      0.648722   
 2      9935      371            8        0.438984      0.268392   
 3      9935      398            8        0.168241     -0.205769   
 4      9935      651            8        0.070492     -0.247868   
 5      9935       56            8        0.257679      0.151676   
 6      9935      281            8        0.091837     -0.244769   
 7      9935      761            8        0.178169      0.230726   
 8      9935       18            8        0.044420     -0.240145   
 9      9935      270            8        0.480142      0.856218   
 10     9935      790            8        0.511957     -0.222361   
 11     9935      496            8        0.199126      0.692777   
 12     9935       54            8        0.337562     -0.082472   
 13     9935       58            8        

In [ ]:
import joblib

bundle = { 'preds_with_counts' : preds_with_counts,'tgt_large' : tgt_large }

# 저장
joblib.dump(bundle, "model.pkl")


['model.pkl']

In [ ]:
ad_id = 446406
top5 = get_expected_conversions_for_ad(preds_with_counts, ad_id, top=5)
print(top5[["mda_idx","yhat_turn","score"]])

   mda_idx  yhat_turn     score
0      563  36.632389  7.815320
2      562   8.114429  3.327594
3      761   1.707933  1.817497
4      634   1.135490  1.482015
1      371   0.686771  3.385891


In [ ]:
# 예: 학습 후 전체 검증(TGT) 구간 평가
metrics_k5  = evaluate_all(preds_with_counts, tgt_large, k=5)
metrics_k10 = evaluate_all(preds_with_counts, tgt_large, k=10)

print("[Eval@5]")
print("  precision@5 :", metrics_k5["precision@5"])
print("  recall@5    :", metrics_k5["recall@5"])
print("  map@5       :", metrics_k5["map@5"])
print("  auc_ranker  :", metrics_k5["auc_ranker"])
print("  auc_regress :", metrics_k5["auc_regressor"])

print("\n[Eval@10]")
print("  precision@10:", metrics_k10["precision@10"])
print("  recall@10   :", metrics_k10["recall@10"])
print("  map@10      :", metrics_k10["map@10"])
print("  auc_ranker  :", metrics_k10["auc_ranker"])
print("  auc_regress :", metrics_k10["auc_regressor"])

ma / rmse 예측 성과

SyntaxError: invalid syntax (3584187346.py, line 19)

In [ ]:
# TGT 구간 확인 (2025-08-20 ~ 2025-08-25)
tgt_period = tgt[
    (tgt["date"] >= pd.to_datetime("2025-08-20")) &
    (tgt["date"] <= pd.to_datetime("2025-08-25"))
].copy()

# 광고별 집계 (전환 합계 포함)
tgt_ads_summary = (
    tgt_period.groupby("ads_idx", as_index=False)
    .agg(clicks=("rpt_time_clk", "sum"),
         turns=("rpt_time_turn", "sum"),
         mdas=("mda_idx", "nunique"))
    .sort_values("turns", ascending=False)
)

# LARGE 광고 ID 집합
large_ids = set(ads_list_large["ads_idx"].astype(int))

# TGT 요약에서 LARGE만 필터링
tgt_ads_summary_large = tgt_ads_summary[
    tgt_ads_summary["ads_idx"].astype(int).isin(large_ids)
].copy()

print("=== TGT 기간 광고별 요약 (LARGE만, 2025-08-20 ~ 2025-08-25) ===")
print(tgt_ads_summary_large.head(20))



=== TGT 기간 광고별 요약 (LARGE만, 2025-08-20 ~ 2025-08-25) ===
      ads_idx  clicks  turns  mdas
920    443285    1951   1146     7
775    439163    2357    659     6
282    131204     873    601     7
1240   444946    1788    593     6
2180   446406     405    322     5
128     28051    1770    298     3
2040   446164     387    293     2
2320   446698     371    268     3
1956   446053     329    261     3
1592   445562     429    233     7
2301   446671     430    232     6
1508   445452     375    227     2
2114   446285     396    216     5
2129   446300     287    213     5
483    334540    1014    206    10
484    334542     759    206     9
485    334543     708    206    10
1965   446070     302    205     6
1410   445333     250    204     2
2187   446418     243    204     2


In [ ]:
# (G) 예: 특정 광고 top-5 (예상 전환수 기준)
ad_id = 22232
top5 = get_expected_conversions_for_ad(preds_with_counts, ad_id, sort_by="yhat_turn", top=5)
print(top5[["mda_idx","yhat_turn","score"]])


   mda_idx   yhat_turn     score
0       22  599.372128  4.871929
1      496  480.184964  3.437047
3      270   42.537119  1.355760
6       54   25.013968 -0.613628
8      761    8.715921 -1.501533


In [ ]:
def get_expected_vs_actual_for_ad(preds_with_counts: Dict[int, pd.DataFrame],
                                  tgt: pd.DataFrame,
                                  ad_id: int,
                                  sort_by: str = "yhat_turn",
                                  top: Optional[int] = 20) -> pd.DataFrame:
    """
    특정 광고의 매체사별 Top-K 추천 결과에서
    예측 전환수(yhat_turn), 점수(score), 실제 전환수(tgt_turn)를 같이 반환
    """
    df_pred = preds_with_counts.get(int(ad_id))
    if df_pred is None or df_pred.empty:
        return pd.DataFrame(columns=["mda_idx","score","yhat_turn","tgt_turn"])

    # 실제 전환수 집계 (TGT 기준)
    tgt_g = (tgt[tgt["ads_idx"]==ad_id]
               .groupby("mda_idx", as_index=False)
               .agg(tgt_turn=("rpt_time_turn","sum")))

    df = df_pred.merge(tgt_g, on="mda_idx", how="left").fillna({"tgt_turn":0.0})

    sort_by = "yhat_turn" if sort_by not in ("yhat_turn","score") else sort_by
    df = df.sort_values(sort_by, ascending=False)
    return df.head(top) if top is not None else df


In [ ]:
ad_id = 443285
top5_cmp = get_expected_vs_actual_for_ad(
    preds_with_counts, tgt_large, ad_id=ad_id,
    sort_by="yhat_turn", top=10
)

print(f"=== Ad {ad_id} 추천 매체사 Top-10 (예상 전환수 vs 실제 전환수) ===")
print(top5_cmp[["mda_idx","yhat_turn","tgt_turn","score"]])


=== Ad 443285 추천 매체사 Top-10 (예상 전환수 vs 실제 전환수) ===
    mda_idx   yhat_turn  tgt_turn     score
0        22  599.372128     453.0  4.871929
1       496  480.184964     398.0  3.437047
3       270   42.537119      59.0  1.355760
6        54   25.013968      26.0 -0.613628
8       761    8.715921     199.0 -1.501533
4       246    6.112913       6.0 -0.202180
11      645    4.702746       5.0 -2.295956
7       848    2.789174       0.0 -1.058276
2       371    1.368531       0.0  2.500271
5       290    0.530539       0.0 -0.294814


In [ ]:
# 0) 전제: 아래 객체들이 이미 준비되어 있어야 함
#   - actx: OBS로 만든 컨텍스트(build_attr_context 결과)
#   - ranker, click_reg, cvr_reg, feat_cols: 직전에 학습한 모델들
#   - predict_all_mda_with_counts_batched, get_expected_conversions_for_ad: 이전 정의 함수

# 1) 신규 광고 메타 준비
#   (ads_new 예시 컬럼: ['ads_idx','ads_type','ads_category','ads_name','ads_os_type',
#                      'ads_contract_price','ads_media_price','ads_reward_price',
#                      'ads_rejoin_type','domain'])
ads_new_meta = ads_new.copy()

# 우리 파이프라인이 기대하는 컬럼명으로 맞추기
ads_new_meta = ads_new_meta.rename(columns={
    "domain": "ads_domain"
})

# 숫자형 정리(가격)
for c in ["ads_contract_price", "ads_reward_price"]:
    if c in ads_new_meta.columns:
        ads_new_meta[c] = pd.to_numeric(ads_new_meta[c], errors="coerce").fillna(0)

# 카테고리/도메인 정규화(낙관적 기본값 처리)
ads_new_meta["ads_category"] = (
    ads_new_meta["ads_category"]
    .astype(str).str.strip().str.lower()
    .replace({"<na>":"__na__", "nan":"__na__", "none":"__na__", "nat":"__na__"})
)
ads_new_meta["ads_domain"] = (
    ads_new_meta["ads_domain"]
    .astype(str).str.strip().str.lower()
    .replace({"<na>":"__na__", "nan":"__na__", "none":"__na__", "nat":"__na__"})
)

# (선택) adv_idx 있으면 가져가고, 없으면 0으로 채움
if "adv_idx" not in ads_new_meta.columns:
    ads_new_meta["adv_idx"] = 0

# 예측에 필요한 최소 컬럼만 남겨도 됨
ads_new_meta = ads_new_meta[[
    "ads_idx", "ads_category", "ads_domain",
    "ads_contract_price", "ads_reward_price", "adv_idx"
]]

# 2) 예측 실행: 신규 광고만 넣어서 예측(학습된 actx와 매체 통계 사용)
preds_new = predict_all_mda_with_counts_batched(
    ranker=ranker,
    click_reg=click_reg,
    cvr_reg=cvr_reg,
    feat_cols=feat_cols,
    actx=actx,                       # OBS에서 만든 컨텍스트(그대로 사용)
    ads_list_with_domain=ads_new_meta,  # ★ 신규 광고만 투입
    K=20,
    batch_size=5000,
    max_predict_ads=None,            # 제한 없이
    media_portfolio=None
)

# 3) 광고별 Top-5 결과 보기 (예상 전환수 기준)
for ad in ads_new_meta["ads_idx"].astype(int).tolist():
    top5 = get_expected_conversions_for_ad(preds_new, ad_id=ad, sort_by="yhat_turn", top=5)
    print(f"\n=== 신규 광고 {ad} 추천 매체사 Top-5 (예상 전환수 기준) ===")
    if top5.empty:
        print("(예측 없음: 후보 매체가 없거나 컨텍스트가 비어있을 수 있음)")
    else:
        print(top5[["mda_idx","yhat_turn","score"]])


[Predict+Count] processed ads 8/8

=== 신규 광고 500000 추천 매체사 Top-5 (예상 전환수 기준) ===
    mda_idx  yhat_turn     score
1       634   0.004583 -3.316352
0       270   0.001140 -3.065564
15      496   0.000612 -4.195602
3       980   0.000415 -3.558058
14      851   0.000338 -4.193136

=== 신규 광고 500001 추천 매체사 Top-5 (예상 전환수 기준) ===
   mda_idx  yhat_turn     score
3      761  24.020499  1.837828
0      563   2.910245  6.150732
1      371   1.595841  3.223704
2      562   0.406377  2.443147
7      645   0.190282 -1.389569

=== 신규 광고 500002 추천 매체사 Top-5 (예상 전환수 기준) ===
    mda_idx  yhat_turn     score
0       761   2.477358  1.337013
3       371   1.825052  0.683859
12      291   1.000618 -0.887503
4       790   0.613124  0.250106
8       645   0.434980 -0.195832

=== 신규 광고 500003 추천 매체사 Top-5 (예상 전환수 기준) ===
   mda_idx  yhat_turn     score
0       22   1.500759  3.566391
2      398   0.795406  0.905209
1       14   0.617327  1.549063
5      401   0.112794  0.487981
7       12   0.017736 -0.34782

In [ ]:
ads_new_meta = ads_new.copy()
ads_new_meta = ads_new_meta.rename(columns={"domain": "ads_domain"})

# 가격 숫자화
for c in ["ads_contract_price", "ads_reward_price"]:
    ads_new_meta[c] = pd.to_numeric(ads_new_meta.get(c, 0), errors="coerce").fillna(0)

# 카테고리/도메인 정규화
ads_new_meta["ads_category"] = (
    ads_new_meta.get("ads_category","__na__")
    .astype(str).str.strip().str.lower()
    .replace({"<na>":"__na__","nan":"__na__","none":"__na__","nat":"__na__"})
)
ads_new_meta["ads_domain"] = (
    ads_new_meta.get("ads_domain","__na__")
    .astype(str).str.strip().str.lower()
    .replace({"<na>":"__na__","nan":"__na__","none":"__na__","nat":"__na__"})
)

# adv_idx/ads_name 기본값
ads_new_meta["adv_idx"]  = ads_new_meta.get("adv_idx", 0)
ads_new_meta["ads_name"] = ads_new_meta.get("ads_name", "").astype(str)

# 예측에 필요한 컬럼(ads_name 포함)만 유지
ads_new_meta = ads_new_meta[[
    "ads_idx","ads_category","ads_domain",
    "ads_contract_price","ads_reward_price","adv_idx","ads_name"  # ← 포함!
]].drop_duplicates("ads_idx")


In [ ]:
preds_new = predict_all_mda_with_counts_batched(
    ranker, click_reg, cvr_reg, feat_cols, actx,
    ads_list_with_domain=ads_new_meta,
    K=20, batch_size=5000, max_predict_ads=None, media_portfolio=None
)

for ad in ads_new_meta["ads_idx"].astype(int):
    top5 = get_expected_conversions_for_ad(preds_new, ad, sort_by="yhat_turn", top=5)
    print(f"\n=== 신규 광고 {ad} 추천 매체사 Top-5 (yhat_turn 기준) ===")
    print("[EMPTY] 이 광고는 예측 결과가 없습니다." if top5.empty else top5[["mda_idx","yhat_turn","score"]])


[Predict+Count] processed ads 8/8

=== 신규 광고 500000 추천 매체사 Top-5 (yhat_turn 기준) ===
    mda_idx  yhat_turn     score
4       634   0.183193 -3.947494
6       344   0.006734 -4.325198
0       270   0.005709 -3.393299
2        22   0.001796 -3.927919
12      563   0.001787 -4.617729

=== 신규 광고 500001 추천 매체사 Top-5 (yhat_turn 기준) ===
   mda_idx  yhat_turn     score
0      563   2.834244  6.694795
2      761   2.811726  2.565953
8      645   0.531498 -1.693402
9      790   0.088987 -2.352940
1      371   0.055422  3.215654

=== 신규 광고 500002 추천 매체사 Top-5 (yhat_turn 기준) ===
   mda_idx  yhat_turn     score
2      371   4.473921  1.458189
0      270   3.706649  1.740088
1      761   1.521165  1.512758
4      817   0.704295  0.300374
7      645   0.665452 -0.615812

=== 신규 광고 500003 추천 매체사 Top-5 (yhat_turn 기준) ===
    mda_idx  yhat_turn     score
1        14  16.987511  2.734970
0        22   5.134284  3.898190
2       398   4.678629  2.230773
5       401   1.032783  0.638468
16      634   0.113

In [ ]:
ㅋㅋㅋㅋ

NameError: name 'ᄏᄏᄏᄏ' is not defined

In [ ]:
# =========================================================
# 신규 광고(ads_new) → 매체사 추천 Top-K (yhat_turn 기준)
#  - ads_name 포함 (이름 피처는 predict 함수 내부에서 자동 부착됨)
#  - (옵션) easy_coldstart_blend_v2로 신규 폭주 완화
# 전제: actx, ranker, click_reg, cvr_reg, feat_cols, 
#       predict_all_mda_with_counts_batched, get_expected_conversions_for_ad,
#       _attach_name_features / build_name_features 포함된 최신 코드가 이미 로드되어 있음
# =========================================================

# 0) (선택) 블렌딩 함수가 없다면 정의
try:
    easy_coldstart_blend_v2
except NameError:
    import numpy as np
    def easy_coldstart_blend_v2(df, actx, tgt=None, cvr_cap=0.12, w=0.7, click_budget=None):
        if df is None or df.empty: return df
        mda_all = actx["mda_all"].copy()
        glb_clk = float(mda_all["mda_clk_global"].sum())
        glb_conv = float(mda_all["mda_conv_global"].sum())
        glb_cvr = (glb_conv / glb_clk) if glb_clk > 0 else 0.0
        glb_cvr = min(glb_cvr, cvr_cap)
        if click_budget is None:
            if (tgt is not None) and ("rpt_time_clk" in tgt.columns):
                per_ad_clk = tgt.groupby("ads_idx")["rpt_time_clk"].sum()
                click_budget = max(30.0, float(per_ad_clk.median()) if len(per_ad_clk) else 50.0)
            else:
                click_budget = 50.0
        base_clk = df["mda_clk_global"].astype(float).clip(lower=0.0)
        share = base_clk / max(1.0, base_clk.sum())
        clk_prior = share * click_budget
        turn_prior = clk_prior * glb_cvr
        out = df.copy()
        turn_model = out["yhat_turn"].astype(float).clip(lower=0.0)
        out["yhat_turn"] = w*turn_model + (1.0 - w)*turn_prior
        if turn_model.notnull().sum() > 0 and (turn_model > 0).any():
            cap_val = max(1.0, float(np.percentile(turn_model[turn_model>0], 99)) * 3.0)
            out["yhat_turn"] = np.minimum(out["yhat_turn"], cap_val)
        return out

# 1) 신규 광고 메타 준비 (ads_name 포함!)
ads_new_meta = ads_new.copy()
ads_new_meta = ads_new_meta.rename(columns={"domain": "ads_domain"})

# 숫자형 정리
for c in ["ads_contract_price", "ads_reward_price"]:
    if c not in ads_new_meta.columns:
        ads_new_meta[c] = 0
    ads_new_meta[c] = pd.to_numeric(ads_new_meta[c], errors="coerce").fillna(0)

# 카테고리/도메인 정규화
ads_new_meta["ads_category"] = (
    ads_new_meta.get("ads_category","__na__")
    .astype(str).str.strip().str.lower()
    .replace({"<na>":"__na__", "nan":"__na__", "none":"__na__", "nat":"__na__"})
)
ads_new_meta["ads_domain"] = (
    ads_new_meta.get("ads_domain","__na__")
    .astype(str).str.strip().str.lower()
    .replace({"<na>":"__na__", "nan":"__na__", "none":"__na__", "nat":"__na__"})
)

# adv_idx 기본값
if "adv_idx" not in ads_new_meta.columns:
    ads_new_meta["adv_idx"] = 0

# 예측에 필요한 최소 컬럼(★ ads_name 포함)
cols_keep = [
    "ads_idx", "ads_category", "ads_domain",
    "ads_contract_price", "ads_reward_price", "adv_idx",
]
if "ads_name" in ads_new_meta.columns:
    cols_keep.append("ads_name")
ads_new_meta = ads_new_meta[cols_keep].drop_duplicates("ads_idx").copy()

# 2) 예측 실행 (신규 광고만 투입)
preds_new = predict_all_mda_with_counts_batched(
    ranker=ranker,
    click_reg=click_reg,
    cvr_reg=cvr_reg,
    feat_cols=feat_cols,
    actx=actx,
    ads_list_with_domain=ads_new_meta,  # 신규 광고 배치
    K=20,
    batch_size=5000,
    max_predict_ads=None,
    media_portfolio=None
)

# 2-1) (선택) 콜드스타트 블렌딩 적용
APPLY_BLEND = True
if APPLY_BLEND:
    for ad in ads_new_meta["ads_idx"].astype(int):
        preds_new[ad] = easy_coldstart_blend_v2(
            preds_new.get(ad), actx, tgt=tgt,  # tgt 있으면 전달 (없어도 동작)
            cvr_cap=0.10,   # 보수적 컷
            w=0.7,          # 모델:prior = 70:30
            click_budget=None
        )

# 3) 광고별 Top-5 출력 (yhat_turn 기준)
def _show_new_ads_topk(preds_dict, ad_ids, k=5):
    for ad in list(map(int, ad_ids)):
        topk = get_expected_conversions_for_ad(preds_dict, ad_id=ad, sort_by="yhat_turn", top=k)
        print(f"\n=== 신규 광고 {ad} 추천 매체사 Top-{k} (yhat_turn 기준) ===")
        if topk.empty:
            print("[EMPTY] 이 광고는 예측 결과가 없습니다.")
        else:
            cols = ["mda_idx","yhat_turn","score"]
            print(topk[cols])

_show_new_ads_topk(preds_new, ads_new_meta["ads_idx"].tolist(), k=5)


[Predict+Count] processed ads 8/8

=== 신규 광고 500000 추천 매체사 Top-5 (yhat_turn 기준) ===
    mda_idx  yhat_turn     score
12      563   0.334154 -4.617729
4       634   0.131704 -3.947494
2        22   0.051424 -3.927919
11      761   0.045003 -4.553074
0       270   0.042408 -3.393299

=== 신규 광고 500001 추천 매체사 Top-5 (yhat_turn 기준) ===
   mda_idx  yhat_turn     score
0      563   2.379667  6.694795
2      761   2.021672  2.565953
8      645   0.372767 -1.693402
4      270   0.070684  0.118599
9      790   0.063358 -2.352940

=== 신규 광고 500002 추천 매체사 Top-5 (yhat_turn 기준) ===
   mda_idx  yhat_turn     score
2      371   3.155706  1.458189
0      270   2.641406  1.740088
1      761   1.119561  1.512758
4      817   0.493860  0.300374
7      645   0.466553 -0.615812

=== 신규 광고 500003 추천 매체사 Top-5 (yhat_turn 기준) ===
    mda_idx  yhat_turn     score
1        14  11.929184  2.734970
0        22   3.628440  3.898190
2       398   3.278044  2.230773
5       401   0.723096  0.638468
18      563   0.240

In [ ]:
ㅋㅋㅋㅋㅋㅋㅋ

NameError: name 'ᄏᄏᄏᄏᄏᄏᄏ' is not defined

In [ ]:
def _per_ad_labels_scores(preds_with_counts: dict, tgt_df: pd.DataFrame, score_col: str = "score"):
    """
    각 ads_idx마다 Top-K 예측(rows)에서의 (y_true, y_score) 배열을 뽑아줌.
    y_true는 해당 mda_idx가 GT에 포함되면 1, 아니면 0.
    """
    gt_pairs = (
        tgt_df.groupby("ads_idx")["mda_idx"]
        .apply(lambda s: list(pd.Series(s).dropna().astype(int).unique()))
        .to_dict()
    )

    out = {}
    for ad, df in preds_with_counts.items():
        true_mdas = set(gt_pairs.get(ad, []))
        if len(df) == 0:
            continue
        y_true = (df["mda_idx"].astype(int).isin(true_mdas)).astype(int).values
        y_score = df[score_col].astype(float).values
        # AUC 계산이 불가능한 올-양성/올-음성 케이스는 건너뜀
        if len(np.unique(y_true)) < 2:
            continue
        out[ad] = (y_true, y_score)
    return out

def summarize_auc(preds_with_counts: dict, tgt_df: pd.DataFrame, score_col: str = "score"):
    """
    per-ad ROC AUC / PR-AUC(average precision) 분포를 요약.
    score_col: "score"(랭커) 또는 "yhat_turn"(회귀)
    """
    per_ad = _per_ad_labels_scores(preds_with_counts, tgt_df, score_col=score_col)
    aucs, ap_aucs = [], []
    for ad, (y_true, y_score) in per_ad.items():
        try:
            aucs.append(roc_auc_score(y_true, y_score))
        except Exception:
            pass
        try:
            ap_aucs.append(average_precision_score(y_true, y_score))
        except Exception:
            pass

    def _stats(x):
        if not x:
            return {"count": 0, "mean": np.nan, "median": np.nan, "p25": np.nan, "p75": np.nan}
        x = np.array(x, float)
        return {
            "count": int(len(x)),
            "mean": float(np.mean(x)),
            "median": float(np.median(x)),
            "p25": float(np.percentile(x, 25)),
            "p75": float(np.percentile(x, 75)),
        }

    return {
        "per_ad_roc_auc": _stats(aucs),
        "per_ad_pr_auc": _stats(ap_aucs),
    }

# ======================
# 사용: (LARGE 기준이라면 tgt_large 사용)
# ======================

auc_ranker_detail   = summarize_auc(preds_with_counts, tgt_large, score_col="score")
auc_regressor_detail= summarize_auc(preds_with_counts, tgt_large, score_col="yhat_turn")

print("\n=== AUC (전체) @k=5 ===")
print(metrics)  # evaluate_all(...) 출력값에 auc_ranker/auc_regressor 포함됨

print("\n=== Per-Ad ROC-AUC / PR-AUC 분포 요약 (Ranker: score) ===")
print(auc_ranker_detail)

print("\n=== Per-Ad ROC-AUC / PR-AUC 분포 요약 (Regressor: yhat_turn) ===")
print(auc_regressor_detail)



=== AUC (전체) @k=5 ===
{'precision@5': 0.3594339622641509, 'recall@5': 0.7775031964890455, 'map@5': 0.6857106044723968, 'hit_rate@5': 0.9669811320754716, 'auc_ranker': 0.846276147728455, 'auc_regressor': 0.8298741516318361}

=== Per-Ad ROC-AUC / PR-AUC 분포 요약 (Ranker: score) ===
{'per_ad_roc_auc': {'count': 635, 'mean': 0.8930818655140802, 'median': 0.9607843137254902, 'p25': 0.8770604395604396, 'p75': 1.0}, 'per_ad_pr_auc': {'count': 635, 'mean': 0.7717749111304111, 'median': 0.8333333333333333, 'p25': 0.6875, 'p75': 1.0}}

=== Per-Ad ROC-AUC / PR-AUC 분포 요약 (Regressor: yhat_turn) ===
{'per_ad_roc_auc': {'count': 635, 'mean': 0.9191186645924249, 'median': 1.0, 'p25': 0.9217218137254901, 'p75': 1.0}, 'per_ad_pr_auc': {'count': 635, 'mean': 0.8494606869185638, 'median': 1.0, 'p25': 0.8048611111111111, 'p75': 1.0}}


In [ ]:
ad_id = 12345
print(get_expected_conversions_for_ad(preds_with_counts, ad_id, sort_by="yhat_turn", top=20))

Empty DataFrame
Columns: [mda_idx, score, yhat_turn]
Index: []


In [ ]:
# ======================================
# Top-10 정/오예측 비교 분석 유틸
# ======================================

def build_gt_pairs(tgt_df: pd.DataFrame) -> dict:
    # ads_idx별 정답 mda_idx 집합
    return (
        tgt_df.groupby("ads_idx")["mda_idx"]
        .apply(lambda s: list(pd.Series(s).dropna().astype(int).unique()))
        .to_dict()
    )

def make_cmp_df(preds_with_counts: dict, gt_pairs: dict, topk: int = 10) -> pd.DataFrame:
    records = []
    for ad, df in preds_with_counts.items():
        if ad not in gt_pairs:
            continue
        true_set = set(gt_pairs[ad])
        # 상위 K만 평가
        for pos, row in enumerate(df.head(topk).itertuples(index=False), start=1):
            mda = int(row.mda_idx)
            score = float(row.score)
            yhat_turn = float(row.yhat_turn)
            is_correct = 1 if mda in true_set else 0
            records.append({
                "ads_idx": int(ad),
                "mda_idx": mda,
                "rank": pos,
                "score": score,
                "yhat_turn": yhat_turn,
                "is_correct": is_correct
            })
    return pd.DataFrame.from_records(records)

def compare_correct_vs_incorrect(cmp_df: pd.DataFrame,
                                 actx: dict,
                                 ads_list_large: pd.DataFrame,
                                 media_portfolio: pd.DataFrame | None = None) -> dict:
    out = {}

    # ---- 메타 병합 (매체 / 광고)
    # mda 전역 특성
    mda_all = actx["mda_all"][["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]].copy()
    mda_all["mda_idx"] = mda_all["mda_idx"].astype(int)
    cmpm = cmp_df.merge(mda_all, on="mda_idx", how="left")

    # 광고 메타(카테고리/가격)
    ad_meta_cols = [c for c in ["ads_idx","ads_category","ads_reward_price","ads_contract_price"] if c in ads_list_large.columns]
    if "ads_category" in ad_meta_cols:
        ad_meta = ads_list_large[ad_meta_cols].drop_duplicates("ads_idx").copy()
        cmpm = cmpm.merge(ad_meta, on="ads_idx", how="left")

    # (선택) media_portfolio 병합
    if media_portfolio is not None:
        cols = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in media_portfolio.columns]
        if "mda_idx" in cols:
            cmpm = cmpm.merge(media_portfolio[cols], on="mda_idx", how="left")

    out["cmp_df"] = cmpm

    # ---- 전체 요약
    total = len(cmpm)
    correct = int(cmpm["is_correct"].sum())
    prec_at_k = correct / total if total else 0.0
    out["summary"] = {
        "samples": total,
        "correct": correct,
        "incorrect": total - correct,
        "precision_empirical": prec_at_k
    }

    # ---- 분포 비교: score / yhat_turn / mda 지표
    num_cols = [c for c in ["score","yhat_turn","mda_cvr_global","mda_pop_norm","mda_clk_global","mda_conv_global",
                            "mda_size","active_ratio","day_ratio","weekend_ratio"]
                if c in cmpm.columns]
    out["by_label_describe"] = cmpm.groupby("is_correct")[num_cols].describe().transpose()

    # ---- 순위 포지션별 정답률
    out["rank_position_accuracy"] = (
        cmpm.groupby("rank")["is_correct"].mean().rename("accuracy_by_rank").to_frame()
    )

    # ---- 카테고리별 정/오예측 비율
    if "ads_category" in cmpm.columns:
        out["category_correct_rate"] = (
            cmpm.groupby("ads_category")["is_correct"].mean().sort_values(ascending=False).to_frame("correct_rate")
        )
        out["category_counts"] = (
            cmpm.groupby(["ads_category","is_correct"]).size().unstack(fill_value=0)
            .rename(columns={0:"incorrect",1:"correct"}).sort_values("correct", ascending=False)
        )

    # ---- 매체사별 오예측 특성 (False Positive가 많은 mda)
    mda_stats = cmpm.groupby("mda_idx").agg(
        total=("is_correct","size"),
        correct=("is_correct","sum"),
        score_mean=("score","mean"),
        yhat_mean=("yhat_turn","mean"),
        mda_cvr_global=("mda_cvr_global","mean"),
        mda_pop_norm=("mda_pop_norm","mean")
    )
    mda_stats["incorrect"] = mda_stats["total"] - mda_stats["correct"]
    mda_stats["correct_rate"] = mda_stats["correct"] / mda_stats["total"]
    out["mda_false_positive_hotlist"] = (
        mda_stats.sort_values(["incorrect","correct_rate"], ascending=[False, True]).head(30)
    )

    # ---- 광고별 커버리지 (Top-10에 정답 존재 여부)
    ad_hit = cmpm.groupby("ads_idx")["is_correct"].max().rename("has_any_correct_in_top10")
    out["ad_hit_rate"] = {
        "ads_with_any_correct_top10": int(ad_hit.sum()),
        "ads_total": int(ad_hit.size),
        "rate": float(ad_hit.mean()) if ad_hit.size else 0.0
    }

    # ---- 정/오예측 평균 차이표 (간단한 mean gap)
    def _mean_gap(col):
        if col not in cmpm.columns: return None
        g = cmpm.groupby("is_correct")[col].mean()
        return pd.Series({
            "correct_mean": g.get(1, np.nan),
            "incorrect_mean": g.get(0, np.nan),
            "gap(correct - incorrect)": g.get(1, np.nan) - g.get(0, np.nan)
        })
    key_cols = [c for c in ["score","yhat_turn","mda_cvr_global","mda_pop_norm","mda_clk_global","mda_conv_global",
                            "mda_size","active_ratio","day_ratio","weekend_ratio"]
                if c in cmpm.columns]
    gap_tbl = pd.concat({c: _mean_gap(c) for c in key_cols}, axis=1).T.sort_values("gap(correct - incorrect)", ascending=False)
    out["mean_gaps"] = gap_tbl

    return out

# =========================
# 실행 (K=5 기준)
# =========================

# 1) GT 생성
gt_pairs = build_gt_pairs(tgt_large if 'tgt_large' in globals() else tgt)

# 2) 비교용 DF 생성 (★ topk=5로 변경)
cmp_df_k5 = make_cmp_df(preds_with_counts, gt_pairs, topk=5)

# 3) 비교 분석 실행
analysis_k5 = compare_correct_vs_incorrect(
    cmp_df=cmp_df_k5,
    actx=actx,
    ads_list_large=ads_list_large if 'ads_list_large' in globals() else ads_list,
    media_portfolio=None  # 있으면 넣기
)

# 4) 주요 결과 출력
print("\n=== SUMMARY (Top-5) ===")
print(analysis_k5["summary"])

print("\n=== BY LABEL DESCRIBE (score, yhat_turn, mda metrics) ===")
print(analysis_k5["by_label_describe"])

print("\n=== RANK POSITION ACCURACY (Top-5 내 순위별 정답률) ===")
print(analysis_k5["rank_position_accuracy"])

if "category_correct_rate" in analysis_k5:
    print("\n=== CATEGORY CORRECT RATE (정답률 높은 카테고리) ===")
    print(analysis_k5["category_correct_rate"].head(20))

    print("\n=== CATEGORY COUNTS (정/오예측 건수) ===")
    print(analysis_k5["category_counts"].head(20))

print("\n=== MDA FALSE POSITIVE HOTLIST (오예측 많이 발생한 매체 Top-30) ===")
print(analysis_k5["mda_false_positive_hotlist"].head(30))

print("\n=== MEAN GAPS (정/오예측 간 평균 차이) ===")
print(analysis_k5["mean_gaps"])




=== SUMMARY (Top-5) ===
{'samples': 3180, 'correct': 1143, 'incorrect': 2037, 'precision_empirical': 0.3594339622641509}

=== BY LABEL DESCRIBE (score, yhat_turn, mda metrics) ===
is_correct                        0             1
score           count  2.037000e+03  1.143000e+03
                mean   1.318327e+00  4.114461e+00
                std    1.455441e+00  2.777740e+00
                min   -4.589903e+00 -1.453916e+00
                25%    6.244200e-01  2.309047e+00
                50%    9.423745e-01  3.414585e+00
                75%    2.658476e+00  7.376601e+00
                max    6.702010e+00  7.376601e+00
yhat_turn       count  2.037000e+03  1.143000e+03
                mean   1.564323e+00  1.634562e+01
                std    2.634145e+00  2.116598e+01
                min    0.000000e+00  0.000000e+00
                25%    5.201536e-01  1.725321e+00
                50%    9.163970e-01  3.993635e+00
                75%    2.365714e+00  2.724357e+01
                max

In [ ]:
# k=5 기준 정/오예측 분석
analysis_k5 = analyze_correct_incorrect_at_k(preds_with_counts, tgt_large, k=5)

print("=== SUMMARY ===")
print(analysis_k5["summary"])

print("\n=== BY LABEL DESCRIBE ===")
print(analysis_k5["by_label"])

print("\n=== RANK POSITION ACCURACY ===")
print(analysis_k5["acc_by_rank"])

print("\n=== CATEGORY CORRECT RATE ===")
print(analysis_k5["cat_rate"].head(10))

print("\n=== CATEGORY COUNTS ===")
print(analysis_k5["cat_counts"])

print("\n=== MDA FALSE POSITIVE HOTLIST ===")
print(analysis_k5["mda_hotlist"])

print("\n=== MEAN GAPS ===")
print(analysis_k5["mean_gaps"])


=== SUMMARY ===
{'samples': 3180, 'correct': 1143, 'incorrect': 2037, 'precision_empirical': np.float64(0.3594339622641509)}

=== BY LABEL DESCRIBE ===
is_correct                 0            1
score     count  2037.000000  1143.000000
          mean      1.318327     4.114461
          std       1.455441     2.777740
          min      -4.589903    -1.453916
          25%       0.624420     2.309047
          50%       0.942375     3.414585
          75%       2.658476     7.376601
          max       6.702010     7.376601
yhat_turn count  2037.000000  1143.000000
          mean      1.564323    16.345621
          std       2.634145    21.165977
          min       0.000000     0.000000
          25%       0.520154     1.725321
          50%       0.916397     3.993635
          75%       2.365714    27.243571
          max      68.365025   134.024780

=== RANK POSITION ACCURACY ===
      accuracy_by_rank
rank                  
0             0.886792
1             0.160377
2         

In [ ]:
# ads_list_large에 ads_category 컬럼이 있는지 확인
print("컬럼 목록:", ads_list_large.columns.tolist())

# ads_category 값 확인
if "ads_category" in ads_list_large.columns:
    print("\n=== ads_list_large['ads_category'] 샘플 ===")
    print(ads_list_large["ads_category"].head(20))

    print("\n=== NaN/Null 개수 ===")
    print(ads_list_large["ads_category"].isna().sum(), "/", len(ads_list_large))

    print("\n=== 고유값 개수 ===")
    print(ads_list_large["ads_category"].nunique(dropna=True))

    print("\n=== 고유값 Top-20 ===")
    print(ads_list_large["ads_category"].value_counts(dropna=False).head(20))
else:
    print("⚠️ ads_category 컬럼이 존재하지 않습니다.")


컬럼 목록: ['ads_idx', 'ads_code', 'aff_idx', 'adv_idx', 'ads_type', 'ads_category', 'ads_name', 'ads_icon_img', 'ads_summary', 'ads_save_way', 'ads_day_cap', 'ads_sdate', 'ads_edate', 'ads_age_min', 'ads_age_max', 'ads_os_type', 'ads_contract_price', 'ads_reward_price', 'ads_order', 'ads_rejoin_type', 'regdate', 'ads_size']

=== ads_list_large['ads_category'] 샘플 ===
8287      8
9762      3
11202     8
11488     8
12196     1
12338     8
12666    10
14762     8
14767     8
15128    10
18449    10
18451    10
19227     8
19632     3
20412     8
20677     8
23121     5
24591     8
25847    10
26185     7
Name: ads_category, dtype: int64

=== NaN/Null 개수 ===
0 / 2643

=== 고유값 개수 ===
12

=== 고유값 Top-20 ===
ads_category
4     2472
8       48
5       34
10      21
11      15
1       14
2       12
0        8
6        7
3        5
7        4
13       3
Name: count, dtype: int64


In [ ]:
def build_simple_features(ads_time, ads_list, ads_domain):
    # 1) 매체사별 전역 클릭수 (과거 성과의 세부 페어 데이터 제외)
    mda_perf = (
        ads_time.groupby("mda_idx", as_index=False)
                .agg(mda_clicks=("rpt_time_clk", "sum"))
    )

    # 2) 광고 메타
    ad_meta = ads_list[["ads_idx", "ads_category", "ads_os_type",
                        "ads_contract_price", "ads_reward_price"]].copy()

    # 3) 광고 도메인
    ad_domain = ads_domain[["ads_idx", "domain"]].copy()

    # 4) 광고 × 매체 cross-join + 피처 결합
    X_simple = (
        ad_meta
        .merge(ad_domain, on="ads_idx", how="left")
        .assign(_key=1)
        .merge(mda_perf.assign(_key=1), on="_key", how="left")
        .drop(columns="_key")
    )

    # 선택: 카테고리형 정리 (LightGBM 등에서 바로 사용하려면)
    for c in ["ads_category", "ads_os_type", "domain"]:
        if c in X_simple.columns:
            X_simple[c] = X_simple[c].astype("category")

    return X_simple

# === 실행 ===
X_simple = build_simple_features(ads_time, ads_list, ads_domain)
# 확인
# X_simple.head()


In [ ]:
X_simple

,ads_idx,ads_category,ads_os_type,ads_contract_price,ads_reward_price,domain,mda_idx,mda_clicks
0,160,1,2,170,150,NaN,12,224440
1,160,1,2,170,150,NaN,14,84025
2,160,1,2,170,150,NaN,18,5776
3,160,1,2,170,150,NaN,22,80177
4,160,1,2,170,150,NaN,26,670
...,...,...,...,...,...,...,...,...
84112366,446918,8,7,300,225,미디어/컨텐츠,1050,1
84112367,446918,8,7,300,225,미디어/컨텐츠,1051,1
84112368,446918,8,7,300,225,미디어/컨텐츠,1055,3
84112369,446918,8,7,300,225,미디어/컨텐츠,1059,6


In [ ]:
# ==============================================
# Attr-only Ranking with Rich Features (No Pair History)
# Split: OBS(학습)=최근7일 이전 전체 / TGT(테스트)=최근7일
# LARGE 광고만 대상, 후보 Top-M, 다운샘플링, Graded relevance
# ==============================================

# -----------------------------
# 0) 유틸
# -----------------------------
def _to_num(s): return pd.to_numeric(s, errors="coerce").fillna(0)
def _to_dt(s):  return pd.to_datetime(s, errors="coerce")

def _mode(s: pd.Series):
    if s.isna().all(): return np.nan
    m = s.mode()
    return m.iloc[0] if len(m) else np.nan

def _norm_cat(s):
    return pd.Series(s, copy=False).astype("string").str.strip().str.lower()

def _pick_col(cols, cands):
    for c in cands:
        if c in cols: return c
    return None

# -----------------------------
# 1) 날짜 스플릿 (최근 7일 TEST)
# -----------------------------
ads_time = ads_time.copy()
ads_time["rpt_time_date"] = _to_dt(ads_time["rpt_time_date"])
max_date = ads_time["rpt_time_date"].max()
cutoff   = (max_date - pd.Timedelta(days=7)).normalize()
print(f"Cutoff: {cutoff.date()} ~ {max_date.date()}")

ads_time_tr = ads_time[ads_time["rpt_time_date"] <= cutoff].copy()   # OBS
ads_time_te = ads_time[ads_time["rpt_time_date"] >  cutoff].copy()   # TGT

# -----------------------------
# 2) LARGE 광고만 대상으로 df_base 제한
# -----------------------------
large_ads = ads_segment.loc[ads_segment["ads_size"]=="LARGE","ads_idx"].unique()
df_base   = X_simple[X_simple["ads_idx"].isin(large_ads)].copy()

# -----------------------------
# 3) 라벨 (TGT/OBS 각각)
# -----------------------------
label_tr = (ads_time_tr.groupby(["ads_idx","mda_idx"], as_index=False)
            .agg(mda_convs=("rpt_time_turn","sum")))
label_te = (ads_time_te.groupby(["ads_idx","mda_idx"], as_index=False)
            .agg(mda_convs=("rpt_time_turn","sum")))

# -----------------------------
# 4) ATTR 컨텍스트 (OBS만 사용) - 1~5번 피처의 재료
# -----------------------------
def build_attr_context(obs: pd.DataFrame, ads_list: pd.DataFrame):
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["conv"] = _to_num(t.get("rpt_time_turn", 0))

    # 1) 매체 전반
    mda_all = (t.groupby("mda_idx", as_index=False)
                 .agg(mda_clk_global=("clk","sum"),
                      mda_conv_global=("conv","sum")))
    mda_all["mda_cvr_global"] = (mda_all["mda_conv_global"] /
                                 mda_all["mda_clk_global"].replace(0, np.nan)).fillna(0.0)
    mu = mda_all["mda_conv_global"].mean()
    sd = mda_all["mda_conv_global"].std(ddof=0)
    mda_all["mda_pop_norm"] = (mda_all["mda_conv_global"] - mu) / (sd + 1e-9)

    # 광고 카테고리 붙여서 2) 카테고리×매체
    al = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    al["ads_category"] = _norm_cat(al["ads_category"])
    t2 = t.merge(al, on="ads_idx", how="left")
    cat_mda = (t2.groupby(["mda_idx","ads_category"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    cat_mda["cat_cvr"] = (cat_mda["conv"] / cat_mda["clk"].replace(0, np.nan)).fillna(0.0)
    mda_tot = cat_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    cat_mda = cat_mda.merge(mda_tot, on="mda_idx", how="left")
    cat_mda["cat_share"] = (cat_mda["conv"] /
                            cat_mda["mda_conv_total"].replace(0, np.nan)).fillna(0.0)
    cat_mda.drop(columns=["mda_conv_total"], inplace=True)

    # 4) 도메인×매체
    dom_col = _pick_col(t.columns, ["req_domain","domain","site_domain","publisher_domain","ads_domain"])
    dom_mda = None
    if dom_col is not None:
        t_dom = t.copy()
        t_dom[dom_col] = _norm_cat(t_dom[dom_col])
        dom_mda = (t_dom.groupby(["mda_idx", dom_col], as_index=False)
                        .agg(clk=("clk","sum"), conv=("conv","sum")))
        dom_mda["dom_cvr"] = (dom_mda["conv"] / dom_mda["clk"].replace(0, np.nan)).fillna(0.0)
        mt2 = dom_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
        dom_mda = dom_mda.merge(mt2, on="mda_idx", how="left")
        dom_mda["dom_share"] = (dom_mda["conv"] /
                                dom_mda["mda_conv_total"].replace(0, np.nan)).fillna(0.0)
        dom_mda = dom_mda.rename(columns={dom_col: "ads_domain"})
        dom_mda.drop(columns=["mda_conv_total"], inplace=True)

    # 5) OS×매체
    os_col = _pick_col(t.columns, ["os_type","device_os","os","ads_os_type"])
    os_mda = None
    if os_col is not None:
        t_os = t.copy()
        t_os[os_col] = _norm_cat(t_os[os_col])
        os_mda = (t_os.groupby(["mda_idx", os_col], as_index=False)
                        .agg(clk=("clk","sum"), conv=("conv","sum")))
        os_mda["os_cvr"] = (os_mda["conv"] / os_mda["clk"].replace(0, np.nan)).fillna(0.0)
        mt3 = os_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
        os_mda = os_mda.merge(mt3, on="mda_idx", how="left")
        os_mda["os_share"] = (os_mda["conv"] /
                              os_mda["mda_conv_total"].replace(0, np.nan)).fillna(0.0)
        os_mda = os_mda.rename(columns={os_col: "os_type"})
        os_mda.drop(columns=["mda_conv_total"], inplace=True)

    return {"mda_all": mda_all, "cat_mda": cat_mda, "dom_mda": dom_mda, "os_mda": os_mda}

actx = build_attr_context(ads_time_tr, ads_list)

# -----------------------------
# 5) 광고 가격/속성 파생 (3번 피처)
# -----------------------------
ad_meta = ads_list[["ads_idx","ads_category","ads_os_type",
                    "ads_contract_price","ads_reward_price"]].drop_duplicates("ads_idx").copy()
ad_meta["ads_category"] = _norm_cat(ad_meta["ads_category"])
ad_meta["os_type"] = _norm_cat(ad_meta.get("ads_os_type", np.nan))
ad_meta["ads_reward_price"]   = _to_num(ad_meta["ads_reward_price"])
ad_meta["ads_contract_price"] = _to_num(ad_meta["ads_contract_price"])
ad_meta["price_reward_log"]   = np.log1p(ad_meta["ads_reward_price"])
ad_meta["price_contract_log"] = np.log1p(ad_meta["ads_contract_price"])
ad_meta["price_ratio"]        = (ad_meta["ads_reward_price"] /
                                 ad_meta["ads_contract_price"].replace(0, np.nan)).fillna(0.0)

# 광고 도메인(있으면) 정규화
ad_dom_col = _pick_col(ads_list.columns, ["ads_domain","domain","site_domain","publisher_domain"])
if ad_dom_col is not None:
    ad_meta["ads_domain"] = _norm_cat(ads_list[[ad_dom_col,"ads_idx"]]
                                      .drop_duplicates("ads_idx")[ad_dom_col])
else:
    ad_meta["ads_domain"] = "__na__"

# -----------------------------
# (선택) 6) media_portfolio 병합
# -----------------------------
def attach_media_portfolio(df, media_portfolio):
    if media_portfolio is None:  # 없으면 0/UNKNOWN 채움
        for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
            df[c] = 0.0
        return df
    mp = media_portfolio.copy()
    keep = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp.columns]
    if "mda_idx" not in keep:
        for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
            df[c] = 0.0
        return df
    df = df.merge(mp[keep], on="mda_idx", how="left")
    for c in ["active_ratio","day_ratio","weekend_ratio"]:
        if c in df.columns:
            df[c] = _to_num(df[c])
        else:
            df[c] = 0.0
    if "mda_size" not in df.columns:
        df["mda_size"] = 0.0
    return df

# -----------------------------
# 7) Train/Test 빌드 + 피처 결합(1~6)
# -----------------------------
def build_featured(df_base, label, ad_meta, actx, media_portfolio=None):
    # 기본(광고×매체)
    df = df_base.merge(label, on=["ads_idx","mda_idx"], how="left")
    df["mda_convs"] = _to_num(df["mda_convs"])
    # 광고 속성 붙이기
    df = df.merge(ad_meta[["ads_idx","ads_category","os_type","ads_reward_price","ads_contract_price",
                           "price_reward_log","price_contract_log","price_ratio","ads_domain"]],
                  on="ads_idx", how="left")

    # 1) 매체 전반
    mda_all = actx["mda_all"]
    df = df.merge(mda_all, on="mda_idx", how="left")

    # 2) 카테고리×매체
    cat_mda = actx["cat_mda"].copy()
    df["ads_category"] = _norm_cat(df["ads_category"])
    cat_mda["ads_category"] = _norm_cat(cat_mda["ads_category"])
    df = df.merge(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]],
                  on=["mda_idx","ads_category"], how="left")

    # 4) 도메인×매체
    if actx["dom_mda"] is not None:
        dom_mda = actx["dom_mda"].copy()
        dom_mda["ads_domain"] = _norm_cat(dom_mda["ads_domain"])
        df["ads_domain"] = _norm_cat(df["ads_domain"])
        df = df.merge(dom_mda[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                      on=["mda_idx","ads_domain"], how="left")
    else:
        df["dom_cvr"]=0.0; df["dom_share"]=0.0

    # 5) OS×매체
    if actx["os_mda"] is not None:
        os_mda = actx["os_mda"].copy()
        os_mda["os_type"] = _norm_cat(os_mda["os_type"])
        df["os_type"]     = _norm_cat(df["os_type"])
        df = df.merge(os_mda[["mda_idx","os_type","os_cvr","os_share"]],
                      on=["mda_idx","os_type"], how="left")
    else:
        df["os_cvr"]=0.0; df["os_share"]=0.0

    # 6) media_portfolio
    df = attach_media_portfolio(df, media_portfolio)

    # 결측 처리
    for c in ["mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm",
              "cat_cvr","cat_share","dom_cvr","dom_share","os_cvr","os_share"]:
        if c in df.columns: df[c] = df[c].fillna(0.0)

    # graded relevance (0,1,2,3)
    y = df["mda_convs"].astype(float)
    rel = np.zeros(len(df), dtype=int)
    pos = y > 0
    if pos.sum() > 0:
        # 상위 전환을 더 높은 등급으로
        ylog = np.log1p(y[pos])
        try:
            bins = pd.qcut(ylog, q=3, labels=False, duplicates="drop") # 3분위 → 1..3
            rel[pos] = bins.astype(int) + 1
        except ValueError:
            rel[pos] = 1
    df["relevance"] = rel

    # prior feature
    df["prior_expected"] = df["mda_clk_global"] * df["mda_cvr_global"]
    df["prior_expected_log1p"] = np.log1p(df["prior_expected"])

    # 라벨 보존
    df["label_cls"] = (df["mda_convs"] > 0).astype(int)
    df["label_reg"] = y
    return df

train_df = build_featured(df_base, label_tr, ad_meta, actx, media_portfolio=None)
test_df  = build_featured(df_base, label_te, ad_meta, actx, media_portfolio=None)

print("Train size:", len(train_df), " Test size:", len(test_df))

# -----------------------------
# 8) 후보 Top-M 축소 (학습/평가 공통)
# -----------------------------
def restrict_topM(df, M=150):
    return (df.sort_values(["ads_idx","prior_expected"], ascending=[True,False])
              .groupby("ads_idx", group_keys=False).head(M))

train_df = restrict_topM(train_df, M=150)
test_df  = restrict_topM(test_df,  M=150)

# -----------------------------
# 9) 다운샘플링 (광고별 neg:pos ≈ 4:1)
# -----------------------------
def downsample_per_ad(df, ratio=4, seed=42):
    rng = np.random.RandomState(seed)
    out = []
    for ad, g in df.groupby("ads_idx", sort=False):
        pos = g[g["label_cls"]==1]
        neg = g[g["label_cls"]==0]
        if len(pos)==0:
            continue
        cap = len(pos) * ratio
        if len(neg) > cap:
            neg = neg.sample(cap, random_state=rng)
        out.append(pd.concat([pos,neg], ignore_index=True))
    return pd.concat(out, ignore_index=True) if out else df

train_df = downsample_per_ad(train_df, ratio=4)

# -----------------------------
# 10) 학습 준비
# -----------------------------
FEATS = [
    # 1) 매체 전반
    "mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm",
    # 2) 카테고리×매체
    "cat_cvr","cat_share",
    # 3) 광고 가격/속성
    "price_reward_log","price_contract_log","price_ratio",
    # 4) 도메인×매체
    "dom_cvr","dom_share",
    # 5) OS×매체
    "os_cvr","os_share",
    # 6) media_portfolio (없으면 0)
    "mda_size","active_ratio","day_ratio","weekend_ratio",
    # prior feature
    "prior_expected","prior_expected_log1p"
]
def ensure_X(df, cols):
    X = df.copy()
    for c in cols:
        if c not in X.columns:
            X[c] = 0.0
        X[c] = _to_num(X[c])
    return X[cols].astype(np.float32)

Xtr = ensure_X(train_df, FEATS)
Xte = ensure_X(test_df,  FEATS)
ytr = train_df["relevance"].astype(int).values
yte = test_df["relevance"].astype(int).values
group_tr = train_df.groupby("ads_idx").size().astype(int).tolist()
group_te = test_df.groupby("ads_idx").size().astype(int).tolist()

# -----------------------------
# 11) Ranker (LambdaRank, graded relevance)
# -----------------------------
ranker = lgb.LGBMRanker(
    objective="lambdarank", metric="ndcg", eval_at=[5,10,20],
    learning_rate=0.05, n_estimators=1200, num_leaves=63,
    min_data_in_leaf=40, subsample=0.85, colsample_bytree=0.85,
    label_gain=[0,1,4,12], random_state=42, verbosity=-1
)
ranker.fit(
    Xtr, ytr, group=group_tr,
    eval_set=[(Xte, yte)], eval_group=[group_te]
)

test_df["pred_score"] = ranker.predict(Xte)

# -----------------------------
# 12) Top-K 추천 & 평가
# -----------------------------
final_rec = (test_df.sort_values(["ads_idx","pred_score"], ascending=[True,False])
                    .groupby("ads_idx", group_keys=False).head(20)
                    .reset_index(drop=True))

def _avg_precision_at_k(labels_sorted, K):
    lab = np.asarray(labels_sorted)[:K]
    if lab.sum()==0: return 0.0
    precisions, hits = [], 0
    for i,y in enumerate(lab, start=1):
        if y==1:
            hits += 1
            precisions.append(hits/i)
    return float(np.mean(precisions)) if precisions else 0.0

def _precision_at_k(labels_sorted, K):
    lab = np.asarray(labels_sorted)[:K]
    return float(lab.mean()) if len(lab) else 0.0

def _recall_at_k(labels_sorted, K, total_pos):
    if total_pos==0: return 0.0
    lab = np.asarray(labels_sorted)[:K]
    return float(lab.sum()/total_pos)

def evaluate_ranking(df_scored, use_col="pred_score", ks=(5,10,20)):
    out = {}
    g = df_scored.sort_values(["ads_idx", use_col], ascending=[True, False]).groupby("ads_idx")
    for K in ks:
        P,R,AP = [],[],[]
        for _, grp in g:
            y_sorted = grp["label_cls"].values
            total_pos = int(grp["label_cls"].sum())
            P.append(_precision_at_k(y_sorted, K))
            R.append(_recall_at_k(y_sorted, K, total_pos))
            AP.append(_avg_precision_at_k(y_sorted, K))
        out[f"P@{K}"]   = float(np.mean(P)) if P else np.nan
        out[f"R@{K}"]   = float(np.mean(R)) if R else np.nan
        out[f"MAP@{K}"] = float(np.mean(AP)) if AP else np.nan
    return out

metrics = evaluate_ranking(final_rec[["ads_idx","label_cls","pred_score"]].copy())
print("=== Final (Attr-only + Rich Features) ===")
print(metrics)


Cutoff: 2025-08-18 ~ 2025-08-25


KeyError: 'ads_category'

In [ ]:
show_ad_all(57056).head(50)

NameError: name 'show_ad_all' is not defined

In [ ]:
# test 데이터셋에서 실제 라벨과 예측값 준비
y_true = test_df["label_cls"].values
y_score = test_df["pred_convs"].values   # 최종 예측 전환수 기반 점수

try:
    auc = roc_auc_score(y_true, y_score)
except ValueError:
    auc = np.nan   # 만약 y_true가 전부 0/1 한쪽으로만 치우친 경우
print("=== AUC (Final pred_convs) ===")
print(auc)


In [ ]:
def show_ad_topk(ad_id, K=5):
    plist = pred.get(ad_id, [])[:K]
    pred_df = pd.DataFrame(plist, columns=["mda_idx","pred_convs"])   # 🔹 pred_convs 포함
    
    # 실제 전환 매체 set
    true_set = gt.get(ad_id, set())
    
    # 실제 전환수 맵핑
    true_conv_map = (test_df.loc[test_df["ads_idx"]==ad_id, ["mda_idx","label_reg"]]
                               .drop_duplicates()
                               .set_index("mda_idx")["label_reg"]
                               .to_dict())
    pred_df["true_convs"] = pred_df["mda_idx"].map(true_conv_map).fillna(0).astype(int)
    pred_df["is_true"] = pred_df["mda_idx"].isin(true_set)

    print(f"\n광고 ID: {ad_id}")
    print(f"예측 Top-{K}: {pred_df['mda_idx'].tolist()}")
    print(f"실제 전환 매체: {sorted(list(true_set))}")

    return pred_df[["mda_idx","pred_convs","true_convs","is_true"]]


In [ ]:
show_ad_topk(57056, K=5)


In [ ]:
# 광고별 Top-K 추천 (예: K=10)
K = 10
topk_rec = (
    final_rec.sort_values(["ads_idx","pred_convs","rank_score"], ascending=[True,False,False])
             .groupby("ads_idx", group_keys=False)
             .head(K)
             .reset_index(drop=True)
)

# 보기 좋게 출력
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)

topk_rec.head(5)   # 여러 광고 섞여서 상위 30행만 미리보기


In [ ]:
# --------------------------------------------
# 준비: test_df(라벨)와 final_rec(예측결과)에서
#  - 광고별 실제 전환 매체 리스트(gt)
#  - 광고별 예측 Top-K 리스트(preds: (mda_idx, pred_convs))
# --------------------------------------------
def build_ground_truth(test_df):
    # 실제 전환된 매체: label_cls == 1
    gt = (
        test_df.loc[test_df["label_cls"] == 1, ["ads_idx", "mda_idx"]]
               .groupby("ads_idx")["mda_idx"].apply(set)
               .to_dict()
    )
    return gt

def build_predictions(final_rec, K=10):
    # 최종 결과에서 광고별 Top-K (pred_convs 우선, 동점 시 rank_score)
    preds = {}
    ordered = (final_rec
               .sort_values(["ads_idx", "pred_convs", "rank_score"], ascending=[True, False, False])
               .groupby("ads_idx", group_keys=False)
               .head(K))
    for ad, grp in ordered.groupby("ads_idx"):
        preds[ad] = list(zip(grp["mda_idx"].tolist(), grp["pred_convs"].tolist()))
    return preds

gt   = build_ground_truth(test_df)
pred = build_predictions(final_rec, K=10)

# --------------------------------------------
# 보기: 특정 광고(ad_id) 대해 표와 비교 세트 출력
# --------------------------------------------
def show_ad_topk(ad_id, K=10):
    # 예측 Top-K
    plist = pred.get(ad_id, [])[:K]
    pred_mda = [m for m,_ in plist]
    pred_df = pd.DataFrame(plist, columns=["mda_idx","pred_convs"])
    # 실제 전환 매체 set
    true_set = gt.get(ad_id, set())
    # 맞춘/놓친
    hits      = set(pred_mda) & true_set
    miss_pred = set(pred_mda) - true_set
    miss_true = true_set - set(pred_mda)
    # 표에 실제 전환 여부/실제 전환수(있으면) 추가
    if "mda_convs" in test_df.columns:
        true_conv_map = (test_df.loc[test_df["ads_idx"]==ad_id, ["mda_idx","mda_convs"]]
                              .drop_duplicates()
                              .set_index("mda_idx")["mda_convs"]
                              .to_dict())
        pred_df["true_convs"] = pred_df["mda_idx"].map(true_conv_map).fillna(0).astype(int)
    pred_df["is_true"] = pred_df["mda_idx"].isin(true_set)

    print(f"\n광고 ID: {ad_id}")
    print(f"예측 Top-{K}: {[m for m,_ in plist]}")
    print(f"실제 전환 매체: {sorted(list(true_set))}")
    print("맞춘 매체 :", hits)
    print("추천했지만 실패 :", miss_pred)
    print("놓친 실제 매체 :", miss_true)

    # 예측표 보기 좋게
    display_cols = ["mda_idx","pred_convs","is_true"]
    if "true_convs" in pred_df.columns:
        display_cols.insert(2,"true_convs")
    return pred_df[display_cols]

# 예시 실행
# ad_id = list(pred.keys())[0]   # 하나 골라보기
# show_ad_topk(ad_id, K=10)


In [ ]:
ad_id = list(pred.keys())[0]   # 하나 골라보기
show_ad_topk(ad_id, K=10)